# Introduzione

Se non già presenti, installiamo i pacchetti python relativi all'autenticazione e all'utilizzo dei servizi di Google. 

In [23]:
!pip install google-auth google-auth-oauthlib googleapiclient

I0000 00:00:1724672501.674395    5594 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1724672501.674878    5594 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient


Importiamo tutti i metodi e le librerie che ci serviranno nel corso dei test.

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai
import vertexai.preview.generative_models as generative_models

Creiamo un'istanza del modello, ossia Gemini Pro. 

In [ ]:
vertexai.init(project="<PROJECT-ID>", location="<REGION>")
model = GenerativeModel("gemini-1.5-pro-001")

Settiamo due configurazioni diverse per quel che riguarda la scelta dei token e i filtri di sicurezza. Questi andranno inseriti in ogni  prompt. 

In [26]:
generation_config1 = {
    "max_output_tokens": 8192,
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 32,
    "candidate_count":1
}

generation_config2 = {
    "max_output_tokens": 8192,
    "temperature": 0.9,
    "top_p": 1.0,
    "top_k": 32,
    "candidate_count":1
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

Creaimo il Rag corpus. Il seguente passaggio crea la base di dati personalizzata a cui si attingerà con le retrieval query partendo dai libri e gli esercizi inerenti al corso presenti su Google Drive. 

In [ ]:
display_name = "reti_logiche"

textbook_path_it = ["<TEXTBOOK_PATH_IT>"]

other_sources_paths_it = ["<OTHER_SOURCES_PATH_IT>"]

rag_corpus = rag.create_corpus(display_name=display_name)

all_file_paths = textbook_path_it + other_sources_paths_it

#il massimo che si può importare per volta è 25 files
file_path_groups = [all_file_paths[i:i+25] for i in range(0, len(all_file_paths), 25)]

for group in file_path_groups:
    response = rag.import_files(
        rag_corpus.name,
        group,
        chunk_size=512,  # Optional
        chunk_overlap=100,  # Optional
    )

Creiamo infine le funzioni con cui interrogheremo Gemini, sia in modalità multimodale che di solo testo, e con cui recuperemo le informazioni di contesto mediante RAG. 

In [ ]:
def askMultimodalGemini(prompt_text, generation_config, *args):
    contents = [prompt_text]
    for arg in args:
        contents.append(arg)
    responses = model.generate_content(
      contents,
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
      )
    
    print("--------Response--------")
    for response in responses:
        print(response.text, end="")
        
def askGemini(prompt_text, generation_config):
    responses = model.generate_content(
      prompt_text,
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
      )
    
    print("--------Response--------")
    for response in responses:
        print(response.text, end="")
          
def retrieveInformationFromCustomDataSource(retrieval_text):
    retrieved_content = rag.retrieval_query(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,
                )
            ],
            text=retrieval_text,
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        )

    text_retrieved = ""
    if retrieved_content.contexts:

        for context in retrieved_content.contexts.contexts:
            text_retrieved += context.text + ". \n\n"
        print("--------Text Retrieved--------")
        print("\n\n" + text_retrieved)
        return text_retrieved
    else:
        print("No content retrieved.")
        return ""

Instanziamo il modello potenziato con tool di recupero.

In [ ]:
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                )
            ],
            similarity_top_k=10,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

rag_model = GenerativeModel(
    model_name="gemini-1.5-pro-001", tools=[rag_retrieval_tool])

# Esempi e contesto per la categoria 1: Reti combinatorie

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image1-cartegory1>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image2-cartegory1>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image3-cartegory1>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """La rete combinatoria ha due ingressi (A, B) e sei uscite (NOT, AND, NAND, OR, NOR, EXOR). 
NOT è generato da una porta logica NOT con ingresso A.
AND è generato da una porta logica AND a due ingressi: A e B.
NAND è generato da una porta logica NAND a due ingressi: A e B.
OR è generato da una porta logica OR a due ingressi: A e B.
NOR è generato da una porta logica NOR a due ingressi: A e B.
EXOR è generato da una porta logica EXOR a due ingressi: A e B.
"""
example_image2_description = """La rete combinatoria ha due ingressi (X, Y) e una uscita (F). 
A è una variabile intermedia generata da una porta logica AND a due ingressi: Y e X negato.
B è una variabile intermedia generata da una porta logica AND a due ingressi: X e Y negato.
F è generato da una porta logica OR a due ingressi: A e B.
"""
example_image3_description ="""
La rete combinatoria ha tre ingressi (X, Y, Z) e due uscite (F, G). 
A è una variabile intermediata generata da una porta logica OR a due ingressi: X e Y.
F è generato da una porta logica EXOR a due ingressi: A e Z negato.
G è generato da una porta logica AND a tre ingressi: X, Y e Z negato. 
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """ A | B | NOT | AND | NAND | OR | NOR | EXOR
---|---|-----|-----|------|----|-----|-----
 0 | 0 |  1  |  0  |  1   |  0 |  1  |  0
 0 | 1 |  1  |  0  |  1   |  1 |  0  |  1
 1 | 0 |  0  |  0  |  1   |  1 |  0  |  1
 1 | 1 |  0  |  1  |  0   |  1 |  0  |  0
"""
solution2 = """ Y | X | F
---|---|---
 0 | 0 | 0
 0 | 1 | 1
 1 | 0 | 1
 1 | 1 | 0
"""
solution3 = """ Z | Y | X | F | G
---|---|---|---|---
 0 | 0 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 0
 0 | 1 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 1
 1 | 0 | 0 | 0 | 0
 1 | 0 | 1 | 1 | 0
 1 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"Rete combinatoria": "{example_image1}", "Descrizione della rete": "{example_image1_description}"}}'
input2 = f'{{"Rete combinatoria": "{example_image2}", "Descrizione della rete": "{example_image2_description}"}}'
input3 = f'{{"Rete combinatoria": "{example_image3}", "Descrizione della rete": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """Che cos'è una rete combinatoria?
Come si analizza una rete combinatoria?
Che cos'è una tabella di verità di una rete combinatoria?
Come si costruisce la tabella di verità di una rete combinatoria?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-it>/<textbook-file>.pdf")

--------Text Retrieved--------


Appunti di reti logiche
Ing. Luca Martini
11 aprile 2003Capitolo 1
Reti combinatorie
Sommario
In questo breve documento mostreremo sia alcuni concetti base sulle reti
combinatorie, che alcuni dei moduli combinatori pi`u usati nella sintesi di
reti logiche.
1.1 Il concetto di rete combinatoria
Useremo per i nostri scopi una logica a due stati: falso (che indicheremo con
0) e vero (che indicheremo con 1). Questo significa che dovremo utilizzare un
tipo di algebra (algebra booleana), in cui le variabili logiche possono assumere
solamente questi due valori.
Una rete combinatoria T pu`o essere modellata come una struttura con n
variabili di ingresso xi e m variabili di uscita zj, tale che il valore delle uscite
dipende dal valore degli ingressi secondo una legge ϕ che associa ad ognuna delle
2
n possibili combinazioni degli ingressi una sola combinazione del valore delle
uscite.
Poich`e il dominio delle funzioni logiche con un numero finito di ingressi ´e
fi

# Esercizio 1

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image1>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video1>.mp4")

image_description = """La rete combinatoria ha tre ingressi (S0, S1, EN) e quattro uscite (U0, U1, U2, U3). 
U0 è generato da una porta logica AND a tre ingressi: S0 negato, S1 negato, EN.
U1 è generato da una porta logica AND a tre ingressi: S0, S1 negato, EN.
U2 è generato da una porta logica AND a tre ingressi: S0 negato, S1, EN.
U3 è generato da una porta logica AND a tre ingressi: S0, S1, EN.
"""

prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Sia {image} un'immagine contenente la rappresentazione grafica di una rete combinatoria, compila la tabella di verità del circuito.

La descrizione del circuito è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [72]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Tabella di verità del circuito combinatorio

| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Spiegazione:**

La tabella di verità mostra tutte le possibili combinazioni di ingresso per S0, S1 ed EN (da 000 a 111) e i corrispondenti valori di uscita per U0, U1, U2 e U3. 

Ad esempio, la seconda riga mostra che quando S0 = 0, S1 = 0 ed EN = 1, l'uscita U0 è 1 mentre le altre uscite (U1, U2, U3) sono 0. Questo perché solo la porta AND che genera U0 ha tutti gli ingressi a 1 in questa combinazione (S0 negato = 1, S1 negato = 1, EN = 1). 


## 1.2 Esercizio con recupero diretto

In [73]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Tabella di Verità del Circuito

| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Spiegazione:**

La tabella di verità elenca tutte le possibili combinazioni dei tre ingressi (S0, S1, EN) e mostra il valore di uscita corrispondente per ciascuna delle quattro uscite (U0, U1, U2, U3).

* **U0** è 1 solo quando S0=0, S1=0 e EN=1, poiché è generato da una porta AND con ingressi negati per S0 e S1.
* **U1** è 1 solo quando S0=1, S1=0 e EN=1.
* **U2** è 1 solo quando S0=0, S1=1 e EN=1.
* **U3** è 1 solo quando S0=1, S1=1 e EN=1.

In tutti gli altri casi, le uscite sono 0. 


## 1.3 Esercizio con recupero diretto e video soluzione

In [76]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

Come si può notare, ogni uscita è attiva (pari a 1) solo per una specifica combinazione degli ingressi, mentre in tutti gli altri casi le uscite sono pari a 0. Questo comportamento è tipico di un decoder, che ha lo scopo di attivare una specifica uscita in base al valore binario presente sugli ingressi. In questo caso, il decoder è abilitato solo quando EN è pari a 1.


## 1.4 Esercizio con video soluzione

In [77]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

Come si può vedere dalla tabella, ciascuna uscita U0, U1, U2, U3 è attiva (pari a 1) solo per una specifica combinazione degli ingressi S0, S1 ed EN. Questo tipo di circuito è un decodificatore, in particolare un decodificatore a 2 bit con abilitazione. 


## 1.5 Esercizio con libro di testo nel prompt

In [78]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Tabella di verità della rete combinatoria

| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Spiegazione:**

* Ogni riga rappresenta una possibile combinazione dei tre ingressi (S0, S1, EN).
* Le colonne U0, U1, U2 e U3 mostrano il valore di uscita corrispondente per ogni combinazione di ingresso.
* Ad esempio, la seconda riga mostra che quando S0=0, S1=0 e EN=1, l'uscita U0 sarà 1 (alto) mentre le altre uscite saranno 0 (basso). Questo perché solo la porta AND che genera U0 ha tutti i suoi ingressi attivi in questa combinazione (S0 negato, S1 negato, EN).

**Nota:** La tabella di verità conferma la descrizione del circuito fornita nell'OBJECTIVE. 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [79]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| EN | S1 | S0 | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |

**Spiegazione:**

* Quando EN = 0, tutte le uscite sono 0 perché EN è un ingresso di tutte le porte AND.
* Quando EN = 1, l'uscita di ogni porta AND dipende dai valori di S1 e S0:
    * U0 = 1 solo quando S1 = 0 e S0 = 0.
    * U1 = 1 solo quando S1 = 0 e S0 = 1.
    * U2 = 1 solo quando S1 = 1 e S0 = 0.
    * U3 = 1 solo quando S1 = 1 e S0 = 1.

In sostanza, questo circuito funge da decodificatore a 2 bit, attivando una delle quattro uscite in base alla combinazione di S1 e S0 quando EN è abilitato (EN = 1).


## 1.7 Esercizio con recupero mediante tool

In [83]:
prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.
Una rete combinatoria è strutturata come segue: 
{image_description}
Compila la tabella di verità del circuito."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| S0 | S1 | EN | U0 | U1 | U2 | U3 |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 | 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "combinatory circuit truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "combinatory circuit truth table"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [31]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1,  example_image1, example_image2, example_image3, image)

--------Response--------
EN | S1 | S0 | U0 | U1 | U2 | U3
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 1 | 0 | 0 | 0 | 0
 0 | 1 | 0 | 0 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 0 | 0 | 0
 1 | 0 | 0 | 1 | 0 | 0 | 0
 1 | 0 | 1 | 0 | 1 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 1 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [81]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
EN | S1 | S0 | U0 | U1 | U2 | U3
---|---|-----|-----|------|----|-----
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 1 | 0 | 0 | 0 | 0
 0 | 1 | 0 | 0 | 0 | 0 | 0
 0 | 1 | 1 | 0 | 0 | 0 | 0
 1 | 0 | 0 | 1 | 0 | 0 | 0
 1 | 0 | 1 | 0 | 1 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 1 


# Esercizio 2

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image2>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video2>.mp4")

image_description = """La rete combinatoria ha quattro ingressi (RPar, D2, D1, D0) e tre uscite (Check1, Check2, ERR). 
A è una variabile intermedia generata da una porta logica OR a due ingressi: RPar e D2.
B è una variabile intermedia generata da una porta logica NAND a due ingressi: RPar e D2.
C è una variabile intermedia generata da una porta logica NAND a due ingressi: D1 e D0.
D è una variabile intermedia generata da una porta logica OR a due ingressi: D1 e D0.
Check1 è generato da una porta logica AND a due ingressi: A e B.
Check2 è generato da una porta logica AND a due ingressi: C e D.
F è una variabile intermedia generata da una porta logica NAND a due ingressi: Check1 e Check2.
G è una variabile intermedia generata da una porta logica OR a due ingressi: Check1 e Check2.
ERR è generato da una porta logica AND a due ingressi: F e G.
"""

prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Sia {image} un'immagine contenente la rappresentazione grafica di una rete combinatoria, compila la tabella di verità del circuito.

La descrizione del circuito è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1

## 1.2 Esercizio con recupero diretto

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0

## 1.3 Esercizio con recupero diretto e video soluzione

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| D0 | D1 | D2 | RPar | Check1 | Check2 | ERR |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 | 0 | 1 | 


## 1.4 Esercizio con video soluzione

In [14]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| D0 | D1 | D2 | RPar | Check1 | Check2 | ERR |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 1 |
| 0 | 0 | 1 | 0 | 1 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 |
| 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 1 | 0 |
| 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 0 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 | 0 | 1 |

Come puoi vedere, la tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito (D0, D1, D2 e RPar) e mostra i corrispondenti valori di uscita per ciascuna combinazione (Check1, Check2 ed ERR).


## 1.5 Esercizio con libro di testo nel prompt

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Tabella di verità della rete combinatoria

| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| D0 | D1 | D2 | RPar | Check1 | Check2 | ERR |
|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 1 | 1 | 0 |

**Spiegazione:**

La tabella di verità elenca tutte le possibili combinazioni dei quattro ingressi (D0, D1, D2, RPar) e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Per compilare la tabella, è necessario analizzare il circuito e determinare il valore di ciascuna uscita per ogni combinazione di ingresso. Ad esempio, per la prima riga della t

## 1.7 Esercizio con recupero mediante tool

In [19]:
prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.
Una rete combinatoria è strutturata come segue: 
{image_description}
Compila la tabella di verità del circuito."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| RPar | D2 | D1 | D0 | A | B | C | D | Check1 | Check2 | F | G | ERR |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "scrivi la tabella di verit\u00e0 del circuito."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "scrivi la tabella di verit\u00e0 del circuito."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [29]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Output:  RPar | D2 | D1 | D0 | Check1 | Check2 | ERR
-------|----|----|----|--------|--------|-----
  0    |  0 |  0 |  0 |   0    |   0    |  0
  0    |  0 |  0 |  1 |   0    |   1    |  0
  0    |  0 |  1 |  0 |   0    |   1    |  0
  0    |  0 |  1 |  1 |   0    |   1    |  0
  0    |  1 |  0 |  0 |   1    |   0    |  0
  0    |  1 |  0 |  1 |   1    |   1    |  1
  0    |  1 |  1 |  0 |   1    |   1    |  1
  0    |  1 |  1 |  1 |   1    |   1    |  1
  1    |  0 |  0 |  0 |   0    |   0    |  0
  1    |  0 |  0 |  1 |   0    |   1    |  0
  1    |  0 |  1 |  0 |   0    |   1    |  0
  1    |  0 |  1 |  1 |   0    |   1    |  0
  1    |  1 |  0 |  0 |   0    |   0    |  0
  1    |  1 |  0 |  1 |   0    |   1    |  0
  1    |  1 |  1 |  0 |   0    |   1    |  0
  1    |  1 |  1 |  1 |   0    |   1    |  0 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [18]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
 RPar | D2 | D1 | D0 | Check1 | Check2 | ERR
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 0
 0 | 0 | 0 | 1 | 0 | 1 | 0
 0 | 0 | 1 | 0 | 0 | 1 | 0
 0 | 0 | 1 | 1 | 0 | 0 | 0
 0 | 1 | 0 | 0 | 1 | 0 | 0
 0 | 1 | 0 | 1 | 1 | 1 | 0
 0 | 1 | 1 | 0 | 1 | 1 | 0
 0 | 1 | 1 | 1 | 1 | 0 | 0
 1 | 0 | 0 | 0 | 0 | 0 | 0
 1 | 0 | 0 | 1 | 0 | 1 | 0
 1 | 0 | 1 | 0 | 0 | 1 | 0
 1 | 0 | 1 | 1 | 0 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 0 | 0
 1 | 1 | 0 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 1 | 0
 1 | 1 | 1 | 1 | 0 | 0 | 0 


# Esercizio 3

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image3>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video3>.mp4")

image_description = """La rete combinatoria ha quattro ingressi (A, B, C, D) e tre uscite (O1, O2, O3). 
X è una variabile intermedia generata da una porta logica EXOR a tre ingressi: A negato, B e C.
Y è una variabile intermedia generata da una porta logica NOR a tre ingressi: B, C negato e D.
O1 è generato da una porta logica AND a quattro ingressi: A, X, C e D.
O2 è generato da una porta logica OR a due ingressi: C e Y.
O3 è generato da una porta logica NAND a due ingressi: A e Y.
"""

prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Sia {image} un'immagine contenente la rappresentazione grafica di una rete combinatoria, compila la tabella di verità del circuito.

La descrizione del circuito è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [44]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Tabella di Verità

| A | B | C | D |  !A | X | !C | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 |  1 | 1 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 |  1 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 |  1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 |  1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |  1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |  1 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 |  1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 |  1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 |  0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 |  0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |  0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 |  0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 |  0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |  0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 |  0 | 0 | 0 | 0 | 0 | 1 | 1 |
| 1 | 1 | 1 | 1 |  0 | 0 | 0 | 0 | 1 | 1 | 1 |

**Legenda:**

* **!A** e **!C** rappresentano le negazioni di A e C, rispettivamente.
* **X** è calcol

## 1.2 Esercizio con recupero diretto

In [45]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D |  !A | X | !C | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |

**Spiegazione della tabella:**

* **Colonne A-D:** Rappresentano tutte le possibili combinazioni di input per il circuito (16 in totale, da 

## 1.3 Esercizio con recupero diretto e video soluzione

In [47]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità per il circuito combinatorio descritto è la seguente:

| A | B | C | D | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1 |
| 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |

**Spiegazione:**

1. **Variabili intermedie X e Y:**
    * X è calcolato come EXOR di A negato, B e C.
    * Y è calcolato come NOR di B, C negato e D.

2. **Uscite O1, O2 e O3:**
    * O1 è calcolato come AND d

## 1.4 Esercizio con video soluzione

In [48]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| A | B | C | D | EXOR | NOR | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 |
| 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 |

Dove:

* **EXOR** rappresenta la variabile intermedia generata dalla porta logica EXOR a tre ingressi (A negato, B e C).
* **NOR** rappresenta la variabile intermedia generata dalla porta logica NOR a tre ingressi (B, C negato e D).

## 1.5 Esercizio con libro di testo nel prompt

In [49]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Tabella di Verità della Rete Combinatoria

| A | B | C | D |  !A | X | !C | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 1 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |

**Spiegazione:**

1. Per ogni possibile combinazione degli ingressi A, B, C e D, calcoliamo il

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [50]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
| A | B | C | D |  ~A | X | Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 |
| 1 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
```

**Spiegazione:**

1. **Variabile intermedia X:** X è generata da una porta XOR a tre ingressi: ~A (A negato), B e C. La porta XOR restituisce 1 se il numero di ingressi a 1 è dispari.
2. **Variabile interm

## 1.7 Esercizio con recupero mediante tool

In [53]:
prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.
Una rete combinatoria è strutturata come segue: 
{image_description}
Compila la tabella di verità del circuito."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| A | B | C | D |  X |  Y | O1 | O2 | O3 |
|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 |  1 |  1 | 0 | 1 | 1 |
| 0 | 0 | 0 | 1 |  1 |  0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 |  0 |  1 | 0 | 1 | 1 |
| 0 | 0 | 1 | 1 |  0 |  1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |  1 |  1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |  1 |  1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 |  0 |  0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 |  0 |  1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 0 |  0 |  1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 |  0 |  0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |  1 |  1 | 1 | 1 | 0 |
| 1 | 0 | 1 | 1 |  1 |  1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 |  0 |  1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |  0 |  1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 |  1 |  0 | 1 | 1 | 0 |
| 1 | 1 | 1 | 1 |  1 |  1 | 1 | 1 | 0 | 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "combinatory circuit truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "combinatory circuit truth table"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [51]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1,example_image1, example_image2, example_image3, image)

--------Response--------
Output:  D | C | B | A | X | Y | O1 | O2 | O3 
---|---|---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 
 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 
 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 1 
 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 1 
 0 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 
 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 
 0 | 1 | 1 | 0 | 0 | 0 | 0 | 1 | 1
 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 1 
 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 
 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 
 1 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 
 1 | 0 | 1 | 1 | 0 | 1 | 1 | 1 | 1 
 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 
 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 
 1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 0 
 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [52]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
 D | C | B | A | O1 | O2 | O3
---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 0 | 1
 0 | 0 | 0 | 1 | 0 | 0 | 1
 0 | 0 | 1 | 0 | 0 | 1 | 1
 0 | 0 | 1 | 1 | 0 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 1 | 1
 0 | 1 | 0 | 1 | 1 | 1 | 1
 0 | 1 | 1 | 0 | 0 | 1 | 0
 0 | 1 | 1 | 1 | 0 | 1 | 0
 1 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 0 | 0 | 1 | 0 | 0 | 0
 1 | 0 | 1 | 0 | 0 | 0 | 1
 1 | 0 | 1 | 1 | 0 | 0 | 0
 1 | 1 | 0 | 0 | 0 | 1 | 1
 1 | 1 | 0 | 1 | 0 | 1 | 0
 1 | 1 | 1 | 0 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0 | 0 


# Esercizio 4

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image4>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video4>.mp4")

image_description = """La rete combinatoria ha quattro ingressi (A, B, C, D) e due uscite (F, G). 
X è una variabile intermedia generata da una porta logica AND a due ingressi: A negato e B.
Y è una variabile intermedia generata da una porta logica AND a due ingressi: A e B negato.
W è una variabile intermedia generata da una porta logica AND a due ingressi: A e B.
Z è una variabile intermedia generata da una porta logica AND a due ingressi: A negato e B negato.
J è una variabile intermedia generata da una porta logica OR a due ingressi: X e Y.
K è una variabile intermedia generata da una porta logica OR a due ingressi: W e Z.
M è una variabile intermedia generata da una porta logica AND a due ingressi: J e C negato.
N è una variabile intermedia generata da una porta logica AND a due ingressi: C negato e D negato.
O è una variabile intermedia generata da una porta logica AND a due ingressi: K e D negato.
F è generato da una porta logica OR a tre ingressi: M, N e O.
G è generato da una porta logica EXOR a quattro ingressi: C, D, M e O.
"""

prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Sia {image} un'immagine contenente la rappresentazione grafica di una rete combinatoria, compila la tabella di verità del circuito.

La descrizione del circuito è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [32]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Tabella di Verità del Circuito

| A | B | C | D |  !A  |  !B  |  !C  |  !D  | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 |  1  |  1  |  1  |  1  | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 |  1  |  1  |  1  |  0  | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 |  1  |  1  |  0  |  1  | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 0 | 1 | 1 |  1  |  1  |  0  |  0  | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 |  1  |  0  |  1  |  1  | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |  1  |  0  |  1  |  0  | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 |  1  |  0  |  0  |  1  | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 |  1  |  0  |  0  |  0  | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 |  0  |  1  |  1  |  1  | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 

## 1.2 Esercizio con recupero diretto

In [33]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| A | B | C | D |  !A  |  !B  |  !C  |  !D  | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|:----:|:----:|:----:|:----:|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 |  1   |  1   |  1   |  1   | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 |  1   |  1   |  1   |  0   | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 |  1   |  1   |  0   |  1   | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 1 | 1 |  1   |  1   |  0   |  0   | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 |  1   |  0   |  1   |  1   | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |  1   |  0   |  1   |  0   | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 |  1   |  0   |  0   |  1   | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |  1   |  0   |  0   |  0   | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 |
| 1 | 0 | 0 | 0 |  0   |  1   |  1   |  1   | 0 | 1 | 1 | 0 | 1 | 1

## 1.3 Esercizio con recupero diretto e video soluzione

In [34]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità per il circuito logico descritto è la seguente:

| A | B | C | D | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |

**Spiegazione:**

La tabella di verità elenca tutte le possibili combinazion

## 1.4 Esercizio con video soluzione

In [35]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La tabella di verità del circuito è la seguente:

| A | B | C | D | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |

**Note:**

* La tabella di verità elenca tutte le possibili combinazioni di input (A, B, C, D).


## 1.5 Esercizio con libro di testo nel prompt

In [36]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Tabella di verità del circuito

| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [38]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 |

## 1.7 Esercizio con recupero mediante tool

In [41]:
prompt_text = f""" Una rete combinatoria è un circuito in cui gli ingressi e le uscite possono assumere solo due stati corrispondenti ai livelli alto o basso, e le uscite sono funzione unicamente degli ingressi. 
Una tabella di verità è uno strumento utilizzato per rappresentare il comportamento logico di un circuito combinatorio in modo tabellare. La tabella di verità elenca tutte le possibili combinazioni dei valori di ingresso del circuito e mostra i corrispondenti valori di uscita per ciascuna combinazione.

Una rete combinatoria è strutturata come segue: 
{image_description}

Compila la tabella di verità del circuito."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| A | B | C | D | X | Y | W | Z | J | K | M | N | O | F | G |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
| 0 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 | 0 |
| 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 1 |
| 0 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 |
| 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 0 |
| 1 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "combinatory circuit truth table"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "combinatory circuit truth table"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [39]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Output:  D | C | B | A | F | G
---|---|---|---|---|---|
 0 | 0 | 0 | 0 | 1 | 0
 0 | 0 | 0 | 1 | 0 | 0
 0 | 0 | 1 | 0 | 1 | 0
 0 | 0 | 1 | 1 | 0 | 0
 0 | 1 | 0 | 0 | 1 | 1
 0 | 1 | 0 | 1 | 0 | 1
 0 | 1 | 1 | 0 | 1 | 1
 0 | 1 | 1 | 1 | 0 | 1
 1 | 0 | 0 | 0 | 1 | 1
 1 | 0 | 0 | 1 | 0 | 1
 1 | 0 | 1 | 0 | 1 | 1
 1 | 0 | 1 | 1 | 0 | 1
 1 | 1 | 0 | 0 | 0 | 1
 1 | 1 | 0 | 1 | 0 | 0
 1 | 1 | 1 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [40]:
input = f'{{"Rete combinatoria": "{image}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti combinatorie:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
D | C | B | A | J | K | M | N | O | F | G
---|---|---|---|---|---|---|---|---|---|---
 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1
 0 | 0 | 0 | 1 | 1 | 0 | 1 | 1 | 0 | 1 | 1
 0 | 0 | 1 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 0
 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0
 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 0
 0 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1
 0 | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 0
 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0
 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1
 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 0
 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 1 | 1
 1 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0
 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1
 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0

# Esempi e contesto per la categoria 2: Mappe di Karnaugh e sintesi minima

In [ ]:
# Richietse degli esempi per il few-shot prompting
input1 = """| X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 0 |
"""
input2 = """| A1 | A0 | B1 | B0 | S1 |
|----|----|----|----|----|
|  0 |  0 |  0 |  0 |  0 |
|  0 |  0 |  0 |  1 |  0 |
|  0 |  0 |  1 |  0 |  1 |
|  0 |  0 |  1 |  1 |  1 |
|  0 |  1 |  0 |  0 |  0 |
|  0 |  1 |  0 |  1 |  1 |
|  0 |  1 |  1 |  0 |  1 |
|  0 |  1 |  1 |  1 |  0 |
|  1 |  0 |  0 |  0 |  1 |
|  1 |  0 |  0 |  1 |  1 |
|  1 |  0 |  1 |  0 |  0 |
|  1 |  0 |  1 |  1 |  0 |
|  1 |  1 |  0 |  0 |  1 |
|  1 |  1 |  0 |  1 |  0 |
|  1 |  1 |  1 |  0 |  0 |
|  1 |  1 |  1 |  1 |  1 |
"""
input3 = """| X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """{"Mappa di Karnaugh":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  0   0   1   0
    01 |  0   0   0   0
    11 |  0   1   0   0
    10 |  0   1   1   0 ",
 "Sintesi AND-OR":
 "F = Y!TZ + !XYT",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in due implicanti primi.
  Il primo gruppo contiene le due celle (1101, 0101). Poichè le celle contengono un uno se Y è uno, T è zero e Z è uno, il primo termine risulta pari a Y!TZ.
  Il secondo gruppo contiene le due celle (0111, 0110). Poichè le celle contengono un uno se X è zero, Y è uno e T è uno, il quarto termine risulta pari a !XYT."}"""
solution2 = """{"Mappa di Karnaugh":
"          B1A0
           00  01  11  10
          ----------------
B0A1  00 |  0   0   1   1
      01 |  1   1   0   0
      11 |  1   0   1   0
      10 |  0   1   0   1 ",
 "Sintesi AND-OR":
 "S1 = !A1A0!B1B0 + A1A0B1B0 + A1!A0!B1 + !A1!A0B1 + A1!B1!B0 + !A1B1!B0",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in sei implicanti primi.
  Il primo gruppo contiene la cella (0101). Poichè le celle contengono un uno se A1 è zero, A0 è uno, B1 è zero e B0 è uno, il primo termine risulta pari a !A1A0!B1B0.
  Il secondo gruppo contiene la cella (1111). Poichè le celle contengono un uno se A1 è uno, A0 è uno, B1 è uno e B0 è uno, il secondo termine risulta pari a A1A0B1B0.
  Il terzo gruppo contiene le due celle (1000, 1001). Poichè le celle contengono un uno se A1 è uno, A0 è zero e B1 è zero, il terzo termine risulta pari a A1!A0!B1.
  Il quarto gruppo contiene le due celle (0010, 0011). Poichè le celle contengono un uno se A1 è zero, A0 è zero e B1 è uno, il quarto termine risulta pari a !A1!A0B1.
  Il quinto gruppo contiene le due celle (1000, 1100). Poichè le celle contengono un uno se A1 è uno, B1 è zero e B0 è zero, il quinto termine risulta pari a A1!B1!B0.
  Il sesto gruppo contiene le due celle (0010, 0110). Poichè le celle contengono un uno se A1 è zero, B1 è uno e B0 è zero, il sesto termine risulta pari a !A1B1!B0."}"""
solution3 = """{"Mappa di Karnaugh":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  0   0   0   0
    01 |  0   1   1   1
    11 |  0   1   1   1
    10 |  1   0   0   1 ",
 "Sintesi AND-OR":
 "F = XY + XT + !X!YZ",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le quattro celle (1100, 1101, 1110, 1111). Poichè le celle contengono un uno se X è uno e Y è uno, il primo termine risulta pari a XY.
  Il secondo gruppo contiene le quattro celle (1110, 1111, 1010, 1011). Poichè le celle contengono un uno se X è uno e T è uno, il secondo termine risulta pari a XT.
  Il secondo gruppo contiene le due celle (0001, 0011). Poichè le celle contengono un uno se X è zero, Y è zero e Z è uno, il secondo termine risulta pari a !X!YZ."}"""

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """Che cos'è una mappa di Karnaugh o K-map?
Come si costruisce una mappa di Karnaugh a partire da una tabella della verità?
Come si raggruppano le celle contenenti uno (1) in una mappa di Karnaugh?
Come si effettua la sintesi minima utilizzando le mappe di Karnaugh?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-it>/<textbook-file>.pdf")

--------Text Retrieved--------


Quan￾do due implicanti differiscono per una sola variabile, fondono. Quindi, li segno in un’altra tabella. 
Continuo finché posso, ricordandomi di omettere i duplicati. 
Alla fine, gli implicanti che non hanno generato fusione sono quelli principali. 
3 1 3 2 1 3 2 0 3 2 0 2 1 0 2 1 0 z x x x x x x x x x x x x x x x x x = ⋅ + ⋅ ⋅ + ⋅ ⋅ + ⋅ ⋅ + ⋅ ⋅ + ⋅ ⋅
Non si fa fatica a capire che questo metodo è identico a quello visto prima, soltanto è più veloce, 
perché evita confronti inutili. 
Da quanto appena visto appare chiaro che un aspetto importante dell’andare verso una sintesi di costo 
minore consiste nell’essere in grado di cercare agevolmente stati di ingresso (o gruppi di stati di in￾gresso) adiacenti riconosciuti dalla rete. Un modo agevole per fare questo, che però è limitato a reti 
molto semplici, è descritto di seguito. Appunti sulle reti combinatorie – Giovanni Stea 
44
6.2 Mappe di Karnaugh 
Le mappe di Karnaugh sono un altro modo di descriver

# Esercizio 5

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video5>.mp4")

input = """| A1 | A0 | B1 | B0 | P1 |
|----|----|----|----|----|
|  0 |  0 |  0 |  0 |  0 |
|  0 |  0 |  0 |  1 |  0 |
|  0 |  0 |  1 |  0 |  0 |
|  0 |  0 |  1 |  1 |  0 |
|  0 |  1 |  0 |  0 |  0 |
|  0 |  1 |  0 |  1 |  0 |
|  0 |  1 |  1 |  0 |  1 |
|  0 |  1 |  1 |  1 |  1 |
|  1 |  0 |  0 |  0 |  0 |
|  1 |  0 |  0 |  1 |  1 |
|  1 |  0 |  1 |  0 |  0 |
|  1 |  0 |  1 |  1 |  1 |
|  1 |  1 |  0 |  0 |  0 |
|  1 |  1 |  0 |  1 |  1 |
|  1 |  1 |  1 |  0 |  1 |
|  1 |  1 |  1 |  1 |  0 |
"""

prompt_text = f"""Le mappe di Karnaugh (K-maps) sono usate per semplificare espressioni booleane. Una K-map è una rappresentazione grafica delle tabelle di verità e aiutano a determinare la forma minima di una funzione booleana.
Crea una mappa di Karnaugh utilizzano la tabella di verità fornita: {input}.
Determina la forma minima della funzione booleana rappresentata."""

## 1.1 Esercizio senza ulteriore contesto

In [17]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
Ecco i passaggi per semplificare l'espressione booleana usando una mappa di Karnaugh basata sulla tabella di verità fornita:

**1. Creare la mappa di Karnaugh:**

La mappa di Karnaugh per una funzione booleana con quattro variabili di input (A1, A0, B1, B0) è una griglia 4x4. Le intestazioni di riga e colonna della mappa corrispondono a tutte le possibili combinazioni degli input A1A0 e B1B0, rispettivamente. L'ordine delle intestazioni è importante e deve seguire il codice Gray, in cui solo un bit cambia tra celle adiacenti.

```
        B1B0
      00  01  11  10
    A1A0 +---+---+---+
     00 | 0  | 0  | 0  | 0 |
       +---+---+---+---+
     01 | 0  | 0  | 1  | 1 |
       +---+---+---+---+
     11 | 0  | 1  | 0  | 1 |
       +---+---+---+---+
     10 | 0  | 1  | 1  | 0 |
       +---+---+---+---+
```

**2. Riempire la mappa di Karnaugh:**

Riempire le celle della mappa di Karnaugh con i valori di output (P1) dalla tabella di verità. Ogni cella nella mappa cor

## 1.2 Esercizio con recupero diretto

In [18]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
| B1B0 \ A1A0 | 00  | 01 | 11 | 10 |
|--------------|----|----|----|----|
| 00           |  0 |  0 |  0 |  0 |
| 01           |  0 |  0 |  1 |  1 |
| 11           |  0 |  1 |  0 |  1 |
| 10           |  0 |  1 |  1 |  0 |

**Sottocubi principali:**

* C'è un sottocubo di ordine 4 che copre tutti gli '1' nella colonna A1=1 e B0=1. L'implicante principale corrispondente è **A1B0**.

* C'è un sottocubo di ordine 2 che copre gli '1' in A1=0, B1=1 e A0=1. L'implicante principale corrispondente è **B1A0**.

**Espressione booleana minima:**

La forma minima della funzione booleana rappresentata dalla mappa di Karnaugh è:

**P1 = A1B0 + B1A0** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [19]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana P1 è:

```
B1B0  00  01  11  10
A1A0
------ --  --  --  --
00      0   0   0   0
01      0   0   1   1
11      0   1   0   1
10      0   1   1   0
```

Ci sono quattro implicanti primi in questa mappa di Karnaugh:

* Rosso:  `¬B1 * B0 * A1`
* Verde:  `A0 * B0 * A1`
* Blu:  `¬A0 * B0 * ¬B1`
* Viola:  `¬A1 * B1 * A0`

Dato che tutti gli uni sono coperti da almeno un implicante primo, questi quattro implicanti primi sono anche implicanti primi essenziali. 

Quindi, la forma minima della funzione booleana rappresentata è:

`P1 = (¬B1 * B0 * A1) + (A0 * B0 * A1) + (¬A0 * B0 * ¬B1) + (¬A1 * B1 * A0)` 


## 1.4 Esercizio con video soluzione

In [20]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
B1\B0  00  01  11  10
   A1\ 0   0   1   0
   A0  0   0   1   0
-------
   1   0   1   1   0
   1   0   1   0   1
```

I gruppi di uni per la forma minima sono:

* Rosso:  A1 B0
* Blu:  A0 B1
* Verde:  B1' B0 A1
* Rosa:  B0' B1 A0

Pertanto, la forma minima della funzione booleana è:

**P1 = A1 B0 + A0 B1 + B1' B0 A1 + B0' B1 A0** 


## 1.5 Esercizio con libro di testo nel prompt

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
B1B0
  \ A1A0  00  01  11  10
-------+-----------------
   00   |  0   0   0   0 
   01   |  0   0   0   1
   11   |  0   1   0   1
   10   |  0   1   1   0
```

Analizzando la mappa di Karnaugh, possiamo identificare i seguenti raggruppamenti di '1':

* Un raggruppamento di due '1' che copre le celle (A1A0 = 10, B1B0 = 01) e (A1A0 = 11, B1B0 = 01). Questo raggruppamento corrisponde al termine **A1B0**.
* Un raggruppamento di due '1' che copre le celle (A1A0 = 11, B1B0 = 10) e (A1A0 = 11, B1B0 = 11). Questo raggruppamento corrisponde al termine **A1B1**.
* Un raggruppamento di due '1' che copre le celle (A1A0 = 01, B1B0 = 10) e (A1A0 = 01, B1B0 = 11). Questo raggruppamento corrisponde al termine **A0B1**.

La forma minima della funzione booleana rappresentata è quindi:

**P1 = A1B0 + A1B1 + A0B1** 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [22]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
Ecco la mappa di Karnaugh per la tabella di verità fornita e la forma minima della funzione booleana:

**Mappa di Karnaugh**

```
B1B0
  \ A1A0  00  01  11  10
  +------+---+---+---+---+
  |  00  | 0 | 0 | 0 | 0 |
  +------+---+---+---+---+
  |  01  | 0 | 0 | 1 | 0 |
  +------+---+---+---+---+
  |  11  | 0 | 1 | 0 | 1 |
  +------+---+---+---+---+
  |  10  | 0 | 1 | 1 | 0 |
  +------+---+---+---+---+
```

**Gruppi**

Ci sono quattro gruppi di 1 nella mappa di Karnaugh:

* Rosso:  `¬B1 * B0 * A1`
* Blu: `B1 * ¬B0 * A1`
* Verde: `B1 * A0`
* Viola: `¬A1 * B1 * A0`

**Forma minima**

La forma minima della funzione booleana è la somma di questi gruppi:

`P1 = ¬B1 * B0 * A1 + B1 * ¬B0 * A1 + B1 * A0 + ¬A1 * B1 * A0` 


## 1.7 Esercizio con recupero mediante tool

In [25]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

```
A1A0 \ B1B0  00  01  11  10
       00     0   0   0   0
       01     0   0   1   1 
       11     0   1   0   1
       10     0   1   1   0
```

La forma minima della funzione booleana è:  `A0B1 + A1B0` 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "create a karnaugh map for the following truth table: | A1 | A0 | B1 | B0 | P1 |\n|----|----|----|----|----|\n|  0 |  0 |  0 |  0 |  0 |\n|  0 |  0 |  0 |  1 |  0 |\n|  0 |  0 |  1 |  0 |  0 |\n|  0 |  0 |  1 |  1 |  0 |\n|  0 |  1 |  0 |  0 |  0 |\n|  0 |  1 |  0 |  1 |  0 |\n|  0 |  1 |  1 |  0 |  1 |\n|  0 |  1 |  1 |  1 |  1 |\n|  1 |  0 |  0 |  0 |  0 |\n|  1 |  0 |  0 |  1 |  1 |"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "create a karnaugh map for the following truth table: | A1 | A0 | B1 | B0 | P1 |\n|----|----|----|----|----|\n|  0 |  0 |  0 |  0 |  0 |\n|  0 |  0 |  0 |  1 |  0 |\n|  0 |  0 |  1 |  0 |  0 |\n|  0 |  0 |  1 |  1 |  0 

## 1.8 Esercizio con few-shot prompting

In [23]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 0 |

Output: {"Mappa di Karnaugh":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  0   0   1   0
    01 |  0   0   1   1
    11 |  0   1   0   1
    10 |  0   0   1   0 ",
 "Sintesi AND-OR":
 "f = YTZ + !XTZ + X!YT",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le due celle (0111, 0110). Poichè le celle contengono un uno se X è zero, Y è uno e T è uno, il primo termine risulta pari a !XYT.
  Il secondo gruppo contiene le due celle (1011, 1111). Poichè le celle contengono un 

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [24]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
{"Mappa di Karnaugh":
"        B1B0
         00  01  11  10
        ----------------
A1A0  00 |  0   0   0   0
    01 |  0   0   1   1
    11 |  0   1   1   0
    10 |  0   1   1   1 ",
 "Sintesi AND-OR":
 "P1 = !B1A1B0 + !A1A0B1B0 + A1A0!B1B0 + A1!A0B0",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in quattro implicanti primi.
  Il primo gruppo contiene le due celle (1011, 1111). Poichè le celle contengono un uno se B1 è zero, A1 è uno e B0 è uno, il primo termine risulta pari a !B1A1B0.
  Il secondo gruppo contiene le due celle (0111, 0110). Poichè le celle contengono un uno se A1 è zero, A0 è uno, B1 è uno e B0 è uno, il secondo termine risulta pari a !A1A0B1B0.
  Il terzo gruppo contiene le due celle (1101, 1111). Poichè le celle contengono un uno se A1 è uno, A0 è uno e B0 è uno, il terzo termine risulta pari a A1A0!B1B0.
  Il quarto gruppo contiene le due celle (1001, 1011). Poichè le celle contengono un uno se A1 è uno, A0 è zero e 

# Esercizio 6

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video6>.mp4")

input = """| X | Y | Z | T | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 1 |
| 0 | 0 | 1 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 1 | 1 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |
"""

prompt_text = f"""Le mappe di Karnaugh (K-maps) sono usate per semplificare espressioni booleane. Una K-map è una rappresentazione grafica delle tabelle di verità e aiutano a determinare la forma minima di una funzione booleana.
Crea una mappa di Karnaugh utilizzano la tabella di verità fornita: {input}.
Determina la forma minima della funzione booleana rappresentata."""

## 1.1 Esercizio senza ulteriore contesto

In [10]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
Ecco come costruire una mappa di Karnaugh e semplificare la funzione booleana dalla tabella di verità fornita:

**1. Creare la mappa di Karnaugh:**

La tabella di verità ha 4 variabili di input (X, Y, Z, T), quindi abbiamo bisogno di una mappa di Karnaugh 4x4. Le intestazioni delle righe e delle colonne della mappa rappresentano le combinazioni delle variabili di input. L'ordinamento è importante e segue il codice Gray, in cui solo un bit cambia tra celle adiacenti:

```
     YZ
   00 01 11 10 
XT  --------
00 | 1  1  0  1 |
01 | 1  0  1  1 |
11 | 1  1  1  1 |
10 | 1  1  1  1 |
   --------
```

**2. Raggruppare gli 1:**

L'obiettivo è raggruppare gli 1 nella mappa in gruppi rettangolari della dimensione più grande possibile. Le regole per il raggruppamento sono:

* I gruppi devono contenere solo 1.
* I gruppi devono avere una dimensione di 2^n (1, 2, 4, 8, ecc.).
* I gruppi possono avvolgere i bordi della mappa.
* I gruppi più grandi sono preferibili ai gruppi 

## 1.2 Esercizio con recupero diretto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
Ecco come si può creare una mappa di Karnaugh e derivare l'espressione booleana semplificata:

**1. Creazione della mappa di Karnaugh:**

* Abbiamo 4 variabili di input (X, Y, Z, T), quindi la mappa di Karnaugh sarà una griglia 4x4.
* Le etichette delle righe e delle colonne della mappa saranno le possibili combinazioni di due variabili (XY e ZT) in ordine di Gray (solo un bit cambia tra celle adiacenti). 

```
      ZT
 XY  00  01  11  10 
  00   1   1   0   1
  01   0   0   1   0
  11   1   1   1   1
  10   1   1   1   1 
```

**2. Inserimento dei valori:**

* Riempi la mappa di Karnaugh con i valori di 'F' dalla tabella di verità. Ogni cella della mappa corrisponde a una riga specifica della tabella di verità.

**3. Raggruppamento degli '1':**

* Raggruppa gli '1' adiacenti in gruppi di 2, 4, 8 o 16. I gruppi devono essere rettangolari o quadrati.
* I gruppi possono "avvolgere" i bordi della mappa.
* Cerca i gruppi più grandi possibili per ottenere la massim

## 1.3 Esercizio con recupero diretto e video soluzione

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana F basata sulla tabella di verità fornita è:

```
    YZ
XT  00  01  11  10
00  1   0   1   1
01  1   0   0   0
11  1   1   1   1
10  1   1   1   1
```

Da questa mappa di Karnaugh, possiamo identificare i seguenti implicanti primi:

* **X:** Questo implicante copre tutte le celle dove X = 1.
* **¬Y¬T:** Questo implicante copre le quattro celle nell'angolo in alto a sinistra dove Y = 0 e T = 0.
* **YZT:** Questo implicante copre le due celle in basso a destra dove Y = 1, Z = 1 e T = 1.

Poiché tutti gli uni nella mappa di Karnaugh sono coperti da questi implicanti primi, la forma minima della funzione booleana F è:

**F = X + ¬Y¬T + YZT** 


## 1.4 Esercizio con video soluzione

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La soluzione a questo esercizio è disponibile nel video allegato. 


## 1.5 Esercizio con libro di testo nel prompt

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
YZ
  \  00  01  11  10
XT \-----------------
00 |  1   1   0   1
01 |  0   0   1   0
11 |  1   1   1   1
10 |  1   1   1   1
```

Per determinare la forma minima della funzione booleana, raggruppiamo gli 1 nella mappa di Karnaugh. Possiamo formare i seguenti gruppi:

* Un gruppo di otto 1 che copre l'intera riga inferiore (XT = 11 e 10). Questo gruppo rappresenta il termine **X**.
* Un gruppo di quattro 1 che copre le due colonne centrali (YZ = 01 e 11). Questo gruppo rappresenta il termine **Z**.
* Un gruppo di due 1 nelle prime due righe della prima colonna (XT = 00 e YZ = 00). Questo gruppo rappresenta il termine **!X!Y!Z**.

La forma minima della funzione booleana è la somma di questi termini:

**F = X + Z + !X!Y!Z**

Questa è la forma minima della funzione booleana rappresentata dalla tabella di verità fornita.


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [17]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
    YZ
XT 00 01 11 10
00  1  0  1  1
01  1  0  0  1
11  1  1  1  1
10  1  1  1  1
```

Da questa mappa di Karnaugh, possiamo determinare la forma minima della funzione booleana rappresentata raggruppando gli 1. 

Il raggruppamento più grande possibile include tutti gli 1 nella parte inferiore della mappa (XT = 11 e 10). Questo raggruppamento è rappresentato da **X**.

Un altro raggruppamento include gli 1 nelle due colonne di sinistra (YZ = 00 e 01). Questo raggruppamento è rappresentato da **Z negato**.

Infine, c'è un raggruppamento di due 1 nell'angolo in alto a sinistra (XT = 00, YZ = 00) e nell'angolo in basso a sinistra (XT = 11, YZ = 00). Questo raggruppamento è rappresentato da **T negato Y negato**.

Combinando questi raggruppamenti con un'operazione OR, otteniamo la forma minima della funzione booleana:

**F = X + Z negato + T negato Y negato** 


## 1.7 Esercizio con recupero mediante tool

In [20]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

```
YZ
XT  00 01 11 10
00  1  1  0  1
01  1  1  1  1
11  1  1  1  1
10  0  0  1  1 
```

La forma minima della funzione booleana è:  
F = X⋅Z + ¬Y + T 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "create a karnaugh map for the following truth table: | X | Y | Z | T | F |\n|---|---|---|---|---|\n| 0 | 0 | 0 | 0 | 1 |\n| 0 | 0 | 0 | 1 | 1 |\n| 0 | 0 | 1 | 0 | 1 |\n| 0 | 0 | 1 | 1 | 0 |\n| 0 | 1 | 0 | 0 | 0 |\n| 0 | 1 | 0 | 1 | 0 |\n| 0 | 1 | 1 | 0 | 0 |\n| 0 | 1 | 1 | 1 | 1 |\n| 1 | 0 | 0 | 0 | 1 |\n| 1 | 0 | 0 | 1 | 1 |\n| 1 | 0 | 1 | 0 | 1 |\n| 1 | 0 | 1 | 1 | 1 |\n| 1 | 1 | 0 | 0 | 1 |\n| 1 |"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "create a karnaugh map for the following truth table: | X | Y | Z | T | F |\n|---|---|---|---|---|\n| 0 | 0 | 0 | 0 | 1 |\n| 0 | 0 | 0 | 1 | 1 |\n| 0 | 0 | 1 | 0 | 1 |\n| 0 | 0 | 1 | 1 | 0 |\n| 0 | 1 | 0

## 1.8 Esercizio con few-shot prompting

In [18]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | A | B | C | D | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 1 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 0 |
| 1 | 0 | 1 | 1 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Mappa di Karnaugh":
"          CD
           00  01  11  10
          ----------------
AB  00 |  0   0   1   0
      01 |  0   1   1   1
      11 |  1   1   1   1
      10 |  0   0   1   0 ",
 "Sintesi AND-OR":
 "F = BC + AB + CD",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le sei celle (0110, 0111, 1110, 1111, 1100, 1101). Poichè le celle contengono un uno se A è uno e B è uno, il primo termine risulta pari a AB.
  Il secondo gruppo contiene le quattro celle (0111, 1111, 1011, 00

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [19]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
```json
{"Mappa di Karnaugh":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  1   1   1   1
    01 |  1   0   1   0
    11 |  1   1   1   1
    10 |  1   1   1   1 ",
 "Sintesi AND-OR":
 "F = X + !YZ + ZT",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le otto celle (1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111). Poichè le celle contengono un uno se X è uno, il primo termine risulta pari a X.
  Il secondo gruppo contiene le due celle (0100, 0110). Poichè le celle contengono un uno se Y è zero e Z è zero, il secondo termine risulta pari a !YZ.
  Il terzo gruppo contiene le quattro celle (0011, 0111, 1011, 1111). Poichè le celle contengono un uno se Z è uno e T è uno, il terzo termine risulta pari a ZT."}
```

# Esercizio 7

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video7>.mp4")

input = """| A1 | A0 | B1 | B0 | G |
|----|----|----|----|---|
|  0 |  0 |  0 |  0 | 0 |
|  0 |  0 |  0 |  1 | 0 |
|  0 |  0 |  1 |  0 | 0 |
|  0 |  0 |  1 |  1 | 0 |
|  0 |  1 |  0 |  0 | 1 |
|  0 |  1 |  0 |  1 | 0 |
|  0 |  1 |  1 |  0 | 0 |
|  0 |  1 |  1 |  1 | 0 |
|  1 |  0 |  0 |  0 | 1 |
|  1 |  0 |  0 |  1 | 1 |
|  1 |  0 |  1 |  0 | 0 |
|  1 |  0 |  1 |  1 | 0 |
|  1 |  1 |  0 |  0 | 1 |
|  1 |  1 |  0 |  1 | 1 |
|  1 |  1 |  1 |  0 | 1 |
|  1 |  1 |  1 |  1 | 0 |
"""

prompt_text = f"""Le mappe di Karnaugh (K-maps) sono usate per semplificare espressioni booleane. Una K-map è una rappresentazione grafica delle tabelle di verità e aiutano a determinare la forma minima di una funzione booleana.
Crea una mappa di Karnaugh utilizzano la tabella di verità fornita: {input}.
Determina la forma minima della funzione booleana rappresentata."""

## 1.1 Esercizio senza ulteriore contesto

In [37]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
Ecco i passaggi per determinare la forma minima dell'espressione booleana usando una mappa di Karnaugh, insieme alla soluzione per la tabella di verità fornita:

**1. Scegliere la mappa di Karnaugh:**

Dato che la tabella di verità ha 4 variabili (A1, A0, B1, B0), abbiamo bisogno di una mappa di Karnaugh a 4 variabili. Questa mappa sarà una griglia 4x4.

**2. Etichettare la mappa di Karnaugh:**

Ogni cella nella mappa di Karnaugh rappresenta una combinazione unica delle variabili di input. Le celle sono etichettate in modo che le celle adiacenti differiscano solo per una variabile. Ecco un modo possibile di etichettare la mappa:

```
    B1B0
 A1A0  00  01  11  10 
   00 |  |   |   |   |
      -----------------
   01 |  |   |   |   |
      -----------------
   11 |  |   |   |   |
      -----------------
   10 |  |   |   |   |
      -----------------
```

**3. Riempire la mappa di Karnaugh:**

Riempire la mappa di Karnaugh con i valori dell'uscita (G) dalla tabe

## 1.2 Esercizio con recupero diretto

In [38]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
| A1A0 \ B1B0 | 00 | 01 | 11 | 10 |
|-------------|---|---|---|---|
| 00          | 0 | 0 | 0 | 0 |
| 01          | 1 | 0 | 0 | 0 |
| 11          | 1 | 1 | 0 | 1 |
| 10          | 1 | 1 | 0 | 0 |

Ora, individuiamo gli implicanti principali:

* **Gruppo 1:** Possiamo raggruppare i quattro '1' nell'angolo in basso a sinistra. Questo gruppo copre A1A0 = 1x e B1B0 = x0. Quindi, l'implicante principale è **A1B0'**. 
* **Gruppo 2:** Possiamo raggruppare i due '1' nella riga superiore dell'A1A0 = 11. Questo gruppo copre A1A0 = 11 e B1B0 = x0. Quindi, l'implicante principale è **A1A0B0'**.

Poiché tutti gli '1' sono coperti da questi due gruppi, non ci sono altri implicanti principali.

Pertanto, la forma minima della funzione booleana è:

**G = A1B0' + A1A0B0'** 


## 1.3 Esercizio con recupero diretto e video soluzione

In [39]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana G, basata sulla tabella di verità fornita, è:

```
B1B0  00  01  11  10
A1A0
------ --  --  --  --
00     0   0   0   0
01     1   0   0   0
11     1   1   0   1
10     1   1   0   0
```

Per trovare la forma minima della funzione booleana, dobbiamo identificare i gruppi più grandi di 1 adiacenti nella mappa di Karnaugh. I gruppi possono essere di dimensione 1, 2, 4, 8, ecc. e devono essere rettangolari o quadrati.

In questa mappa, possiamo identificare i seguenti gruppi:

* Un gruppo di quattro 1 che copre le celle (A1A0 = 11, B1B0 = 00, 01, 10). Questo gruppo rappresenta il termine **A1**.
* Un gruppo di due 1 che copre le celle (A1A0 = 01, B1B0 = 00). Questo gruppo rappresenta il termine **¬B1¬B0A0**.
* Un gruppo di due 1 che copre le celle (A1A0 = 11, B1B0 = 00, 10). Questo gruppo rappresenta il termine **A1¬B0**.

La forma minima della funzione booleana G è quindi:

**G = A1 + ¬B1¬B0A0 + A1¬B0** 


## 1.4 Esercizio con video soluzione

In [40]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la tabella di verità data è:

```
B1B0 \ A1A0 | 00  | 01  | 11  | 10 
-------------|-----|-----|-----|-----
      00      |  0  |  1  |  1  |  1 
      01      |  0  |  0  |  1  |  1 
      11      |  0  |  0  |  0  |  0 
      10      |  0  |  0  |  1  |  0 
```

Ci sono tre implicanti primi essenziali nella mappa:

* Il quadrato verde che copre le celle A1A0 = 11 e B1B0 = 00, 01. Questo implicante primo è rappresentato dall'espressione booleana **A1¬B1**.
* Il rettangolo rosso che copre le celle A1A0 = 01, 11 e B1B0 = 00. Questo implicante primo è rappresentato dall'espressione booleana **A0¬B1¬B0**.
* Il rettangolo blu che copre le celle A1A0 = 10, 11 e B1B0 = 10. Questo implicante primo è rappresentato dall'espressione booleana **A1¬A0¬B0**.

La forma minima della funzione booleana è la somma di questi implicanti primi essenziali:

**G = A1¬B1 + A0¬B1¬B0 + A1¬A0¬B0**


## 1.5 Esercizio con libro di testo nel prompt

In [41]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
B1B0
A1A0 00 01 11 10
--------------------
  00 | 0  0  0  0
  01 | 1  0  0  0
  11 | 1  1  0  1
  10 | 1  1  0  1
```

Dalla mappa di Karnaugh, possiamo identificare i seguenti raggruppamenti di 1:

* Un raggruppamento di quattro 1 che copre l'intera colonna `A1A0 = 1x`. Questo raggruppamento corrisponde al termine `A1`.
* Un raggruppamento di due 1 che copre le caselle `A1A0 = 01` e `B1B0 = 00`. Questo raggruppamento corrisponde al termine `A0¬B1¬B0`.

Pertanto, la forma minima della funzione booleana è:

**G = A1 + A0¬B1¬B0**


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [42]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana G basata sulla tabella di verità fornita è:

```
B1B0
  \ A1A0 00 01 11 10
   +------+------+------+------+
 00|  0   |  1   |  1   |  1   |
   +------+------+------+------+
 01|  0   |  0   |  1   |  1   |
   +------+------+------+------+
 11|  0   |  0   |  0   |  1   |
   +------+------+------+------+
 10|  0   |  1   |  0   |  0   |
   +------+------+------+------+
```

Dalla mappa di Karnaugh, possiamo identificare i seguenti implicanti primi:

* **A1¬B1**: questo gruppo copre le quattro celle in alto a destra.
* **¬B0A0**: questo gruppo copre le due celle in alto al centro e le due celle in basso al centro.
* **¬B1A0**: questo gruppo copre le due celle in alto a sinistra.

La forma minima della funzione booleana G è la somma di questi implicanti primi:

**G = A1¬B1 + ¬B0A0 + ¬B1A0**


## 1.7 Esercizio con recupero mediante tool

In [43]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

```
A1A0\B1B0 | 00  | 01  | 11  | 10 
-------|----|----|----|----
    00  | 0   | 0   | 0   | 0
    01  | 1   | 0   | 0   | 0
    11  | 1   | 1   | 0   | 1
    10  | 1   | 1   | 0   | 0
```

La forma minima della funzione booleana è:  `A0B0' + A1` 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "create a karnaugh map for the following truth table: | A1 | A0 | B1 | B0 | G |\n|----|----|----|----|---|\n|  0 |  0 |  0 |  0 | 0 |\n|  0 |  0 |  0 |  1 | 0 |\n|  0 |  0 |  1 |  0 | 0 |\n|  0 |  0 |  1 |  1 | 0 |\n|  0 |  1 |  0 |  0 | 1 |\n|  0 |  1 |  0 |  1 | 0 |\n|  0 |  1 |  1 |  0 | 0 |\n|  0 |  1 |  1 |  1 | 0 |\n|  1 |  0 |  0 |  0 | 1 |\n|  1 |  0 |  0 |  1 | 1 |\n|  1 |  0 |"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "create a karnaugh map for the following truth table: | A1 | A0 | B1 | B0 | G |\n|----|----|----|----|---|\n|  0 |  0 |  0 |  0 | 0 |\n|  0 |  0 |  0 |  1 | 0 |\n|  0 |  0 |  1 |  0 | 0 |\n|  0 |  0 |  1 |  1 | 0 |\n| 

## 1.8 Esercizio con few-shot prompting

In [44]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | A | B | C | D | G |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 1 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 1 |
| 0 | 1 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 1 |
| 1 | 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Mappa di Karnaugh":
"        CD
         00  01  11  10
        ----------------
AB  00 |  0   0   0   1
    01 |  0   1   1   1
    11 |  0   1   1   1
    10 |  0   0   0   1 ",
 "Sintesi AND-OR":
 "G = BC + !AD + ABD",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le sei celle (0111, 0110, 1111, 1110, 1101, 0101). Poichè le celle contengono un uno se B è uno e C è uno, il primo termine risulta pari a BC.
  Il secondo gruppo contiene le due celle (0010, 1010). Poichè le celle conte

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [45]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
{"Mappa di Karnaugh":
"        B1B0
         00  01  11  10
        ----------------
A1A0  00 |  0   0   0   1
    01 |  1   0   0   0
    11 |  1   1   0   1
    10 |  1   1   1   0 ",
 "Sintesi AND-OR":
 "G = A1!B1 + A0!B1!B0 + A1A0!B0",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le otto celle (1000, 1001, 1100, 1101, 1110, 1010, 1011, 1111). Poichè le celle contengono un uno se A1 è uno e B1 è zero, il primo termine risulta pari a A1!B1.
  Il secondo gruppo contiene le due celle (0100, 0101). Poichè le celle contengono un uno se A0 è uno, B1 è zero e B0 è zero, il secondo termine risulta pari a A0!B1!B0.
  Il terzo gruppo contiene le quattro celle (1110, 1100, 1010, 1000). Poichè le celle contengono un uno se A1 è uno, A0 è uno e B0 è zero, il terzo termine risulta pari a A1A0!B0."}

# Esercizio 8

## 1.0 Definizione del prompt

In [ ]:
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video8>.mp4")

input = """| A1 | A0 | B1 | B0 | E |
|----|----|----|----|---|
|  0 |  0 |  0 |  0 | 1 |
|  0 |  0 |  0 |  1 | 0 |
|  0 |  0 |  1 |  0 | 0 |
|  0 |  0 |  1 |  1 | 0 |
|  0 |  1 |  0 |  0 | 0 |
|  0 |  1 |  0 |  1 | 1 |
|  0 |  1 |  1 |  0 | 0 |
|  0 |  1 |  1 |  1 | 0 |
|  1 |  0 |  0 |  0 | 0 |
|  1 |  0 |  0 |  1 | 0 |
|  1 |  0 |  1 |  0 | 1 |
|  1 |  0 |  1 |  1 | 0 |
|  1 |  1 |  0 |  0 | 0 |
|  1 |  1 |  0 |  1 | 0 |
|  1 |  1 |  1 |  0 | 0 |
|  1 |  1 |  1 |  1 | 1 |
"""

prompt_text = f"""Le mappe di Karnaugh (K-maps) sono usate per semplificare espressioni booleane. Una K-map è una rappresentazione grafica delle tabelle di verità e aiutano a determinare la forma minima di una funzione booleana.
Crea una mappa di Karnaugh utilizzano la tabella di verità fornita: {input}.
Determina la forma minima della funzione booleana rappresentata."""

## 1.1 Esercizio senza ulteriore contesto

In [47]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askGemini(prompt, generation_config1)

--------Response--------
Ecco i passaggi per semplificare l'espressione booleana usando una mappa di Karnaugh (K-map) basata sulla tabella di verità fornita:

**1. Capire le variabili di input e output:**

La tabella di verità ha quattro variabili di input: A1, A0, B1, B0 ed un'uscita E. Le variabili di input rappresentano due numeri binari di due bit, A (A1 A0) e B (B1 B0).

**2. Creare la K-map:**

Creiamo una K-map a 4 variabili per questo esempio. Poiché abbiamo quattro variabili di input, la K-map sarà una griglia 4x4. Le righe e le colonne della mappa saranno etichettate con le possibili combinazioni di A1A0 e B1B0, rispettivamente. L'ordine sarà Gray code, in cui solo un bit cambia tra celle adiacenti.

```
        B1B0
    \  00  01  11  10 
 A1A0  +---+---+---+---+
  00  | 1 | 0 | 0 | 0 |
      +---+---+---+---+
  01  | 0 | 1 | 0 | 0 |
      +---+---+---+---+
  11  | 0 | 0 | 1 | 0 |
      +---+---+---+---+
  10  | 0 | 0 | 0 | 1 |
      +---+---+---+---+
```

**3. Riempire la K

## 1.2 Esercizio con recupero diretto

In [48]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askGemini(prompt, generation_config1)

--------Response--------
```
        B1B0
    A1A0  00  01  11  10 
       --------------
    00  | 1 | 0 | 0 | 0 |
       --------------
    01  | 0 | 1 | 0 | 0 |
       --------------
    11  | 0 | 0 | 1 | 0 |
       --------------
    10  | 0 | 0 | 0 | 1 |
       --------------
```

La forma minima della funzione booleana è: **E = A1'A0'B1'B0' + A1'A0B1'B0 + A1A0B1B0 + A1A0'B1B0'** 

Ecco come ho ottenuto questo risultato:

1. **Creare la Mappa di Karnaugh:** Ho organizzato la mappa K con le variabili di input A1, A0, B1 e B0 come mostrato sopra. Ogni cella rappresenta una combinazione unica delle variabili di input e contiene il valore di output corrispondente (E) dalla tabella di verità.

2. **Identificare gli Implicanti Primi:** L'obiettivo è trovare i raggruppamenti più grandi possibili di '1' adiacenti nella mappa K. 
    * C'è un '1' isolato nell'angolo in alto a sinistra (A1'A0'B1'B0'). Questo è un implicante primo di ordine 1.
    * Ci sono due '1' che possono essere raggrup

## 1.3 Esercizio con recupero diretto e video soluzione

In [49]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana E, basata sulla tabella di verità fornita, è la seguente:

```
B1B0  00  01  11  10
A1A0
-------------
00     1   0   0   0
01     0   1   0   0
11     0   0   1   0
10     0   0   0   1 
```

Da questa mappa di Karnaugh, possiamo identificare i seguenti implicanti primi essenziali:

*  **!A1 !A0 !B1 !B0:**  Corrispondente alla cella con "1" in alto a sinistra (00 00).
*  **!A1 A0 !B1 B0:** Corrispondente alla cella con "1" nella seconda riga, seconda colonna (01 01).
*  **A1 A0 B1 B0:** Corrispondente alla cella con "1" in basso a destra (11 11).
*  **A1 !A0 B1 !B0:** Corrispondente alla cella con "1" nella quarta riga, quarta colonna (10 10).

Dato che questi implicanti primi essenziali coprono tutti gli "1" nella mappa di Karnaugh, la forma minima della funzione booleana E è:

**E = (!A1 !A0 !B1 !B0) + (!A1 A0 !B1 B0) + (A1 A0 B1 B0) + (A1 !A0 B1 !B0)** 


## 1.4 Esercizio con video soluzione

In [50]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, video)

--------Response--------
La mappa di Karnaugh per la tabella di verità fornita è:

```
B1 B0
  \  \
A1 A0 00 01 11 10
-------
00 | 1  0  0  0
01 | 0  1  0  0
11 | 0  0  1  0
10 | 0  0  0  1

```

La forma minima della funzione booleana rappresentata è:

**E = Ā1Ā0B̅1B̅0 + Ā1A0B̅1B0 + A1A0B1B0 + A1Ā0B1B̅0**

Questa espressione è stata ottenuta identificando i quattro implicanti primi essenziali nella mappa di Karnaugh:

* Il quadrato rosso in alto a sinistra rappresenta Ā1Ā0B̅1B̅0
* Il quadrato verde in alto al centro rappresenta Ā1A0B̅1B0
* Il quadrato blu in basso al centro rappresenta A1A0B1B0
* Il quadrato giallo in basso a destra rappresenta A1Ā0B1B̅0

Poiché tutti gli "1" nella mappa di Karnaugh sono coperti da questi quattro implicanti primi essenziali, la loro somma rappresenta la forma minima della funzione booleana E.


## 1.5 Esercizio con libro di testo nel prompt

In [51]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook)

--------Response--------
La mappa di Karnaugh per la tabella di verità data è:

```
B1B0
  \ A1A0 00 01 11 10
   +------+------+------+------+
  0|  1   |  0   |  0   |  0   |
   +------+------+------+------+
  1|  0   |  1   |  1   |  1   |
   +------+------+------+------+
```

Dalla K-map, possiamo identificare i seguenti raggruppamenti di 1:

* Un raggruppamento di quattro 1 nell'angolo in basso a destra, che corrisponde al termine **B1**.
* Un raggruppamento di due 1 nella seconda riga, che corrisponde al termine **A0B0**.

Quindi, la forma minima della funzione booleana è:

**E = B1 + A0B0** 


## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [52]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
La mappa di Karnaugh per la funzione booleana E, basata sulla tabella di verità fornita, è la seguente:

```
B1B0
  \ A1A0 00 01 11 10
   +------+------+------+------+
00 |  1  |  0  |  1  |  0  |
   +------+------+------+------+
01 |  0  |  1  |  1  |  0  |
   +------+------+------+------+
11 |  0  |  0  |  1  |  0  |
   +------+------+------+------+
10 |  0  |  0  |  0  |  1  |
   +------+------+------+------+
```

Per determinare la forma minima della funzione booleana, cerchiamo i raggruppamenti più grandi di 1 adiacenti nella mappa di Karnaugh. Le celle adiacenti possono essere orizzontali o verticali, e i bordi della mappa si avvolgono. I raggruppamenti devono avere una dimensione che è una potenza di 2 (1, 2, 4, 8, ecc.).

In questa mappa, possiamo identificare i seguenti raggruppamenti:

* Un raggruppamento di quattro 1 che copre le celle (00,00), (01,01), (11,11) e (10,10). Questo raggruppamento corrisponde al termine **B0**.
* Un raggruppamento di due

## 1.7 Esercizio con recupero mediante tool

In [55]:
responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

```
A1A0\B1B0  00  01  11  10
       ----------------
    00 |  1   0   0   0
    01 |  0   1   0   0
    11 |  0   0   1   0
    10 |  0   0   0   1 
```

La forma minima della funzione booleana è:  `A1B1B0' + A1'A0B0 + A1'B1'B0'` 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "create a karnaugh map for the truth table below: | A1 | A0 | B1 | B0 | E |\n|----|----|----|----|---|\n|  0 |  0 |  0 |  0 | 1 |\n|  0 |  0 |  0 |  1 | 0 |\n|  0 |  0 |  1 |  0 | 0 |\n|  0 |  0 |  1 |  1 | 0 |\n|  0 |  1 |  0 |  0 | 0 |\n|  0 |  1 |  0 |  1 | 1 |\n|  0 |  1 |  1 |  0 | 0 |\n|  0 |  1 |  1 |  1 | 0 |\n|  1 |  0 |  0 |  0 | 0 |\n|  1 |  0 |  0 |  1 | 0 |\n|  1 |  0 |"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "create a karnaugh map for the truth table below: | A1 | A0 | B1 | B0 | E |\n|----|----|----|----|---|\n|  0 |  0 |  0 |  0 | 1 |\n|  0 |  0 |  0 |  1 | 0 |\n|  0 |  0 |  1 |  0 | 0 |\n|  0 |  0 |  1 |  1 | 0 |\n|  0 |  1 

## 1.8 Esercizio con few-shot prompting

In [53]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askGemini(prompt, generation_config1)

--------Response--------
Input: | X | Y | T | Z | F |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 1 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 | 1 |
| 0 | 1 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 | 0 |
| 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 0 |
| 1 | 1 | 0 | 0 | 0 |
| 1 | 1 | 0 | 1 | 0 |
| 1 | 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 | 1 |

Output: {"Mappa di Karnaugh":
"        TY
         00  01  11  10
        ----------------
ZX  00 |  1   1   0   0
    01 |  0   0   0   0
    11 |  0   0   1   0
    10 |  0   0   0   1 ",
 "Sintesi AND-OR":
 "f = !X!Y!T + !XYZT + XYTZ",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in tre implicanti primi.
  Il primo gruppo contiene le due celle (0000, 0001). Poichè le celle contengono un uno se X è zero, Y è zero e T è zero, il primo termine risulta pari a !X!Y!T.
  Il secondo gruppo contiene le due celle (0101, 1111). Poichè le celle conten

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [54]:
prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle mappe di Karnaugh:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, textbook, video)

--------Response--------
{"Mappa di Karnaugh":
"        A1A0
         00  01  11  10
        ----------------
B1B0  00 |  1   0   0   0
    01 |  0   1   0   1
    11 |  0   0   1   1
    10 |  1   0   0   0 ",
 "Sintesi AND-OR":
 "E = !A1!A0!B1B0 + !A1A0B0!B1 + A1A0B1B0 + A1!A0!B0B1",
 "Spiegazione":
 "Possiamo raccogliere le celle contenenti gli uno in quattro implicanti primi.
  Il primo gruppo contiene la cella (0000). Poichè la cella contiene un uno se A1 è zero, A0 è zero, B1 è zero e B0 è zero, il primo termine risulta pari a !A1!A0!B1B0.
  Il secondo gruppo contiene la cella (0101). Poichè la cella contiene un uno se A1 è zero, A0 è uno, B1 è zero e B0 è uno, il secondo termine risulta pari a !A1A0B0!B1.
  Il terzo gruppo contiene la cella (1111). Poichè la cella contiene un uno se A1 è uno, A0 è uno, B1 è uno e B0 è uno, il terzo termine risulta pari a A1A0B1B0.
  Il quarto gruppo contiene la cella (1010). Poichè la cella contiene un uno se A1 è uno, A0 è zero, B1 è uno e B0 è

# Esempi e contesto per la categoria 3: Reti sequenziali

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image1-cartegory3>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image2-cartegory3>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image3-cartegory3>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha tre uscite (OUT1, OUT2, OUT3). 
La struttura della rete è descritta di seguito: 

* **Primo flip-flop di tipo JK-pet:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Gli ingressi J e K sono entrambi collegato all'ingresso EN.
    * L'uscita (Q) è collegata a uno dei due ingressi della prima porta logica AND. L'uscita (Q) è l'uscita della rete OUT1.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso CLEAR viene negato e poi collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Secondo flip-flop di tipo JK-pet:**
    * Riceve il segnale di clock (CK) comune.
    * Gli ingressi J e K sono entrambi collegati all'uscita della prima porta logica AND.
    * L'uscita (Q) è collegata a uno dei due ingressi della seconda porta logica AND. L'uscita (Q) è l'uscita della rete OUT2.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso CLEAR viene negato e poi collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Terzo flip-flop di tipo JK-pet:**
    * Riceve il segnale di clock (CK) comune.
    * Gli ingressi J e K sono entrambi collegati all'uscita della seconda porta logica AND.
    * L'uscita (Q) è l'uscita della rete OUT3.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso CLEAR viene negato e poi collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Prima porta logica AND:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del primo flip-flop JK chiamata OUT1.
        * Segnale EN.
    * L'uscita della prima porta AND entra in ingresso al secondo flip-flop JK e alla seconda porta AND.
* **Seconda porta logica AND:**
     * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del secondo flip-flop JK chiamata OUT2.
        * Uscita della prima porta logica AND.
    * L'uscita della seconda porta AND entra in ingresso al terzo flip-flop JK.
* **Porta logica NOT:**
    * Il suo ingresso è il segnale CLEAR.
    * L'uscita della porta NOT è collegato al -CLEAR di tutti e tre i flip-flop.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **CLEAR (Clear):**  Si tratta di un segnale attivo-alto. Quando attivo(1), forza le uscite di tutti flip-flop a zero (Q=0).
* **EN (Enable):**  In ingresso alla primo flip-flop JK e alla prima porta logica AND. La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha tre uscite (OUT1, OUT2, OUT3). 
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""
example_image2_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha tre uscite (OUT1, OUT2, OUT3). 
La struttura della rete è descritta di seguito: 

* **Primo flip-flop di tipo E-pet:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * L'ingresso D è collegato all'uscita della seconda porta EXOR.
    * L'uscita (Q) è collegata all'ingresso D del secondo filp-flop E-PET e al primo ingresso della porta logica NOR. L'uscita (Q) è l'uscita della rete OUT3.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso -CLEAR è collegato all'ingresso -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
    * Il segnale di ingresso EN è collegato all'ingresso E del flip-flop.
* **Secondo flip-flop di tipo E-pet:**
    * Riceve il segnale di clock (CK) comune.
    * L'ingresso D è collegato all'uscita Q del primo flip-flop E-PET.
    * L'uscita (Q) è collegata all'ingresso D del terzo filp-flop E-PET, al secondo ingresso della porta logica NOR e al primo ingresso della prima porta logica EXOR. L'uscita (Q) è l'uscita della rete OUT2.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso -CLEAR è collegato all'ingresso -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
    * Il segnale di ingresso EN è collegato all'ingresso E del flip-flop.
* **Terzo flip-flop di tipo E-pet:**
    * Riceve il segnale di clock (CK) comune.
     * L'ingresso D è collegato all'uscita Q del secondo flip-flop E-PET.
    * L'uscita (Q) è collegata al secondo ingresso della prima porta logica EXOR. L'uscita (Q) è l'uscita della rete OUT1.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso -CLEAR è collegato all'ingresso -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
    * Il segnale di ingresso EN è collegato all'ingresso E del flip-flop.
* **Porta logica NOR:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del primo flip-flop E-PET chiamata OUT3.
        * Uscita (Q) del secondo flip-flop E-PET chiamata OUT2.
    * L'uscita della porta NOR entra in ingresso alla seconda porta logica EXOR. 
* **Prima porta logica EXOR:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del secondo flip-flop E-PET chiamata OUT2.
        * Uscita (Q) del terzo flip-flop E-PET chiamata OUT1.
    * L'uscita della prima porta EXOR entra in ingresso alla seconda porta logica EXOR.
* **Seconda porta logica EXOR:**
     * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita della porta logica NOR.
        * Uscita della prima porta logica EXOR.
    * L'uscita della seconda porta EXOR entra in ingresso al primo flip-flop E-PET.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **CLEAR (Clear):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti i flip-flop a zero (Q=0).
* **EN (Enable):** Si tratta di un segnale che abilita i flip-flop E-PET quando attivo. Quando inattivo forza i flip-flop a mantenere lo stato precedente. 
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""
example_image3_description ="""La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha due uscite (OUT1, OUT2). 
La struttura della rete è descritta di seguito: 

* **Flip-flop di tipo D-pet:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * L'ingresso D è collegato all'uscita della porta EXOR.
    * L'uscita (Q) è collegata al primo dei tre ingressi della porta logica EXOR. L'uscita (Q) è l'uscita della rete OUT1.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale di ingresso -CLEAR è collegato all'ingresso -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop. 
* **Flip-flop di tipo E-pet:**
    * Riceve il segnale di clock (CK) comune.
    * L'ingresso D è collegato all'uscita negata (-Q) dello stesso flip-flop E-PET.
    * L'uscita (Q) è collegata al terzo dei tre ingressi della porta logica EXOR. L'uscita (Q) è l'uscita della rete OUT2.
    * L'uscita negata (-Q) è collegata all'ingresso D dello stesso flip-flop.
    * Il segnale di ingresso -CLEAR è collegato all'ingresso -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
    * Il segnale di ingresso EN è collegato all'ingresso E del flip-flop.
* **Porta logica EXOR:**
    * I suoi tre ingressi sono collegati rispettivamente a:
        * Uscita (Q) del flip-flop D-PET chiamata OUT1.
	* Segnale di ingresso IN.
        * Uscita (Q) del flip-flop E-PET chiamata OUT2.
    * L'uscita della porta EXOR entra in ingresso al flip-flop D-PET.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **CLEAR (Clear):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti flip-flop a zero (Q=0).
* **EN (Enable):** Si tratta di un segnale che abilita i flip-flop E-PET quando attivo. Quando inattivo forza i flip-flop a mantenere lo stato precedente. 
* **IN (Input):** Si tratta di un segnale di input collegato al secondo ingresso della porta logica EXOR. 
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """ Fronte 1: OUT1 è indeterminato, OUT2 è indeterminato, OUT3 è indeterminato.
Tra fronte 1 e fronte 2: L’ingresso CLEAR passa da zero a uno. Quando CLEAR commuta, OUT1, OUT2 e OUT3 passano da indeterminato a zero.
Fronte 2: OUT1 è zero, OUT2 è zero, OUT3 è zero.
Tra fronte 2 e fronte 3: L’ingresso CLEAR passa da uno a zero.
Fronte 3: OUT1 è zero, OUT2 è zero, OUT3 è zero.
Fronte 4: OUT1 è zero, OUT2 è zero, OUT3 è zero.
Tra fronte 4 e fronte 5: L’ingresso EN passa da zero a uno. 
Fronte 5: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è zero, OUT3 è zero. 
Fronte 6: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte, OUT3 è zero.
Fronte 7: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è uno, OUT3 è zero.
Fronte 8: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte, OUT3 passa da zero a uno poco dopo il fronte.
Fronte 9: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è zero, OUT3 è uno.
Fronte 10: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte, OUT3 è uno.
Fronte 11: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è uno, OUT3 è uno.
Tra fronte 11 e fronte 12: L’ingresso EN passa da uno a zero. 
Fronte 12: OUT1 è uno, OUT2 è uno, OUT3 è uno.
Fronte 13: OUT1 è uno, OUT2 è uno, OUT3 è uno.
Fronte 14: OUT1 è uno, OUT2 è uno, OUT3 è uno.
Fronte 15: OUT1 è uno, OUT2 è uno, OUT3 è uno.
Fronte 16: OUT1 è uno, OUT2 è uno, OUT3 è uno.
"""
solution2 = """Fronte 1: OUT1 è indeterminato, OUT2 è indeterminato, OUT3 è indeterminato.
Tra fronte 1 e fronte 2: L’ingresso CLEAR passa da uno a zero. Quando CLEAR commuta, OUT1, OUT2 e OUT3 passano da indeterminato a zero.
Fronte 2: OUT1 è zero, OUT2 è zero, OUT3 è zero.
Tra fronte 2 e fronte 3: L’ingresso CLEAR passa da zero a uno.
Fronte 3: OUT1 è zero, OUT2 è zero, OUT3 è zero.
Tra fronte 3 e fronte 4: L’ingresso EN passa da zero a uno. 
Fronte 4: OUT1 è zero, OUT2 è zero, OUT3 passa da zero a uno poco dopo il fronte.
Fronte 5: OUT1 è zero, OUT2 passa da zero a uno poco dopo il fronte, OUT3 passa da uno a zero poco dopo il fronte. 
Fronte 6: OUT1 passa da zero a uno poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte, OUT3 passa da zero a uno poco dopo il fronte.
Fronte 7: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte, OUT3 è uno.
Fronte 8: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è uno, OUT3 è uno.
Fronte 9: OUT1 è uno, OUT2 è uno, OUT3 passa da uno a zero poco dopo il fronte.
Fronte 10: OUT1 è uno, OUT2 passa da uno a zero poco dopo il fronte, OUT3 è zero.
Fronte 11: OUT1 passa da uno a zero poco dopo il fronte, OUT2 è zero, OUT3 è zero.
Fronte 12: OUT1 è zero, OUT2 è zero, OUT3 passa da zero a uno poco dopo il fronte.
Fronte 13: OUT1 è zero, OUT2 passa da zero a uno poco dopo il fronte, OUT3 passa da uno a zero poco dopo il fronte.
Fronte 14: OUT1 passa da zero a uno poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte, OUT3 passa da zero a uno poco dopo il fronte.
Fronte 15: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte, OUT3 è uno.
Fronte 16: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è uno, OUT3 è uno.
Tra fronte 16 e fronte 17: L’ingresso EN passa da uno a zero. 
Fronte 17: OUT1 è uno, OUT2 è uno, OUT3 è uno.
"""
solution3 = """Fronte 1: OUT1 è indeterminato, OUT2 è indeterminato.
Tra fronte 1 e fronte 2: L’ingresso CLEAR passa da uno a zero. Quando CLEAR commuta, OUT1 e OUT2 passano da indeterminato a zero. Dopodiché l’ingresso CLEAR passa da zero a uno. 
Fronte 2: OUT1 è zero, OUT2 è zero.
Tra fronte 2 e fronte 3: L’ingresso IN passa da zero a uno.
Fronte 3: OUT1 passa da zero a uno poco dopo il fronte, OUT2 è zero.
Fronte 4: OUT1 passa da uno a zero poco dopo il fronte, OUT2 è zero.
Tra fronte 4 e fronte 5: L’ingresso IN passa da uno a zero.
Fronte 5: OUT1 è zero, OUT2 è zero. 
Fronte 6: OUT1 è zero, OUT2 è zero.
Tra fronte 6 e fronte 7: L’ingresso EN passa da zero a uno.
Fronte 7: OUT1 è zero, OUT2 passa da zero a uno poco dopo il fronte.
Fronte 8: OUT1 passa da zero a uno poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte.
Tra fronte 8 e fronte 9: L’ingresso IN passa da zero a uno.
Fronte 9: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte.
Fronte 10: OUT1 è zero, OUT2 passa da uno a zero poco dopo il fronte. 
Fronte 11: OUT1 passa da zero a uno poco dopo il fronte, OUT2 passa da zero a uno poco dopo il fronte.
Tra fronte 11 e fronte 12: L’ingresso IN passa da uno a zero.
Fronte 12: OUT1 passa da uno a zero poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte.
Fronte 13: OUT1 è zero, OUT2 passa da zero a uno poco dopo il fronte.
Fronte 14: OUT1 passa da zero a uno poco dopo il fronte, OUT2 passa da uno a zero poco dopo il fronte.
Tra fronte 14 e fronte 15: L’ingresso EN passa da uno a zero.
Fronte 15: OUT1 è uno, OUT2 è zero.
Fronte 16: OUT1 è uno, OUT2 è zero.
"""

# Richietse degli esempi per il few-shot prompting
prompt_text1 = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.
Fronte 1: CLEAR è zero ed EN è zero
Tra fronte 1 e fronte 2: CLEAR passa da zero a uno ed EN è zero.
Fronte 2: CLEAR è uno ed EN è zero.
Tra fronte 2 e fronte 3: CLEAR passa da uno a zero ed EN è zero.
Fronti 3-4: CLEAR è zero ed EN è zero.
Tra fronte 4 e fronte 5: CLEAR è zero ed EN passa da zero a uno.
Fronti 5-11: CLEAR è zero ed EN è uno.
Tra fronte 11 e fronte 12: CLEAR è zero ed EN passa da uno a zero.
Fronti 12-16: CLEAR è zero ed EN è uno.
Dati i valori sopra per gli ingressi CLEAR ed EN, esegui l’analisi temporale della rete sequenziale riportata in {example_image1}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 16).
"""
prompt_text2 = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.
Fronte 1: -CLEAR è uno ed EN è zero. 
Tra fronte 1 e fronte 2: -CLEAR passa da uno a zero ed EN è zero.
Fronte 2: -CLEAR è zero ed EN è zero. 
Tra fronte 2 e fronte 3: -CLEAR passa da zero a uno ed EN è zero.
Fronte 3: -CLEAR è uno ed EN è zero.
Tra fronte 3 e fronte 4: -CLEAR è uno ed EN passa da zero a uno.
Fronti 4-16: -CLEAR è uno ed EN è uno.
Tra fronte 16 e fronte 17: -CLEAR è uno ed EN passa da uno a zero.
Fronte 17: -CLEAR è uno ed EN è zero.
Dati i valori sopra per gli ingressi -CLEAR ed EN, esegui l’analisi temporale della rete sequenziale riportata in {example_image2}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 17).
"""
prompt_text3 = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.
Fronte 1:-CLEAR è uno, IN è zero ed EN è zero. 
Tra fronte 1 e fronte 2: -CLEAR passa da uno a zero e subito dopo da zero a uno, In è zero ed EN è zero. 
Fronte 2: -CLEAR è uno, IN è zero ed EN è zero.
Tra fronte 2 e fronte 3: -CLEAR è uno, IN passa da zero a uno ed EN è zero.
Fronti 3-4: -CLEAR è uno, IN è uno ed EN è zero.
Tra fronte 4 e fronte 5: -CLEAR è uno, IN passa da uno a zero ed EN è zero.
Fronti 5-6: -CLEAR è uno, IN è zero ed EN è zero.
Tra fronte 6 e fronte 7: -CLEAR è uno, IN è zero ed EN passa da zero a uno.
Fronti 7-8: -CLEAR è uno, IN è zero ed EN è uno.
Tra fronte 8 e fronte 9:  -CLEAR è uno, IN passa da zero a uno ed EN è uno.
Fronti 9-11: -CLEAR è uno, IN è uno ed EN è uno.
Tra fronte 11 e 12: -CLEAR è uno, IN passa da uno a zero ed EN è uno.
Fronti 12-14: -CLEAR è uno, IN è zero ed EN è uno.
Tra fronte 14 e fronte 15: -CLEAR è uno, IN è zero ed EN passa da uno a zero.
Fronti 15-16: -CLEAR è uno, IN è zero ed EN è zero.
Dati i valori sopra per gli ingressi -CLEAR, IN ed EN, esegui l’analisi temporale della rete sequenziale riportata in {example_image3}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 16).
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"Rete sequenziale": "{example_image1}", "Richiesta": "{prompt_text1}", "Descrizione della rete": "{example_image1_description}"}}'
input2 = f'{{"Rete sequenziale": "{example_image2}", "Richiesta": "{prompt_text2}", "Descrizione della rete": "{example_image2_description}"}}'
input3 = f'{{"Rete sequenziale": "{example_image3}", "Richiesta": "{prompt_text3}", "Descrizione della rete": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """Che cos'è una rete sequenziale?
Come si analizza una rete sequenziale?
Cosa sono i flip-flop PET e quali sono?
Che cos'è un diagramma temporale di una rete sequenziale?
Come si costruisce un diagramma temporale di una rete sequenziale?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://reti-logiche-esercizi/Libro-capitoli-5-e-6.pdf")

--------Text Retrieved--------


32), perch´e la sequenza
generata `e solo apparentemente casuale, ma in realt`a si ripete ciclicamente.
6.4.3 Esempio n. 3
La rete di questo esempio impiega due fip-fop D-PET, con i segnali di Clock
e di Reset in comune, e non presenta ingressi di comando. Le uscite U0 e U1
sono prelevate direttamente dalle uscite Q0 e Q1 dei fip-fop, mentre T C `e
ottenuta attraverso una porta logica:
Applichiamo la stessa procedura di analisi impiegata in precedenza: si tratta
di valutare D0 e D1 in corrispondenza del fronte attivo del Clock, sapendo
che, all’arrivo di quel fronte, saranno caricati nei fip-fop. Pu`o essere utile
“separare” dallo schema complessivo le reti combinatorie che producono D0,
D1 e le uscite della rete U0, U2 e T C.6.4 Analisi di reti 231
Le ridisegniamo a parte in forma circuitale, quindi, oppure (o in aggiunta) le
esprimiamo come espressioni booleane:
D0 = Q0; D1 = Q0 ⊕ Q1;
U0 = Q0; U1 = Q1; T C = Q0 · Q1
Utilizzando le fgure o le espressio

# Esercizio 11

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image11>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video11>.mp4")

image_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha quattro uscite (A, B, C, D). 
La struttura della rete è descritta di seguito: 

* **Flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Il suo ingresso dati (D) è collegato all'uscita della porta logica NAND.
    * L'uscita (Q) è collegata al primo degli ingressi della porta NAND e all'ingresso J del primo flip-flop JK-PET.
    * L'uscita negata (-Q) è collegata all'ingresso K del primo flip-flop JK-PET.
    * Il segnale -RES è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Primo flip-flop di tipo JK-PET:**
    * Riceve il segnale di clock (CK) comune.
    * L'ingresso J è collegato all'uscita Q del flip-flop D. L'ingresso K è collegato all'uscita negata (-Q) del filp-flop D.
    * L'uscita (Q) è collegata al secondo degli ingressi della porta NAND e all'ingresso J del secondo flip-flop JK-PET.
    * L'uscita negata (-Q) è collegata all'ingresso K del secondo flip-flop JK-PET.
    * Il segnale -RES è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Secondo flip-flop di tipo JK-PET:**
    * Riceve il segnale di clock (CK) comune.
    * L'ingresso J è collegato all'uscita Q del primo flip-flop JK. L'ingresso K è collegato all'uscita negata (-Q) del pirmo filp-flop JK.
    * L'uscita (Q) è collegata al terzo degli ingressi della porta NAND.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RES è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Porta logica NAND:**
    * I suoi quattro ingressi sono collegati rispettivamente a:
        * Uscita (Q) del flip-flop D chiamata A.
        * Uscita (Q) del primo flip-flop JK chiamata B.
        * Uscita (Q) del secondo flip-flop JK chiamata C.
        * Segnale EN.
    * L'uscita della porta NAND produce il segnale D.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **-RES (Reset):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti i flip-flop a zero (Q=0).
* **EN (Enable):**  In ingresso alla porta NAND.
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""

prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronti 1-4: -RES è uno e EN è zero.
Tra fronte 4 e fronte 5: -RES passa da uno a zero e EN è zero.
Fronte 5: -RES è zero e EN è zero.
Tra fronte 5 e fronte 6: -RES passa da zero a uno e EN è zero.
Fronte 6: -RES è uno e EN è zero.
Tra fronte 6 e fronte 7: EN passa da zero a uno.
Fronti 7-12: -RES è uno e EN è uno.

Dati i valori sopra per gli ingressi –RES ed EN, esegui l’analisi temporale della rete sequenziale riportata in {image}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 12).

La descrizione della struttura delle rete è disponibile di seguito:
{image_description}
"""

## 1.1 Esercizio senza ulteriore contesto

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

Ecco l'analisi temporale della rete sequenziale, considerando i valori di ingresso forniti e lo stato iniziale dei flip-flop a 0:

| Fronte di Clock | -RES | EN | D | J1 | K1 | J2 | K2 | A | B | C | D |
|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 1 | X | X | X | X | X | X | **0** | **0** | **0** | **1** |
| 1 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | **1** | **0** | **0** | **1** |
| 2 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | **1** | **1** | **0** | **1** |
| 3 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | **1** | **1** | **1** | **1** |
| 4 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | **1** | **1** | **1** | **1** |
| 5 | 0 | 0 | X | X | X | X | X | **0** | **0** | **0** | **1** |
| 6 | 1 | 0 | 1 | 1 | 0 | 0 | 1 | **1** | **0** | **0** | **1** |
| 7 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | **1** | **1** | **0** | **0** |
| 8 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | **1** | **1** | **1** | **0** |
| 9 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | **1** | **1** | **1** | **0** |
| 10 | 1 | 1 | 1 

## 1.2 Esercizio con recupero diretto

In [17]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| Fronte di Clock | -RES | EN | A | B | C | D | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 1 | Tutti i flip-flop sono resettati a 0 da -RES. L'uscita D è 1 perché EN è 0. |
| 2 | 1 | 0 | 0 | 0 | 0 | 1 | Nessun cambiamento perché -RES è ancora attivo. |
| 3 | 1 | 0 | 0 | 0 | 0 | 1 | Nessun cambiamento perché -RES è ancora attivo. |
| 4 | 1 | 0 | 0 | 0 | 0 | 1 | Nessun cambiamento perché -RES è ancora attivo. |
| 5 | 0 | 0 | 0 | 0 | 0 | 1 | -RES passa a 0, ma non ha effetto sui flip-flop perché è attivo basso. |
| 6 | 1 | 0 | 0 | 0 | 0 | 1 | -RES torna a 1, i flip-flop rimangono a 0. |
| 7 | 1 | 1 | 0 | 0 | 0 | 1 | EN passa a 1, ma D rimane a 1 perché tutti gli ingressi della porta NAND sono 0. |
| 8 | 1 | 1 | 1 | 0 | 0 | 1 | Il flip-flop D memorizza il valore di D (1) al fronte di clock precedente. |
| 9 | 1 | 1 | 1 | 1 | 0 | 1 | Il primo flip-flop JK commuta perché J=1 e K=0. |
| 10 | 1 | 1 | 1 | 1 | 1 | 1 | Il secondo flip-flop JK commu

## 1.3 Esercizio con recupero diretto e video soluzione

In [18]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
Ecco l'analisi temporale della rete sequenziale, con i valori di uscita (A, B, C, D) a ogni fronte di clock (da 1 a 12):

| Fronte di Clock | -RES | EN | A | B | C | D |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 1 | ? | ? | 1 |
| 2 | 1 | 0 | 1 | 1 | ? | 1 |
| 3 | 1 | 0 | 1 | 1 | 1 | 1 |
| 4 | 1 | 0 | 1 | 1 | 1 | 1 |
| 5 | 0 | 0 | 0 | 0 | 0 | 1 |
| 6 | 1 | 0 | 0 | 0 | 0 | 1 |
| 7 | 1 | 1 | 1 | 0 | 0 | 1 |
| 8 | 1 | 1 | 1 | 1 | 0 | 1 |
| 9 | 1 | 1 | 1 | 1 | 1 | 0 |
| 10 | 1 | 1 | 0 | 1 | 1 | 1 |
| 11 | 1 | 1 | 1 | 0 | 1 | 1 |
| 12 | 1 | 1 | 1 | 1 | 0 | 1 |

**Note:**

* **?** indica uno stato indeterminato, poiché non abbiamo informazioni sufficienti per determinare il valore dell'uscita prima del primo fronte di salita del clock.
* I flip-flop di tipo D memorizzano il valore presente all'ingresso D al fronte di salita del clock.
* I flip-flop di tipo JK si comportano come segue:
    * J=1, K=0: l'uscita Q passa a 1.
    * J=0, K=1: l'uscita Q passa a 0.
    * 

## 1.4 Esercizio con video soluzione

In [19]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is analyzing a sequential network with four outputs (A, B, C, D) based on given input values for -RES and EN. Here is a breakdown of the analysis:

**Initial State (Before the First Clock Edge)**

* **EN is 0:** This means the output of the NAND gate (D) will be 1 regardless of the states of A, B, and C.
* **-RES is 1:** The flip-flops are not in reset.
* **A, B, C are indeterminate:** The initial states of the flip-flops are unknown.

**Clock Edge 1**

* **D is 1:** As EN is 0, the NAND gate output is forced to 1.
* **A transitions to 1:** The D flip-flop receives D=1 and updates its output (A) to 1 shortly after the clock edge.
* **B and C remain indeterminate:**  Their states depend on the previous indeterminate states of the flip-flops.

**Clock Edges 2-3**

* **D is 1:** EN is still 0.
* **A remains 1:** The D flip-flop continues to receive D=1.
* **B and C remain indeterminate.**

**Clock Edge 4**

* **D is 1:** EN is still 0.
* **A remains 1.**


## 1.5 Esercizio con libro di testo nel prompt

In [20]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Analisi temporale della rete sequenziale

La rete sequenziale presentata è un contatore Johnson a 3 bit con abilitazione e reset asincrono. Analizziamo il suo comportamento temporale passo dopo passo, considerando i valori di ingresso forniti e le caratteristiche dei flip-flop.

**Legenda:**

* **A:** Uscita Q del flip-flop D-PET
* **B:** Uscita Q del primo flip-flop JK-PET
* **C:** Uscita Q del secondo flip-flop JK-PET
* **D:** Uscita della porta NAND

**Stato iniziale:** All'inizio, tutti i flip-flop sono resettati a 0 a causa del segnale -RES attivo.

**Fronti 1-4:**

* -RES è uno (inattivo) e EN è zero.
* I flip-flop mantengono il loro stato precedente a causa dell'ingresso EN a zero che disabilita la porta NAND.
* **A = 0, B = 0, C = 0, D = 1** (NAND con EN a zero produce sempre 1).

**Tra fronte 4 e fronte 5:**

* -RES passa da uno a zero (attivo) e EN è zero.
* Il segnale -RES resetta tutti i flip-flop a zero, indipendentemente dal valore di EN.
* **A

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Analisi temporale della rete sequenziale

Ecco l'analisi temporale della rete sequenziale, con i valori di uscita (A, B, C, D) a ogni fronte di clock (da 1 a 12):

| Fronte CK | -RES | EN | A | B | C | D | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | ? | ? | ? | 1 | A, B, C indeterminati a causa del reset attivo prima del fronte 1. D=1 perché EN=0 forza l'uscita della NAND a 1. |
| 2 | 1 | 0 | 1 | ? | ? | 1 | A=1 perché D=1 al fronte 1. B e C rimangono indeterminati. D=1 come al fronte 1. |
| 3 | 1 | 0 | 1 | 1 | ? | 1 | B=1 perché A=1 al fronte 2 e J=1, K=0 per il JK-PET centrale. C rimane indeterminato. D=1 come ai fronti precedenti. |
| 4 | 1 | 0 | 1 | 1 | 1 | 1 | C=1 perché B=1 al fronte 3 e J=1, K=0 per il JK-PET di destra. D=1 come ai fronti precedenti. |
| 5 | 0 | 0 | 0 | 0 | 0 | 1 | -RES=0 forza A, B, C a 0. D=1 come ai fronti precedenti. |
| 6 | 1 | 0 | 0 | 0 | 0 | 1 | -RES torna a 1, ma A, B, C rimangono a 0. D=1 come ai fronti precedenti.

## 1.7 Esercizio con recupero mediante tool

In [22]:
prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.
Fronti 1-4: -RES è uno e EN è zero.
Tra fronte 4 e fronte 5: -RES passa da uno a zero e EN è zero.
Fronte 5: -RES è zero e EN è zero.
Tra fronte 5 e fronte 6: -RES passa da zero a uno e EN è zero.
Fronte 6: -RES è uno e EN è zero.
Tra fronte 6 e fronte 7: EN passa da zero a uno.
Fronti 7-12: -RES è uno e EN è uno.
Dati i valori sopra per gli ingressi –RES ed EN, esegui l’analisi temporale della rete sequenziale descritta:
{image_description}
Determina per ogni uscita il valore a ogni fronte di clock (da 1 a 12).
"""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

Una rete sequenziale è un circuito in cui i valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti. Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo. Data la rete sequenziale descritta, con quattro uscite (A, B, C, D), e i valori degli ingressi –RES ed EN, l'analisi temporale per ogni uscita a ogni fronte di clock (da 1 a 12) è:

**Fronte 1:** A = 0, B = 0, C = 0, D = 0
**Fronte 2:** A = 0, B = 0, C = 0, D = 0
**Fronte 3:** A = 0, B = 0, C = 0, D = 0
**Fronte 4:** A = 0, B = 0, C = 0, D = 0
**Fronte 5:** A = 0, B = 0, C = 0, D = 0
**Fronte 6:** A = 0, B = 0, C = 0, D = 0
**Fronte 7:** A = 0, B = 0, C = 0, D = 1
**Fronte 8:** A = 1, B = 0, C = 0, D = 1
**Fronte 9:** A = 1, B = 1, C = 0, D = 1
**Fronte 10:** A = 1, B = 1, C = 1, D = 0
**Fronte 11:** A = 0, B = 1, C = 1, D = 1
**Fronte 12:** A = 0, B = 0, C = 1, D = 1 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "1. A = 0, B = 0, C = 0, D = 0\n2. A = 0, B = 0, C = 0, D = 0\n3. A = 0, B = 0, C = 0, D = 0\n4. A = 0, B = 0, C = 0, D = 0\n5. A = 0, B = 0, C = 0, D = 0\n6. A = 0, B = 0, C = 0, D = 0\n7. A = 0, B = 0, C = 0, D = 0\n8. A = 0, B = 0, C = 0, D = 0\n9. A = 0, B = 0, C = 0, D = 0\n10. A = 0, B = 0, C = 0, D = 0\n11. A = 0, B = 0, C = 0, D = 0\n12. A = 0, B = 0"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "1. A = 0, B = 0, C = 0, D = 0\n2. A = 0, B = 0, C = 0, D = 0\n3. A = 0, B = 0, C = 0, D = 0\n4. A = 0, B = 0, C = 0, D = 0\n5. A = 0, B = 0, C = 0, D = 0\n6. A = 0, B = 0, C = 0, D = 0\n7. A = 0, B = 0, C = 0, D = 0\n8. A = 0, B = 0, C = 0, D = 0

## 1.8 Esercizio con few-shot prompting

In [23]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Fronte 1: A è zero, B è zero, C è zero, D è uno.
Tra fronte 1 e fronte 2: -RES passa da uno a zero. Quando -RES commuta, A, B e C passano da zero a uno.
Fronte 2: A è uno, B è uno, C è uno, D è zero.
Tra fronte 2 e fronte 3: -RES passa da zero a uno.
Fronte 3: A è zero, B è uno, C è uno, D è uno.
Fronte 4: A è uno, B è zero, C è uno, D è uno.
Fronte 5: A è zero, B è uno, C è zero, D è uno.
Tra fronte 5 e fronte 6: EN passa da zero a uno.
Fronte 6: A è uno, B è zero, C è uno, D è uno.
Fronte 7: A è zero, B è uno, C è zero, D è zero.
Fronte 8: A è uno, B è zero, C è uno, D è zero.
Fronte 9: A è zero, B è uno, C è zero, D è zero.
Fronte 10: A è uno, B è zero, C è uno, D è zero.
Fronte 11: A è zero, B è uno, C è zero, D è zero.
Fronte 12: A è uno, B è zero, C è uno, D è zero.


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [24]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Fronte 1: A è indeterminato, B è indeterminato, C è indeterminato, D è uno.
Tra fronte 1 e fronte 2: -RES passa da uno a zero. Quando -RES commuta, A, B e C passano da indeterminato a zero.
Fronte 2: A è zero, B è zero, C è zero, D è uno.
Tra fronte 2 e fronte 3: -RES passa da zero a uno.
Fronte 3: A è zero, B è zero, C è zero, D è uno.
Fronte 4: A è zero, B è zero, C è zero, D è uno.
Tra fronte 4 e fronte 5: -RES passa da uno a zero. Quando -RES commuta, A, B e C passano da zero a uno. Dopodiché -RES passa da zero a uno.
Fronte 5: A è uno, B è uno, C è uno, D è uno.
Tra fronte 5 e fronte 6: -RES passa da zero a uno.
Fronte 6: A è uno, B è uno, C è uno, D è uno.
Tra fronte 6 e fronte 7: EN passa da zero a uno.
Fronte 7: A passa da uno a zero poco dopo il fronte, B è uno, C è uno, D passa da uno a zero poco dopo il fronte.
Fronte 8: A passa da zero a uno poco dopo il fronte, B passa da uno a zero poco dopo il fronte, C è uno, D passa da zero a uno poco dopo il f

# Esercizio 12

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image12>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video12>.mp4")

image_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha quattro uscite (Q0, Q1, Q2, -TC). 
La struttura della rete è descritta di seguito: 

* **Primo flip-flop di tipo JK-PET:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Gli ingressi J e K sono collegati insieme e collegati al segnale di input EN.
    * L'uscita (Q) non è collegata a niente.
    * L'uscita negata (-Q) è collegata sia all'ingresso J che all'ingresso K del secondo flip-flop JK-PET e a un ingresso della porta logica AND. L'uscita -Q è chiamata Q0.
    * Il valore costante '1' è collegato all'ingresso -CLEAR del flip-flop.
    * Il segnale -PRESET è collegato al -PRESET del flip-flop.
* **Secondo flip-flop di tipo JK-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * Gli ingressi J e K sono collegati insieme e collegati all'uscita negata del primo flip-flop JK-PET.
    * L'uscita (Q) non è collegata a niente.
    * L'uscita negata (-Q) è collegata a un ingresso della porta logica AND. L'uscita -Q è chiamata Q1.
    * Il valore costante '1' è collegato all'ingresso -CLEAR del flip-flop.
    * Il segnale -PRESET è collegato al -PRESET del flip-flop.
* **Terzo flip-flop di tipo JK-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * Gli ingressi J e K sono collegati insieme e collegati all'uscita della porta logica AND.
    * L'uscita (Q) non è collegata a niente.
    * L'uscita negata (-Q) è collegata a un ingresso della porta logica NAND. L'uscita -Q è chiamata Q2.
    * Il valore costante '1' è collegato all'ingresso -CLEAR del flip-flop.
    * Il segnale -PRESET è collegato al -PRESET del flip-flop.
* **Porta logica AND:**
    * I suoi tre ingressi sono collegati rispettivamente a:
        * Uscita negata (-Q) del primo flip-flop JK-PET chiamata Q0.
        * Uscita negata (-Q) del secondo flip-flop JK-PET chiamata Q1.
        * Segnale EN.
    * L'uscita della porta AND è collegata a un ingresso della porta NAND.
* **Porta logica NAND:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita della porta logica AND.
        * Uscita negata (-Q) del terzo flip-flop JK-PET chiamata Q2.
    * L'uscita della porta NAND produce il segnale di output -TC.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **-PRESET (Preset):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti i flip-flop a uno (Q=1).
* **EN (Enable):**  In ingresso alla porta AND e al primo flip-flop JK-PET.
* ** '1' (alto):** Si tratta di segnale costante a 1.  
"""

prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronte 1: -PRESET è uno ed EN è zero.
Tra fronte 1 e fronte 2: -PRESET passa da uno a zero e subito dopo da zero a uno ed EN è zero. 
Fronte 2: -PRESET è uno ed EN è zero.
Tra fronte 2 e fronte 3: -PRESET è uno e EN passa da zero a uno.
Fronti 3-9: -PRESET è uno ed EN è uno.
Tra fronte 9 e fronte 10: -PRESET è uno ed EN passa da uno a zero.
Fronti 10-16: -RESET è uno ed EN è zero.

Dati i valori sopra per gli ingressi -PRESET ed EN, esegui l’analisi temporale della rete sequenziale riportata in {image}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 16).

La descrizione della struttura delle rete è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [10]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
| Fronte di Clock | -PRESET | EN | Q0 | Q1 | Q2 | -TC | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | -PRESET è attivo, tutti i flip-flop sono forzati a 1. |
| 2 | 1 | 0 | 1 | 1 | 1 | 1 | -PRESET è stato momentaneamente a 0 tra i fronti 1 e 2, ma questo non ha effetto perché è attivo basso e il fronte di clock arriva quando è di nuovo a 1. |
| 3 | 1 | 1 | 0 | 1 | 1 | 1 | Il primo flip-flop commuta perché J=K=EN=1. |
| 4 | 1 | 1 | 1 | 0 | 1 | 1 | Il secondo flip-flop commuta perché J=K=Q0=1. |
| 5 | 1 | 1 | 0 | 1 | 1 | 1 | Il primo flip-flop commuta perché J=K=EN=1. |
| 6 | 1 | 1 | 1 | 0 | 0 | 0 | Il secondo e terzo flip-flop commutano perché J=K=Q0=1 e l'uscita della AND è 1. |
| 7 | 1 | 1 | 0 | 1 | 1 | 1 | Il primo e terzo flip-flop commutano. |
| 8 | 1 | 1 | 1 | 0 | 1 | 1 | Il secondo flip-flop commuta. |
| 9 | 1 | 1 | 0 | 1 | 1 | 1 | Il primo flip-flop commuta. |
| 10 | 1 | 0 | 1 | 0 | 1 | 1 | Il secondo flip-flop commuta, EN diventa

## 1.2 Esercizio con recupero diretto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

Ecco l'analisi temporale della rete sequenziale, tenendo conto dei valori di -PRESET ed EN forniti e considerando che i flip-flop sono attivati dal fronte di salita del clock:

| Fronte | -PRESET | EN | Q0 | Q1 | Q2 | -TC | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | -PRESET forza tutti i flip-flop a 1. |
| 2 | 1 | 0 | 1 | 1 | 1 | 0 | -PRESET rimane a 1, quindi i flip-flop mantengono il loro stato. |
| 3 | 1 | 1 | 0 | 1 | 1 | 0 | Il primo flip-flop commuta perché J=K=EN=1. |
| 4 | 1 | 1 | 1 | 0 | 1 | 0 | Il secondo flip-flop commuta perché J=K=-Q0=1. |
| 5 | 1 | 1 | 0 | 1 | 1 | 0 | Il primo flip-flop commuta. |
| 6 | 1 | 1 | 1 | 0 | 0 | 1 | Il secondo e il terzo flip-flop commutano. |
| 7 | 1 | 1 | 0 | 1 | 0 | 1 | Il primo flip-flop commuta. |
| 8 | 1 | 1 | 1 | 0 | 1 | 0 | Il secondo e il terzo flip-flop commutano. |
| 9 | 1 | 1 | 0 | 1 | 1 | 0 | Il primo flip-flop commuta. |
| 10 | 1 | 0 |

## 1.3 Esercizio con recupero diretto e video soluzione

In [12]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
Certo, ecco l'analisi temporale della rete sequenziale:

**Stato iniziale:**

* Fronte 1: -PRESET è uno, EN è zero. Tutti i flip-flop sono in stato indeterminato (Q0, Q1, Q2 = ?) poiché non conosciamo il loro stato precedente. -TC è uno perché EN è zero, quindi l'uscita della porta AND è zero, e la porta NAND con un ingresso a zero ha uscita uno.

**Tra Fronte 1 e Fronte 2:**

* -PRESET passa da uno a zero e subito dopo da zero a uno. Questo impulso forza le uscite dei flip-flop a uno (Q0, Q1, Q2 = 1). EN rimane a zero, quindi -TC rimane a uno.

**Fronte 2:**

* -PRESET è uno, EN è zero.  I flip-flop mantengono il loro stato precedente (Q0, Q1, Q2 = 1). -TC rimane a uno.

**Tra Fronte 2 e Fronte 3:**

* -PRESET è uno, EN passa da zero a uno. I flip-flop mantengono il loro stato (Q0, Q1, Q2 = 1). -TC rimane a uno.

**Fronte 3:**

* -PRESET è uno, EN è uno. Il primo flip-flop riceve J=K=1 (da EN) e commuta, passando da Q0=1 a Q0=0. Il secondo flip-flop riceve J=K

## 1.4 Esercizio con video soluzione

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
Il video fornito spiega come analizzare il diagramma temporale della rete sequenziale. Ecco la tabella che mostra i valori delle uscite Q0, Q1, Q2 e -TC a ogni fronte di clock (da 1 a 16):

| Fronte di Clock | -PRESET | EN | Q0 | Q1 | Q2 | -TC |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 1 |
| 2 | 1 | 0 | 0 | 0 | 0 | 1 |
| 3 | 1 | 1 | 1 | 0 | 0 | 1 |
| 4 | 1 | 1 | 0 | 1 | 0 | 1 |
| 5 | 1 | 1 | 1 | 0 | 0 | 1 |
| 6 | 1 | 1 | 0 | 1 | 0 | 1 |
| 7 | 1 | 1 | 1 | 0 | 0 | 1 |
| 8 | 1 | 1 | 0 | 1 | 0 | 1 |
| 9 | 1 | 1 | 1 | 0 | 1 | 1 |
| 10 | 1 | 0 | 1 | 1 | 1 | 0 |
| 11 | 1 | 0 | 1 | 1 | 1 | 1 |
| 12 | 1 | 0 | 1 | 1 | 1 | 1 |
| 13 | 1 | 0 | 1 | 1 | 1 | 1 |
| 14 | 1 | 0 | 1 | 1 | 1 | 1 |
| 15 | 1 | 0 | 1 | 1 | 1 | 1 |
| 16 | 1 | 0 | 1 | 1 | 1 | 1 |

**Note:**

* I valori di Q0, Q1 e Q2 al fronte di clock 1 sono indeterminati perché non è specificato lo stato iniziale dei flip-flop.
* Tra il fronte 1 e il fronte 2, -PRESET passa da 1 a 0 e subito dopo da 0 a

## 1.5 Esercizio con libro di testo nel prompt

In [14]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Analisi temporale della rete sequenziale

**Tabella riassuntiva dei valori delle uscite a ogni fronte di clock:**

| Fronte | -PRESET | EN | Q0 | Q1 | Q2 | -TC | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 1 | 1 | 1 | 1 | Inizializzazione con -PRESET |
| 2 | 1 | 0 | 1 | 1 | 1 | 1 | Nessuna variazione, EN=0 |
| 3 | 1 | 1 | 0 | 1 | 1 | 1 | Q0 cambia stato, EN=1 |
| 4 | 1 | 1 | 1 | 0 | 1 | 1 | Q1 cambia stato |
| 5 | 1 | 1 | 0 | 1 | 0 | 1 | Q2 cambia stato |
| 6 | 1 | 1 | 1 | 0 | 1 | 1 | Q0 cambia stato |
| 7 | 1 | 1 | 0 | 1 | 0 | 1 | Q2 cambia stato |
| 8 | 1 | 1 | 1 | 0 | 1 | 1 | Q0 cambia stato |
| 9 | 1 | 1 | 0 | 1 | 0 | 1 | Q2 cambia stato |
| 10 | 1 | 0 | 1 | 0 | 1 | 1 | Nessuna variazione, EN=0 |
| 11 | 1 | 0 | 1 | 0 | 1 | 1 | Nessuna variazione, EN=0 |
| 12 | 1 | 0 | 1 | 0 | 1 | 1 | Nessuna variazione, EN=0 |
| 13 | 1 | 0 | 1 | 0 | 1 | 1 | Nessuna variazione, EN=0 |
| 14 | 1 | 0 | 1 | 0 | 1 | 1 | Nessuna variazione, EN=0 |
| 15 | 1 | 0 | 1 | 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
```
Fronte | -PRESET | EN | Q0 | Q1 | Q2 | -TC
------- | -------- | -------- | -------- | -------- | -------- | --------
1       | 1        | 0        | 1        | 1        | 1        | 1
2       | 1        | 0        | 1        | 1        | 1        | 1
3       | 1        | 1        | 0        | 1        | 1        | 1
4       | 1        | 1        | 1        | 0        | 1        | 1
5       | 1        | 1        | 0        | 1        | 1        | 1
6       | 1        | 1        | 1        | 0        | 1        | 1
7       | 1        | 1        | 0        | 1        | 1        | 1
8       | 1        | 1        | 1        | 0        | 1        | 1
9       | 1        | 1        | 0        | 1        | 1        | 1
10      | 1        | 0        | 1        | 0        | 1        | 1
11      | 1        | 0        | 1        | 0        | 1        | 1
12      | 1        | 0        | 1        | 0        | 1        | 1
13      | 1        | 0        | 1        | 0      

## 1.7 Esercizio con recupero mediante tool

In [16]:
prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronte 1: -PRESET è uno ed EN è zero.
Tra fronte 1 e fronte 2: -PRESET passa da uno a zero e subito dopo da zero a uno ed EN è zero. 
Fronte 2: -PRESET è uno ed EN è zero.
Tra fronte 2 e fronte 3: -PRESET è uno e EN passa da zero a uno.
Fronti 3-9: -PRESET è uno ed EN è uno.
Tra fronte 9 e fronte 10: -PRESET è uno ed EN passa da uno a zero.
Fronti 10-16: -RESET è uno ed EN è zero.

Dati i valori sopra per gli ingressi -PRESET ed EN, esegui l’analisi temporale della rete sequenziale descritta:
{image_description}

Determina per ogni uscita il valore a ogni fronte di clock (da 1 a 16)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## Analisi temporale della rete sequenziale:

**Stato iniziale (fronte 1):**

* Tutti i flip-flop hanno -PRESET a 0, quindi le uscite Q0, Q1 e Q2 sono a 1.
* EN è a 0, quindi l'uscita della porta AND è a 0.
* L'uscita della porta NAND (-TC) è a 1.

**Tra fronte 1 e 2:**

* -PRESET passa da 1 a 0 e subito dopo da 0 a 1, ma questo non ha effetto perché i flip-flop cambiano stato solo sul fronte di salita del clock.

**Fronte 2 - 9:**

* -PRESET è a 1 e EN è a 0 fino al fronte 3, poi EN passa a 1.
* I flip-flop rimangono nello stato precedente (Q0 = Q1 = Q2 = 1) perché EN è a 0 fino al fronte 3.
* Dal fronte 3 in poi, EN è a 1, quindi:
    * Il primo flip-flop JK-PET commuta ad ogni fronte di clock perché J e K sono collegati a EN. Q0 si alterna tra 0 e 1.
    * Il secondo flip-flop JK-PET riceve in ingresso Q0 negato, quindi commuta in modo opposto al primo. Q1 si alterna tra 1 e 0.
    * La porta AND ha in ingresso Q0, Q1 ed EN. La sua uscita è a 1 solo quando Q0, Q1 ed EN sono tutti a 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1."
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1."
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [17]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Fronte 1: Q0 è uno, Q1 è uno, Q2 è uno, -TC è zero.
Tra fronte 1 e fronte 2: -PRESET passa da uno a zero e subito dopo da zero a uno. Quando -PRESET va a zero, Q0, Q1 e Q2 passano da uno a uno. Quando -PRESET torna a uno, Q0, Q1 e Q2 rimangono a uno.
Fronte 2: Q0 è uno, Q1 è uno, Q2 è uno, -TC è zero.
Tra fronte 2 e fronte 3: EN passa da zero a uno.
Fronte 3: Q0 passa da uno a zero poco dopo il fronte, Q1 è uno, Q2 è uno, -TC è zero.
Fronte 4: Q0 passa da zero a uno poco dopo il fronte, Q1 passa da uno a zero poco dopo il fronte, Q2 è uno, -TC è zero.
Fronte 5: Q0 passa da uno a zero poco dopo il fronte, Q1 passa da zero a uno poco dopo il fronte, Q2 è uno, -TC è zero.
Fronte 6: Q0 passa da zero a uno poco dopo il fronte, Q1 passa da uno a zero poco dopo il fronte, Q2 passa da uno a zero poco dopo il fronte, -TC è uno.
Fronte 7: Q0 passa da uno a zero poco dopo il fronte, Q1 passa da zero a uno poco dopo il fronte, Q2 passa da zero a uno poco dopo il fronte, -T

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [18]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Fronte 1: Q0 è uno, Q1 è uno, Q2 è uno, -TC è uno.
Tra fronte 1 e fronte 2: -PRESET passa da uno a zero e subito dopo da zero a uno.
Fronte 2: Q0 è uno, Q1 è uno, Q2 è uno, -TC è uno.
Tra fronte 2 e fronte 3: EN passa da zero a uno.
Fronte 3: Q0 passa da uno a zero poco dopo il fronte, Q1 è uno, Q2 è uno, -TC è uno.
Fronte 4: Q0 passa da zero a uno poco dopo il fronte, Q1 passa da uno a zero poco dopo il fronte, Q2 è uno, -TC è uno.
Fronte 5: Q0 passa da uno a zero poco dopo il fronte, Q1 passa da zero a uno poco dopo il fronte, Q2 è uno, -TC è uno.
Fronte 6: Q0 passa da zero a uno poco dopo il fronte, Q1 passa da uno a zero poco dopo il fronte, Q2 passa da uno a zero poco dopo il fronte, -TC è zero.
Fronte 7: Q0 passa da uno a zero poco dopo il fronte, Q1 passa da zero a uno poco dopo il fronte, Q2 passa da zero a uno poco dopo il fronte, -TC è uno.
Fronte 8: Q0 passa da zero a uno poco dopo il fronte, Q1 passa da uno a zero poco dopo il fronte, Q2 è uno, -TC 

# Esercizio 13

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image13>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video13>.mp4")

image_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha quattro uscite (A, B, C, D). 
La struttura della rete è descritta di seguito: 

* **Primo flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Il suo ingresso dati (D) è collegato all'uscita della seconda porta logica EXOR.
    * L'uscita (Q) è collegata all'ingresso D del secondo flip-flop D-PET. L'uscita Q è chiamata D.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Secondo flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * Il suo ingresso dati (D) è collegato all'uscita del primo flip-flop D-PET. 
    * L'uscita (Q) è collegata all'ingresso D del terzo flip-flop D-PET. L'uscita Q è chiamata C.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Terzo flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * Il suo ingresso dati (D) è collegato all'uscita del secondo flip-flop D-PET.
    * L'uscita (Q) è collegata all'ingresso D del quarto flip-flop D-PET e a un ingresso della prima porta logica EXOR. L'uscita Q è chiamata B.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Quarto flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Il suo ingresso dati (D) è collegato all'uscita del terzo flip-flop D-PET.
    * L'uscita (Q) è collegata a un ingresso della prima porta logica EXOR. L'uscita Q è chiamata A.
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Prima porta logica EXOR:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del terzo flip-flop D-PET chiamata B.
        * Uscita (Q) del quarto flip-flop D-PET chiamata A.
    * L'uscita della prima porta EXOR è collegata a un ingresso della seconda porta EXOR.
* **Seconda porta logica EXOR:**
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita della prima porta EXOR.
        * Segnale IN.
    * L'uscita della seconda porta EXOR è collegata all'ingresso D del primo flip-flop D-PET.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **-RESET (Reset):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti i flip-flop a zero (Q=0).
* **IN (Input):**  In ingresso alla seconda porta EXOR.
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""

prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronte 1: -RESET è uno e IN è zero.
Tra fronte 1 e fronte 2: -RESET passa da uno a zero e IN è zero.
Fronte 2: -RESET è zero e IN è zero.
Tra fronte 2 e fronte 3: -RESET passa da zero a uno e IN è zero.
Fronti 3-4: -RESET è uno e IN è zero.
Tra fronte 4 e fronte 5: -RESET è uno ed IN passa da zero a uno.
Fronti 5-13: -RESET è uno ed IN è uno.

Dati i valori sopra per gli ingressi -RESET ed IN, esegui l’analisi temporale della rete sequenziale riportata in {image}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 13).

La descrizione della struttura delle rete è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [27]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

Di seguito l'analisi temporale della rete sequenziale, determinando il valore di ogni uscita (A, B, C, D) ad ogni fronte di clock (da 1 a 13), dati i valori di ingresso -RESET e IN.

**Nota:** 

* Assumiamo che i flip-flop siano sensibili al fronte di salita del clock (CK).
* "X" indica uno stato indeterminato.

| Fronte | -RESET | IN | D | C | B | A | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 0 | -RESET è attivo, quindi tutti i flip-flop sono resettati a 0. |
| 2 | 0 | 0 | 1 | 0 | 0 | 0 | -RESET diventa inattivo. L'uscita della seconda porta EXOR è 1 (1 XOR 0 = 1), quindi D passa a 1 al fronte di clock. |
| 3 | 1 | 0 | 0 | 1 | 0 | 0 | D passa a 0 a causa del -RESET. C passa a 1 perché riceve il valore precedente di D (1). |
| 4 | 1 | 0 | 0 | 0 | 1 | 0 | B passa a 1 perché riceve il valore precedente di C (1). |
| 5 | 1 | 1 | 0 | 0 | 0 | 1 | A passa a 1 perché riceve il valore precedente di B

## 1.2 Esercizio con recupero diretto

In [28]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

**Tabella dei valori:**

| Fronte di Clock | -RESET | IN | D | C | B | A |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 0 | 
| 2 | 0 | 0 | 0 | 0 | 0 | 0 | 
| 3 | 1 | 0 | 0 | 0 | 0 | 0 | 
| 4 | 1 | 0 | 0 | 0 | 0 | 0 | 
| 5 | 1 | 1 | 1 | 0 | 0 | 0 | 
| 6 | 1 | 1 | 1 | 1 | 0 | 0 | 
| 7 | 1 | 1 | 0 | 1 | 1 | 0 | 
| 8 | 1 | 1 | 0 | 0 | 1 | 1 | 
| 9 | 1 | 1 | 1 | 0 | 1 | 1 | 
| 10 | 1 | 1 | 1 | 1 | 0 | 1 | 
| 11 | 1 | 1 | 0 | 1 | 1 | 0 | 
| 12 | 1 | 1 | 0 | 0 | 1 | 1 | 
| 13 | 1 | 1 | 1 | 0 | 1 | 1 | 

**Spiegazione:**

* **Fronte 1:**  -RESET è attivo (1), quindi tutti i flip-flop sono forzati a 0 (Q=0) indipendentemente dagli altri ingressi.
* **Fronte 2:** -RESET passa a 0, ma i flip-flop mantengono il loro stato precedente (0) fino al successivo fronte di clock.
* **Fronte 3:** -RESET torna a 1 e i flip-flop rimangono a 0. Da questo punto in poi, il comportamento del circuito è determinato dagli ingressi D e

## 1.3 Esercizio con recupero diretto e video soluzione

In [29]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La rete sequenziale presentata nell'immagine è un registro a scorrimento a 4 bit con retroazione tramite due porte XOR. L'analisi temporale della rete, considerando i valori di ingresso forniti per -RESET e IN, e determinando i valori di uscita (A, B, C, D) ad ogni fronte di clock (da 1 a 13), è la seguente:

**Fronte 1:**
* -RESET è 1, IN è 0.
* Lo stato iniziale dei flip-flop è indeterminato.
* A, B, C, D sono indeterminati.

**Tra Fronte 1 e Fronte 2:**
* -RESET passa da 1 a 0, IN è 0.
* L'attivazione di -RESET forza tutti i flip-flop a 0.
* A, B, C, D sono 0.

**Fronte 2:**
* -RESET è 0, IN è 0.
* I flip-flop mantengono il loro stato precedente (0).
* A, B, C, D sono 0.

**Tra Fronte 2 e Fronte 3:**
* -RESET passa da 0 a 1, IN è 0.
* -RESET si disattiva, consentendo ai flip-flop di cambiare stato in base agli ingressi.
* A, B, C, D sono 0.

**Fronti 3-4:**
* -RESET è 1, IN è 0.
* Il primo flip-flop riceve in ingresso 0 (IN XOR (A XOR B)).
* Gli altri flip-f

## 1.4 Esercizio con video soluzione

In [30]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video shows the step by step analysis of a sequential network. The network is composed of 4 D-PET flip-flops and 2 XOR gates. The inputs to the network are IN, Clock and Reset. The outputs of the network are A, B, C and D.

The video starts by showing the initial state of the network, where all the flip-flops are in an indeterminate state. Then, the video shows how the network evolves over time, as the inputs change.

The speaker explains how the different components of the network work, and how they interact with each other. The speaker also explains how to read the timing diagram, and how to determine the state of the network at any given time.

The video ends by showing the final state of the network, after all the inputs have been applied.

Here is a summary of the key points from the video:

* The D-PET flip-flop is a type of flip-flop that stores the value of its D input on the rising edge of the clock signal.
* The XOR gate is a logic gate that outpu

## 1.5 Esercizio con libro di testo nel prompt

In [31]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Analisi temporale della rete sequenziale

La rete sequenziale presentata è un registro a scorrimento a 4 bit con retroazione dalla sua uscita A all'ingresso del primo flip-flop D-PET, modificato da due porte EXOR. La prima porta EXOR riceve in ingresso A e B, mentre la seconda riceve l'uscita della prima EXOR e il segnale IN.

**Tabella di analisi temporale:**

| Fronte | -RESET | IN | A | B | C | D | Note |
|---|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | 0 | -RESET attivo, tutti i flip-flop azzerati |
| 2 | 0 | 0 | 0 | 0 | 0 | 0 | -RESET disattivato, nessun cambiamento |
| 3 | 1 | 0 | 0 | 0 | 0 | 0 | -RESET attivo, nessun cambiamento |
| 4 | 1 | 0 | 0 | 0 | 0 | 0 | Nessun cambiamento |
| 5 | 1 | 1 | 0 | 0 | 0 | 1 | IN diventa 1, D cambia a 1 |
| 6 | 1 | 1 | 1 | 0 | 0 | 0 | Scorrimento: A=D(fronte precedente) |
| 7 | 1 | 1 | 0 | 1 | 0 | 1 | Scorrimento: B=A, D=EXOR(B,A,IN) |
| 8 | 1 | 1 | 1 | 0 | 1 | 0 | Scorrimento: A=D, B=C, D=EXOR(B,A,IN) |
| 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [32]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Analisi temporale della rete sequenziale

Ecco l'analisi temporale della rete sequenziale, mostrando il valore di ogni uscita (A, B, C, D) ad ogni fronte di clock (da 1 a 13):

| Fronte di clock | -RESET | IN | D | C | B | A |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | **0** | **0** | **0** | **0** |
| 2 | 0 | 0 | 0 | 0 | 0 | 0 |
| 3 | 1 | 0 | **0** | **0** | **0** | **0** |
| 4 | 1 | 0 | 0 | 0 | 0 | 0 |
| 5 | 1 | 1 | **1** | **0** | **0** | **0** |
| 6 | 1 | 1 | 1 | **1** | **0** | **0** |
| 7 | 1 | 1 | 1 | 1 | **1** | **0** |
| 8 | 1 | 1 | 0 | 1 | 1 | **1** |
| 9 | 1 | 1 | 1 | 0 | 1 | 1 |
| 10 | 1 | 1 | 1 | 1 | 0 | 1 |
| 11 | 1 | 1 | 0 | 1 | 1 | 0 |
| 12 | 1 | 1 | 1 | 0 | 1 | 1 |
| 13 | 1 | 1 | 1 | 1 | 0 | 1 |

**Spiegazione:**

* **Fronte 1:** Il segnale -RESET è attivo, forzando tutte le uscite (A, B, C, D) a 0.
* **Tra fronte 1 e fronte 2:**  -RESET passa a 0, ma le uscite rimangono a 0 perché il segnale di reset è asincrono e agisce indipendentemente 

## 1.7 Esercizio con recupero mediante tool

In [33]:
prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo. 

Fronte 1: -RESET è uno e IN è zero.
Tra fronte 1 e fronte 2: -RESET passa da uno a zero e IN è zero.
Fronte 2: -RESET è zero e IN è zero.
Tra fronte 2 e fronte 3: -RESET passa da zero a uno e IN è zero.
Fronti 3-4: -RESET è uno e IN è zero.
Tra fronte 4 e fronte 5: -RESET è uno ed IN passa da zero a uno.
Fronti 5-13: -RESET è uno ed IN è uno.

Dati i valori sopra per gli ingressi -RESET ed IN, esegui l’analisi temporale della rete sequenziale descritta:
{image_description} 

Determina per ogni uscita il valore a ogni fronte di clock (da 1 a 13)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## Analisi temporale della rete sequenziale:

**Stato iniziale (prima del fronte 1):**

- Tutti i flip-flop sono in stato di reset (-RESET = 0), quindi A = B = C = D = 0.

**Fronte 1 (-RESET = 1, IN = 0):**

- I flip-flop escono dallo stato di reset.
- D = 1 (a causa del -PRESET del primo flip-flop).
- Gli altri flip-flop rimangono a 0 (C = B = A = 0) perché i loro ingressi D sono 0.

**Tra fronte 1 e fronte 2 (-RESET passa da 1 a 0, IN = 0):**

- I flip-flop tornano allo stato di reset (A = B = C = D = 0).

**Fronte 2 (-RESET = 0, IN = 0):**

- I flip-flop rimangono a 0 (A = B = C = D = 0) perché sono in stato di reset.

**Tra fronte 2 e fronte 3 (-RESET passa da 0 a 1, IN = 0):**

- I flip-flop escono dallo stato di reset.
- D = 1 (a causa del -PRESET del primo flip-flop).
- Gli altri flip-flop rimangono a 0 (C = B = A = 0).

**Fronti 3-4 (-RESET = 1, IN = 0):**

- D = 1, C = 1, B = 0, A = 0.

**Tra fronte 4 e fronte 5 (-RESET = 1, IN passa da 0 a 1):**

- Nessun cambiamento avviene 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "1. 1 0 0 0\n2. 0 0 0 0\n3. 0 0 0 0\n4. 0 0 0 0\n5. 0 1 0 0\n6. 0 1 0 0\n7. 0 1 0 0\n8. 0 1 0 0\n9. 0 1 0 0\n10. 0 1 0 0\n11. 0 1 0 0\n12. 0 1 0 0\n13. 0 1 0 0"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "1. 1 0 0 0\n2. 0 0 0 0\n3. 0 0 0 0\n4. 0 0 0 0\n5. 0 1 0 0\n6. 0 1 0 0\n7. 0 1 0 0\n8. 0 1 0 0\n9. 0 1 0 0\n10. 0 1 0 0\n11. 0 1 0 0\n12. 0 1 0 0\n13. 0 1 0 0"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [34]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Fronte 1: A è zero, B è zero, C è zero, D è zero.
Tra fronte 1 e fronte 2: L’ingresso -RESET passa da uno a zero. Quando -RESET commuta, A, B, C e D passano da zero a indeterminato. Dopodiché l’ingresso -RESET passa da zero a uno. 
Fronte 2: A è indeterminato, B è indeterminato, C è indeterminato, D è indeterminato.
Tra fronte 2 e fronte 3: L’ingresso -RESET passa da zero a uno. 
Fronti 3-4: A è indeterminato, B è indeterminato, C è indeterminato, D è indeterminato.
Tra fronte 4 e fronte 5: L’ingresso IN passa da zero a uno.
Fronti 5-13: A è indeterminato, B è indeterminato, C è indeterminato, D è indeterminato. 

L’uscita è indeterminata da fronte 2 a 13 perché il segnale -RESET non deve andare a livello logico basso quando il clock è attivo. 


## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [35]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Fronte 1: A è indeterminato, B è indeterminato, C è indeterminato, D è indeterminato.
Tra fronte 1 e fronte 2: -RESET passa da uno a zero. Quando -RESET commuta, A, B, C e D passano da indeterminato a zero.
Fronte 2: A è zero, B è zero, C è zero, D è zero.
Tra fronte 2 e fronte 3: -RESET passa da zero a uno. 
Fronte 3: A è zero, B è zero, C è zero, D è zero.
Fronte 4: A è zero, B è zero, C è zero, D è zero.
Tra fronte 4 e fronte 5: IN passa da zero a uno.
Fronte 5: A è zero, B è zero, C è zero, D passa da zero a uno poco dopo il fronte.
Fronte 6: A è zero, B è zero, C passa da zero a uno poco dopo il fronte, D passa da uno a zero poco dopo il fronte.
Fronte 7: A è zero, B passa da zero a uno poco dopo il fronte, C passa da uno a zero poco dopo il fronte, D passa da zero a uno poco dopo il fronte.
Fronte 8: A passa da zero a uno poco dopo il fronte, B passa da uno a zero poco dopo il fronte, C passa da zero a uno poco dopo il fronte, D è uno.
Fronte 9: A passa d

# Esercizio 14

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image14>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video14>.mp4")

image_description = """La rete sequenziale è sincrona (tutti i flip-flop condividono lo stesso clock) e ha tre uscite (O1, O2, O3). 
La struttura della rete è descritta di seguito: 

* **Primo flip-flop di tipo JK-PET:** 
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * Gli ingressi J e K sono collegati insieme e collegati al segnale di input IN.
    * L'uscita (Q) è collegata all'ingresso D del flip-flop D-PET. L'uscita Q è chiamata O1. 
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Flip-flop di tipo D-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * L'ingresso D è collegato all'uscita del primo flip-flop JK-PET.
    * L'uscita (Q) non è collegata a niente. L'uscita Q è chiamata O2. 
    * L'uscita negata (-Q) è collegata agli ingressi J e K del secondo flip-flop JK-PET.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.
* **Seconda flip-flop di tipo JK-PET:** 
    * Riceve il segnale di clock (CK) comune.
    * Gli ingressi J e K sono collegati insieme e collegati all'uscita negata del flip-flop D-PET.
    * L'uscita (Q) non è collegata a niente. L'uscita Q è chiamata O3. 
    * L'uscita negata (-Q) non è collegata a niente.
    * Il segnale -RESET è collegato al -CLEAR del flip-flop.
    * Il valore costante '1' è collegato all'ingresso -PRESET del flip-flop.

**Segnali di controllo:**

* **CK (Clock):**  Determina la temporizzazione del circuito, attivando i flip-flop ad ogni fronte di salita.
* **-RESET (Reset):**  Si tratta di un segnale attivo-basso. Quando attivo (0), forza le uscite di tutti i flip-flop a zero (Q=0).
* **IN (Enable):**  In ingresso a J e K del primo flip-flop JK-PET.
* ** '1' (alto):** Si tratta di segnale costante a 1. 
"""

prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronti 1-2: -RESET è uno e IN è zero.
Tra fronte 2 e fronte 3: -RESET passa da uno a zero e subito dopo da zero a uno e IN è zero. 
Fronti 3-5: -RESET è uno e IN è zero.
Tra fronte 5 e fronte 6: -RESET è uno e IN passa da zero a uno.
Fronti 6-11: -RESET è uno e IN è uno.
Tra fronte 11 e fronte 12: -RESET è uno e IN passa da uno a zero.
Fronti 12-21: -RESET è uno e IN è zero.

Dati i valori sopra per gli ingressi -RESET ed IN, esegui l’analisi temporale della rete sequenziale riportata in {image}, determinando per ogni uscita il valore a ogni fronte di clock (da 1 a 21).

La descrizione della struttura delle rete è disponibile di seguito:
{image_description}"""

## 1.1 Esercizio senza ulteriore contesto

In [37]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

**Stato iniziale (fronte 1):**

* -RESET = 1 (inattivo)
* IN = 0
* Assumiamo che tutti i flip-flop siano inizialmente a 0 (Q1 = Q2 = Q3 = 0)

**Fronte 2:**

* -RESET = 1 (inattivo)
* IN = 0
* Nessuna variazione negli ingressi, le uscite rimangono invariate:
    * O1 = 0
    * O2 = 0
    * O3 = 0

**Tra fronte 2 e fronte 3:**

* -RESET passa da 1 a 0 e subito dopo da 0 a 1
* IN = 0
* Il breve impulso di -RESET porta tutti i flip-flop a 0. Tuttavia, poiché questo avviene tra due fronti di clock, le uscite non cambiano al fronte 3.

**Fronte 3:**

* -RESET = 1 (inattivo)
* IN = 0
* Le uscite rimangono a 0:
    * O1 = 0
    * O2 = 0
    * O3 = 0

**Fronti 4-5:**

* -RESET = 1 (inattivo)
* IN = 0
* Nessuna variazione negli ingressi, le uscite rimangono invariate:
    * O1 = 0
    * O2 = 0
    * O3 = 0

**Tra fronte 5 e fronte 6:**

* -RESET = 1 (inattivo)
* IN passa da 0 a 1
* Il primo flip-flop JK-PET ha J = K = 1, quind

## 1.2 Esercizio con recupero diretto

In [38]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Analisi temporale della rete sequenziale

**Tabella dei valori:**

| Fronte | -RESET | IN | O1 (JK1_Q) | O2 (D_Q) | O3 (JK2_Q) | Note |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | Inizializzazione a 0 per -RESET = 1 |
| 2 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 3 | 1 | 0 | 0 | 0 | 0 | -RESET impulsivo, non cambia lo stato |
| 4 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 5 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 6 | 1 | 1 | 0 | 0 | 0 | IN diventa 1, ma O1 cambia al fronte successivo |
| 7 | 1 | 1 | 1 | 0 | 0 | JK1 toggle (J=K=1) |
| 8 | 1 | 1 | 0 | 1 | 0 | JK1 toggle, D memorizza O1 precedente |
| 9 | 1 | 1 | 0 | 0 | 1 | JK1 toggle, D memorizza O1 precedente, JK2 toggle |
| 10 | 1 | 1 | 1 | 0 | 0 | JK1 toggle, D memorizza O1 precedente, JK2 toggle |
| 11 | 1 | 1 | 0 | 1 | 0 | JK1 toggle, D memorizza O1 precedente |
| 12 | 1 | 0 | 0 | 0 | 1 | IN diventa 0, JK1 toggle, D memorizza O1 precedente, JK2 toggle |
| 13 | 1 | 0 | 0 | 0 | 1 | Ne

## 1.3 Esercizio con recupero diretto e video soluzione

In [39]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
Ecco l'analisi temporale della rete sequenziale:

| Fronte di clock | -RESET | IN | O1 | O2 | O3 | Note |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | X | X | X |  Stato iniziale indeterminato |
| 2 | 1 | 0 | 0 | X | X |  Il primo flip-flop JK-PET mantiene il suo stato precedente perché J=K=IN=0. |
| 3 | 1 | 0 | 0 | X | 1 |  Il segnale -RESET forza O2 a 0. Il secondo flip-flop JK-PET commuta perché J=K=-O2=1. |
| 4 | 1 | 0 | 0 | 0 | 0 |  Il primo flip-flop JK-PET mantiene il suo stato precedente. Il flip-flop D-PET memorizza il valore di O1. Il secondo flip-flop JK-PET commuta. |
| 5 | 1 | 0 | 0 | 0 | 1 |  Il primo flip-flop JK-PET mantiene il suo stato precedente. Il flip-flop D-PET memorizza il valore di O1. Il secondo flip-flop JK-PET commuta. |
| 6 | 1 | 1 | 1 | 0 | 0 |  Il primo flip-flop JK-PET commuta perché J=K=IN=1. Il flip-flop D-PET memorizza il valore di O1. Il secondo flip-flop JK-PET commuta. |
| 7 | 1 | 1 | 0 | 1 | 1 |  Il primo flip-flop JK-PET c

## 1.4 Esercizio con video soluzione

In [48]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
Il video mostra la soluzione dell'esercizio. 

Ecco i valori di ogni uscita ad ogni fronte di clock:

| Fronte CK | IN  | -RES | O1  | O2  | O3  |
|------------|-----|-------|-----|-----|-----|
| 1          | 0   | 1     | X   | X   | X   |
| 2          | 0   | 1     | X   | X   | X   |
| 3          | 0   | 1     | 0   | 0   | 1   |
| 4          | 0   | 1     | 0   | 0   | 0   |
| 5          | 0   | 1     | 0   | 0   | 1   |
| 6          | 1   | 1     | 1   | 0   | 0   |
| 7          | 1   | 1     | 0   | 1   | 1   |
| 8          | 1   | 1     | 1   | 0   | 0   |
| 9          | 1   | 1     | 0   | 1   | 1   |
| 10         | 1   | 1     | 1   | 0   | 0   |
| 11         | 1   | 1     | 0   | 1   | 1   |
| 12         | 0   | 1     | 0   | 1   | 0   |
| 13         | 0   | 1     | 0   | 0   | 1   |
| 14         | 0   | 1     | 0   | 0   | 0   |
| 15         | 0   | 1     | 0   | 0   | 1   |
| 16         | 0   | 1     | 0   | 0   | 0   |
| 17         | 0   | 1     | 

## 1.5 Esercizio con libro di testo nel prompt

In [42]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Analisi temporale della rete sequenziale

**Tabella riassuntiva:**

| Fronte | -RESET | IN | O1 | O2 | O3 | Note |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | Reset attivo, tutti i flip-flop a 0 |
| 2 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 3 | 1 | 0 | 0 | 0 | 0 | Il breve impulso di Reset non ha effetto |
| 4 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 5 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 6 | 1 | 1 | 1 | 0 | 0 | IN = 1, O1 passa a 1 |
| 7 | 1 | 1 | 1 | 1 | 0 | O2 passa a 1 |
| 8 | 1 | 1 | 1 | 1 | 1 | O3 passa a 1 |
| 9 | 1 | 1 | 0 | 1 | 1 | O1 passa a 0 |
| 10 | 1 | 1 | 0 | 0 | 1 | O2 passa a 0 |
| 11 | 1 | 1 | 0 | 0 | 0 | O3 passa a 0 |
| 12 | 1 | 0 | 0 | 0 | 0 | IN = 0, nessuna variazione |
| 13 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| ... | ... | ... | ... | ... | ... | Nessuna variazione |
| 21 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |

**Spiegazione dettagliata:**

* **Fronti 1-2:** Il segnale -RESET è attivo, forzando 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [43]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Analisi temporale della rete sequenziale

Di seguito è riportata l'analisi temporale della rete sequenziale, con i valori di uscita (O1, O2, O3) a ogni fronte di clock (da 1 a 21):

| Fronte di Clock | -RESET | IN | O1 | O2 | O3 | Note |
|---|---|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 | 0 | Reset attivo, tutte le uscite a 0 |
| 2 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 3 | 1 | 0 | 0 | 0 | 0 | Il breve impulso di reset non ha effetto |
| 4 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 5 | 1 | 0 | 0 | 0 | 0 | Nessuna variazione |
| 6 | 1 | 1 | 1 | 0 | 0 | IN passa a 1, O1 si attiva |
| 7 | 1 | 1 | 0 | 1 | 0 | O1 commuta, attivando O2 |
| 8 | 1 | 1 | 1 | 0 | 1 | O1 e O3 commutano |
| 9 | 1 | 1 | 0 | 1 | 1 | O1 commuta |
| 10 | 1 | 1 | 1 | 0 | 0 | O1 e O3 commutano |
| 11 | 1 | 1 | 0 | 1 | 0 | O1 commuta |
| 12 | 1 | 0 | 0 | 1 | 1 | IN passa a 0, O3 commuta |
| 13 | 1 | 0 | 0 | 0 | 1 | O2 commuta |
| 14 | 1 | 0 | 0 | 0 | 0 | O3 commuta |
| 15 | 1 | 0 | 0 |

## 1.7 Esercizio con recupero mediante tool

In [44]:
prompt_text = f"""Una rete sequenziale è un circuito i cui valori di uscita non dipendono soltanto dai valori di ingresso correnti, ma anche da quelli precedenti.
Un diagramma temporale è un grafico che rappresenta in modo schematico come variano ingressi ed uscite di un circuito digitale in funzione del tempo.

Fronti 1-2: -RESET è uno e IN è zero.
Tra fronte 2 e fronte 3: -RESET passa da uno a zero e subito dopo da zero a uno e IN è zero. 
Fronti 3-5: -RESET è uno e IN è zero.
Tra fronte 5 e fronte 6: -RESET è uno e IN passa da zero a uno.
Fronti 6-11: -RESET è uno e IN è uno.
Tra fronte 11 e fronte 12: -RESET è uno e IN passa da uno a zero.
Fronti 12-21: -RESET è uno e IN è zero.

Dati i valori sopra per gli ingressi -RESET ed IN, esegui l’analisi temporale della rete sequenziale descritta:
{image_description}

Determina per ogni uscita il valore a ogni fronte di clock (da 1 a 21)."""

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

Analizzando il circuito dato, con gli ingressi descritti, si ottiene il seguente andamento per le uscite:
* **Fronte 1:** O1=0, O2=0, O3=0 
* **Fronte 2:** O1=0, O2=0, O3=0
* **Fronte 3:** O1=0, O2=0, O3=0
* **Fronte 4:** O1=0, O2=0, O3=0
* **Fronte 5:** O1=0, O2=0, O3=0
* **Fronte 6:** O1=1, O2=0, O3=1
* **Fronte 7:** O1=1, O2=1, O3=0
* **Fronte 8:** O1=1, O2=1, O3=0
* **Fronte 9:** O1=1, O2=1, O3=0
* **Fronte 10:** O1=1, O2=1, O3=0
* **Fronte 11:** O1=1, O2=1, O3=0
* **Fronte 12:** O1=0, O2=1, O3=1
* **Fronte 13:** O1=0, O2=0, O3=1
* **Fronte 14:** O1=0, O2=0, O3=1
* **Fronte 15:** O1=0, O2=0, O3=1
* **Fronte 16:** O1=0, O2=0, O3=1
* **Fronte 17:** O1=0, O2=0, O3=1
* **Fronte 18:** O1=0, O2=0, O3=1
* **Fronte 19:** O1=0, O2=0, O3=1
* **Fronte 20:** O1=0, O2=0, O3=1
* **Fronte 21:** O1=0, O2=0, O3=1 


ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "esecuzione temporale della rete sequenziale"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "esecuzione temporale della rete sequenziale"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [45]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
Fronte 1: O1 è indeterminato, O2 è indeterminato, O3 è indeterminato.
Tra fronte 1 e fronte 2: L’ingresso -RESET passa da uno a zero. Quando -RESET commuta, O1, O2 e O3 passano da indeterminato a zero. Dopodiché l’ingresso -RESET passa da zero a uno. 
Fronte 2: O1 è zero, O2 è zero, O3 è zero.
Fronte 3: O1 è zero, O2 è zero, O3 è zero.
Fronte 4: O1 è zero, O2 è zero, O3 è zero.
Fronte 5: O1 è zero, O2 è zero, O3 è zero.
Tra fronte 5 e fronte 6: L’ingresso IN passa da zero a uno.
Fronte 6: O1 passa da zero a uno poco dopo il fronte, O2 è zero, O3 è uno.
Fronte 7: O1 passa da uno a zero poco dopo il fronte, O2 passa da zero a uno poco dopo il fronte, O3 passa da uno a zero poco dopo il fronte.
Fronte 8: O1 passa da zero a uno poco dopo il fronte, O2 passa da uno a zero poco dopo il fronte, O3 passa da zero a uno poco dopo il fronte.
Fronte 9: O1 passa da uno a zero poco dopo il fronte, O2 passa da zero a uno poco dopo il fronte, O3 passa da uno a zero poco dopo i

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [47]:
input = f'{{"Rete sequenziale": "{image}", "Richiesta": "{prompt_text}", "Descrizione della rete": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle reti sequenziali:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
Fronte 1: O1 è indeterminato, O2 è indeterminato, O3 è indeterminato.
Tra fronte 1 e fronte 2: -RESET passa da uno a zero. Quando -RESET commuta, O1, O2 e O3 passano da indeterminato a zero. Dopodiché -RESET passa da zero a uno.
Fronte 2: O1 è zero, O2 è zero, O3 è zero.
Fronte 3: O1 è zero, O2 è zero, O3 è zero.
Fronte 4: O1 è zero, O2 è zero, O3 è zero.
Fronte 5: O1 è zero, O2 è zero, O3 è zero.
Tra fronte 5 e fronte 6: IN passa da zero a uno.
Fronte 6: O1 passa da zero a uno poco dopo il fronte, O2 è zero, O3 è zero.
Fronte 7: O1 passa da uno a zero poco dopo il fronte, O2 passa da zero a uno poco dopo il fronte, O3 è uno.
Fronte 8: O1 passa da zero a uno poco dopo il fronte, O2 passa da uno a zero poco dopo il fronte, O3 passa da uno a zero poco dopo il fronte.
Fronte 9: O1 passa da uno a zero poco dopo il fronte, O2 passa da zero a uno poco dopo il fronte, O3 passa da zero a uno poco dopo il fronte.
Fronte 10: O1 passa da zero a uno poco dopo il fronte, O2

# Esempi e contesto per la categoria 4: Macchina a stati finiti

In [ ]:
# Immagini degli esempi per il few-shot prompting
example_image1 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image1-category4>.png")
example_image2 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image2-category4>.png")
example_image3 = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image3-category4>.png")

# Descrizione delle immagini degli esempi per il few-shot prompting
example_image1_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha sei ingressi (GO, D0, D1, D2, CK, -Reset) e quattro uscite (DFF, EFF, SER, TC).
La FSM riveste il ruolo di controllore e va progettata.

* **La FSM ha:
    * variabili di stato X, Y, Z e W;
    * uscite DFF ed EFF;
    * ingressi GO, D0, D1, D2 e TC.
    
Il datapath è costituito da:

* ** Flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincorni.
    * L'ingresso -Clear del flip-flop è collegato al segnale -Reset.
    * L'ingresso -Preset del flip-flop è costante a uno. 
    * L'ingresso D è collegato al segnale DFF che esce dalla FSM.
    * L'ingresso E è collegato al segnale EFF che esce dalla FSM.
    * L'uscita (Q) costituisce l'uscita del sistema ed è chiamata SER. 
    * L'uscita negata (-Q) non è collegata a niente. 
    
* ** Contatore universale a 4 bit **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione al conteggio (En) è costante a uno.
    * Il segnale di abilitazione di TC (Et) è costante a uno. 
    * Il segnale U/-D che indica la direzione del conteggio è costante a zero.
    * Riceve in ingresso -Reset come segnale di -Clear. 
    * Il segnale di caricamento (LD) è collegato all'uscita TC.
    * L'uscita TC entra in ingresso alla FSM.
    * Gli ingressi P3...P0 hanno un valore costante pari a 2.
    * Le uscite Q3...Q0 non sono collegate a niente.
"""
example_image2_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha tre ingressi (SER, CK, -Reset) e otto uscite (CMP, ENR, MM, NEG, Q0, Q1, CLR, RDY).
La FSM riveste il ruolo di controllore e va progettata.

* **La FSM ha:
    * variabili di stato X, Y, Z e W;
    * uscite E0, E1, NEG, MM, ENR, CLR e RDY;
    * ingressi SER, Q0, Q1 e CMP.
    
Il datapath è costituito da:

* ** Primo flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincorni.
    * L'ingresso -Clear del flip-flop è collegato al segnale -Reset.
    * L'ingresso -Preset del flip-flop è costante a uno. 
    * L'ingresso D è collegato al segnale SER.
    * L'ingresso E è collegato al segnale E0 che esce dalla FSM.
    * L'uscita (Q) costituisce una uscita del sistema ed è chiamata Q0. 
    * L'uscita negata (-Q) non è collegata a niente. 

* ** Secondo flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincorni.
    * L'ingresso -Clear del flip-flop è collegato al segnale -Reset.
    * L'ingresso -Preset del flip-flop è costante a uno. 
    * L'ingresso D è collegato al segnale SER.
    * L'ingresso E è collegato al segnale E1 che esce dalla FSM.
    * L'uscita (Q) costituisce una uscita del sistema ed è chiamata Q1. 
    * L'uscita negata (-Q) non è collegata a niente. 
    
* ** Registro PiPo a 4 bit **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione E è collegato al segnale ENR che esce dalla FSM.
    * Gli ingressi P0...P3 sono collegati alle uscite S0...S3 del sommatore.
    * Le uscite Q0...Q3 sono collegati agli ingressi B3...B0 del comparatore e agli ingressi A0...A3 del sommatore.
    * Il segnale di inizializzazione -CL è collegato al segnale negato CLR che esce dalla FSM. 

* ** Sommatore a 4 bit **
    * L'ingresso B0 è costante a uno. Gli ingressi B1...B3 sono collegati insieme e dipendono dal segnale NEG che esce dalla FSM.
    * Gli ingressi A0...A3 sono collegati alle uscite Q0...Q3 del registro PiPo. 
    * Le uscite S0...S3 sono collegate agli ingressi P0...P3 del registro PiPo.
    * Il riporto in ingresso Ci è costante a zero. 

* ** Comparatore a 4 bit **
    * Gli ingressi A3...A0 sono collegati insieme e dipendono dal segnale MM che esce dalla FSM.
    * Gli ingressi B3...B0 sono collegati alle uscite Q0...Q3 del registro PiPo. 
    * L'uscita A=B entra in ingresso alla FSM con il nome CMP.

* ** Porta logica NOT ***
   * Ha in ingresso il segnale CLR che esce dalla FSM.
   * L'uscita è collegata all'ingresso -CL del registro PiPo.
"""
example_image3_description ="""Il sistema presenta una struttura controllore - datapath. 
Il sistema ha tre ingressi (GO, CK, -Reset) e due uscite (PRS, SER).
La FSM riveste il ruolo di controllore e va progettata.

* **La FSM ha:
    * variabili di stato X, Y, Z e W;
    * uscite PRS, ENC, LDC, ENS e STR;
    * ingresso GO.
    
Il datapath è costituito da:

* **Contatore universale a 8 bit **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione al conteggio (En) è collegato al segnale ENC che esce dalla FSM.
    * Il segnale di abilitazione di TC (Et) è costante a uno. 
    * Il segnale U/-D che indica la direzione del conteggio è costante a zero.
    * Riceve in ingresso -Reset come segnale di -Clear. 
    * Il segnale di caricamento (LD) è collegato all'uscita TC.
    * L'uscita TC non è collegata a niente. 
    * Gli ingressi P7...P0 hanno un valore costante pari a 0Bh. 
    * Le uscite Q7...Q0 sono collegate agli ingressi P7...P0 del registro PiSo. 
    
* ** Registro PiSo a 8 bit **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione (E) è collegato al segnale ENS che esce dalla FSM.
    * Il segnale di caricamento (LD) è collegato al segnale LDC che esce dalla FSM. 
    * Gli ingressi P7...P0 sono collegati alle uscite Q7...Q0 del contatore.
    * L'uscita Q0 è collegata in ingresso alla porta logica EXOR.
    * Riceve in ingresso -Reset come segnale di -Clear. 

* ** Flip-flop D-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincorni.
    * L'ingresso -Clear del flip-flop è collegato al segnale -Reset.
    * L'ingresso -Preset del flip-flop è costante a uno. 
    * L'ingresso D è collegato all'uscita della porta logica EXOR.
    * L'uscita (Q) costituisce l'uscita del sistema ed è chiamata SER. 
    * L'uscita negata (-Q) non è collegata a niente. 

* ** Porta logica EXOR **
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita Q0 del registro PiSo.
        * Segnale STR che esca dalla FSM.
    * L'uscita della porta EXOR è collegata all'ingresso D del flip-flop D-PET ed è chiamata D.
"""

# Soluzioni degli esempi per il few-shot prompting
solution1 = """+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
| Stato | W | Z | Y | X | GO | D0 | D1 | D2 | TC | DFF | EFF | Stato Successivo | iW | iZ | iY | iX |
+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
|  rs   | 0 | 0 | 0 | 0 |  0 |  - |  - |  - |  - |  0  |  0  |        rs        |  0 |  0 |  0 |  0 |
|  rs   | 0 | 0 | 0 | 0 |  1 |  - |  - |  - |  - |  0  |  0  |        a         |  0 |  0 |  0 |  1 |
|  a    | 0 | 0 | 0 | 1 |  0 |  - |  - |  - |  - |  0  |  0  |        ws        |  0 |  0 |  1 |  0 |
|  a    | 0 | 0 | 0 | 1 |  1 |  - |  - |  - |  - |  0  |  0  |        a         |  0 |  0 |  0 |  1 |
|  ws   | 0 | 0 | 1 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        ws        |  0 |  0 |  1 |  0 |
|  ws   | 0 | 0 | 1 | 0 |  - |  - |  - |  - |  1 |  0  |  0  |        st        |  0 |  0 |  1 |  1 |
|  st   | 0 | 0 | 1 | 1 |  - |  - |  - |  - |  - |  1  |  1  |        w0        |  0 |  1 |  0 |  0 |
|  w0   | 0 | 1 | 0 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        w0        |  0 |  1 |  0 |  0 |
|  w0   | 0 | 1 | 0 | 0 |  - |  0 |  - |  - |  1 |  0  |  0  |        n0        |  0 |  1 |  0 |  1 |
|  w0   | 0 | 1 | 0 | 0 |  - |  1 |  - |  - |  1 |  0  |  0  |        d0        |  0 |  1 |  1 |  0 |
|  n0   | 0 | 1 | 0 | 1 |  - |  - |  - |  - |  - |  0  |  1  |        w1        |  0 |  1 |  1 |  1 |
|  d0   | 0 | 1 | 1 | 0 |  - |  - |  - |  - |  - |  1  |  1  |        w1        |  0 |  1 |  1 |  1 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        w1        |  0 |  1 |  1 |  1 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  0 |  - |  1 |  0  |  0  |        n1        |  1 |  0 |  0 |  0 |
|  w1   | 0 | 1 | 1 | 1 |  - |  - |  1 |  - |  1 |  0  |  0  |        d1        |  1 |  0 |  0 |  1 |
|  n1   | 1 | 0 | 0 | 0 |  - |  - |  - |  - |  - |  0  |  1  |        w2        |  1 |  0 |  1 |  0 |
|  d1   | 1 | 0 | 0 | 1 |  - |  - |  - |  - |  - |  1  |  1  |        w2        |  1 |  0 |  1 |  0 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  - |  0 |  0  |  0  |        w2        |  1 |  0 |  1 |  0 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  0 |  1 |  0  |  0  |        n2        |  1 |  0 |  1 |  1 |
|  w2   | 1 | 0 | 1 | 0 |  - |  - |  - |  1 |  1 |  0  |  0  |        d2        |  1 |  1 |  0 |  0 |
|  n2   | 1 | 0 | 1 | 1 |  - |  - |  - |  - |  - |  0  |  1  |        w3        |  1 |  1 |  0 |  1 |
|  d2   | 1 | 1 | 0 | 0 |  - |  - |  - |  - |  - |  1  |  1  |        w3        |  1 |  1 |  0 |  1 |
|  w3   | 1 | 1 | 0 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        w3        |  1 |  1 |  0 |  1 |
|  w3   | 1 | 1 | 0 | 1 |  - |  - |  - |  - |  1 |  0  |  0  |        n3        |  1 |  1 |  1 |  0 |
|  n3   | 1 | 1 | 1 | 0 |  - |  - |  - |  - |  - |  0  |  1  |        sp        |  1 |  1 |  1 |  1 |
|  sp   | 1 | 1 | 1 | 1 |  - |  - |  - |  - |  0 |  0  |  0  |        sp        |  1 |  1 |  1 |  1 |
|  sp   | 1 | 1 | 1 | 1 |  - |  - |  - |  - |  1 |  0  |  0  |        rs        |  0 |  0 |  0 |  0 |
+-------+---+---+---+---+----+----+----+----+----+-----+-----+-----------------+----+----+----+----+
"""
solution2 = """+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
| Stato | W | Z | Y | X | SER | Q0| Q1|CMP| E0| E1|NEG| MM|ENR|CLR|RDY| Stato Successivo | iW| iZ| iY| iX|
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  0  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  -  | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  -  | - | - | - | 0 | 1 | 0 | 0 | 0 | 0 | 0 |        s        | 1 | 0 | 1 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 0 | 0 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 1 | 0 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        e        | 0 | 1 | 0 | 0 |
|   s   | 1 | 0 | 1 | 0 |  0  | 0 | 1 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        f        | 0 | 1 | 0 | 1 |
|   s   | 1 | 0 | 1 | 0 |  0  | 1 | 1 | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        h        | 0 | 1 | 1 | 0 |
|   s   | 1 | 0 | 1 | 0 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        d        | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 1 | 1 |  0  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   d   | 0 | 0 | 1 | 1 |  1  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        d        | 0 | 0 | 1 | 1 |
|   e   | 0 | 1 | 0 | 0 |  -  | - | - | - | 0 | 0 | 0 | 0 | 0 | 1 | 0 |        p        | 1 | 0 | 0 | 1 |
|   f   | 0 | 1 | 0 | 1 |  -  | - | - | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |        g        | 0 | 1 | 1 | 1 |
|   f   | 0 | 1 | 0 | 1 |  -  | - | - | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   h   | 0 | 1 | 1 | 0 |  -  | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        m        | 1 | 0 | 0 | 0 |
|   h   | 0 | 1 | 1 | 0 |  -  | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 |
|   g   | 0 | 1 | 1 | 1 |  -  | - | - | - | 0 | 0 | 0 | 0 | 1 | 0 | 0 |        p        | 1 | 0 | 0 | 1 |
|   m   | 1 | 0 | 0 | 0 |  -  | - | - | - | 0 | 0 | 1 | 0 | 1 | 0 | 0 |        p        | 1 | 0 | 0 | 1 |
|   p   | 1 | 0 | 0 | 1 |  -  | - | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 1 |        a        | 0 | 0 | 0 | 0 |
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
"""
solution3 = """+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+
| Stato | W | Z | Y | X | GO | PRS| ENC| LDC| ENS| STR| Stato Successivo | iW| iZ| iY| iX|
+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  0 |  0 |  1 |  0 |  0 |  0 |        a        | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 |  1 |  0 |  1 |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  0 |  1 |  0 |  0 |  0 |  0 |        st       | 0 | 0 | 1 | 0 |
|   b   | 0 | 0 | 0 | 1 |  1 |  1 |  0 |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|  st   | 0 | 0 | 1 | 0 |  - |  0 |  0 |  1 |  0 |  1 |        d0       | 0 | 0 | 1 | 1 |
|  d0   | 0 | 0 | 1 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d1       | 0 | 1 | 0 | 0 |
|  d1   | 0 | 1 | 0 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d2       | 0 | 1 | 0 | 1 |
|  d2   | 0 | 1 | 0 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d3       | 0 | 1 | 1 | 0 |
|  d3   | 0 | 1 | 1 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d4       | 0 | 1 | 1 | 1 |
|  d4   | 0 | 1 | 1 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d5       | 1 | 0 | 0 | 0 |
|  d5   | 1 | 0 | 0 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        d6       | 1 | 0 | 0 | 1 |
|  d6   | 1 | 0 | 0 | 1 |  - |  0 |  0 |  0 |  1 |  0 |        d7       | 1 | 0 | 1 | 0 |
|  d7   | 1 | 0 | 1 | 0 |  - |  0 |  0 |  0 |  1 |  0 |        a        | 0 | 0 | 0 | 0 |
+-------+---+---+---+---+----+----+----+----+----+----+-----------------+---+---+---+---+
"""

# Richietse degli esempi per il few-shot prompting
prompt_text1 = f"""Si consideri il sistema digitale rappresentato nella figura {example_image1}, composto dal controllore (FSM), dal contatore “Cnt4” e da un flip-flop E-PET.

Compito del sistema (un trasmettitore seriale) è di generare, sull’uscita SER, tramite il flip-flop E-PET, un pacchetto di 5 bit, composto da un bit di start a 1, tre bit di dato (nell’ordine: D0, D1 e D2), e un bit di stop a 0.
La durata N di ciascun bit del pacchetto (il tempo di bit) è un multiplo del periodo del clock CK, ed è funzione del numero P da impostare agli ingressi P3..P 0 del contatore.
I dati da trasmettere, D0, D1 e D2, sono disponibili agli ingressi del controllore.
Un fronte di discesa sull’ingresso GO fa partire la generazione del pacchetto.

Progetta la FSM come macchina di Moore sincrona e realizza la tabella degli stati della FSM. 
"""
prompt_text2 = f"""Si consideri il sistema digitale rappresentato nella figura {example_image2}, composto dal controllore (FSM) che riceve dati da una linea seriale e gestisce una rete composta da due flip-flop E-PET, un registro e alcuni circuiti aritmetici.

Il controllore riceve, su SER, un pacchetto di 4 bit, composto da un bit di start a 1, due bit di dato (nell’ordine D0 e D1), e un bit di stop a 0. 
La durata del tempo di bit è pari al periodo di CK; i due bit di dato codificano un’operazione che il sistema esegue sulle uscite N3..N0.
Se è ricevuto un bit di stop errato (= 1), non si eseguono operazioni e si aspetta che SER torni a zero, prima di attendere il successivo pacchetto. 

Alla ricezione di un pacchetto valido (bit di stop = 0), a seconda del valore ricevuto di D1 e D0 il sistema esegue le seguenti operazioni:
| D1 | D0 | Operazione                    |
|----|----|-------------------------------|
|  0 |  0 | Nessuna (NOP)                 |
|  0 |  1 | Azzera l’uscita N3..N0        |
|  1 |  0 | Incrementa di uno l’uscita N3..N0 |
|  1 |  1 | Decrementa di uno l’uscita N3..N0 |

L’uscita N3..N0 non è incrementata quando ha raggiunto il valore massimo, né decrementata quando ha raggiunto il minimo. 
L’uscita RDY è attivata per un ciclo di clock alla ricezione dei comandi, tranne la NOP.
Si noti che la MSF memorizza i bit ricevuti D1 e D0 rispettivamente nei flipflop Q1 e Q0, in modo da disporre dei valori ricevuti anche dopo il controllo del bit di stop; l’uscita NEG della MSF permette di impostare all’ingresso del sommatore Add4 due diversi valori, mentre l’uscita MM consente al comparatore Cp4 di confrontare il numero N3..N0 con due diversi valori.

Progetta la FSM come macchina di Moore sincrona e realizza la tabella degli stati della FSM. 
"""
prompt_text3 = f"""Si consideri il sistema digitale rappresentato nella figura {example_image3}, composto dal controllore (FSM), da un contatore a 8 bit “Cnt8”, un registro a scorrimento “PiSo8”, un flip-flop D-PET e alcune porte logiche.

Al rilascio del pulsante GO, il sistema trasmette su SER un segnale seriale, composto da un bit di start, 8 bit di dato e un bit di stop. Gli 8 bit di dato da trasmettere sono prelevati dalle uscite del contatore. 
A riposo, quando il pulsante non è premuto (GO = 0), il contatore è abilitato, tramite la linea ENC; per tutto il resto del tempo, il conteggio è disabilitato. Alla pressione del pulsante (GO = 1), la MSF attiva la linea PRS.

Al rilascio del pulsante, il controllore inizia le operazioni per la trasmissione, caricando le uscite del contatore nel registro a scorrimento, tramite la linea LDC. 
Si noti che la linea STR permette di generare il bit di start per un ciclo di clock, e che l’uscita seriale SER è generata tramite il flip-flop.

La serializzazione è comandata dal controllore tramite la linea ENS che abilita il registro a scorrimento; terminata la generazione di SER, il sistema torna nuovamente a controllare il pulsante GO. 

Progetta la FSM come macchina di Moore sincrona e realizza la tabella degli stati della FSM. 
"""

# Costruzione dei prompt degli esempi per il few-shot prompting
input1 = f'{{"Sistema": "{example_image1}", "Richiesta": "{prompt_text1}", "Descrizione del sistema": "{example_image1_description}"}}'
input2 = f'{{"Sistema": "{example_image2}", "Richiesta": "{prompt_text2}", "Descrizione del sistema": "{example_image2_description}"}}'
input3 = f'{{"Sistema": "{example_image3}", "Richiesta": "{prompt_text3}", "Descrizione del sistema": "{example_image3_description}"}}'

# Recupero esplicito delle informazioni sull'argomento (direct retrieval)
retrieval_text = """Che cos'è una macchina a stati finiti?
Che cos'è una strttura controllore - datapath?
Come si progetta un sistema usando una strttura controllore - datapath?
Cosa sono i flip-flop PET e quali sono?
Cosa si intende per contantore, registro SiPo, registro Pipo, registro Piso, sommatore e comparatore?"""

text_retrieved = retrieveInformationFromCustomDataSource(retrieval_text)

# Libro di testo sull'argomento
textbook = Part.from_uri(mime_type="application/pdf", uri="gs://<bucket-name-it>/<textbook-file>.pdf")

--------Text Retrieved--------


2.7 . 84
11.5 Esercizio 6.2.8 . 84
11.6 Esercizio 7.2 . 86
12 Esercizi di riepilogo . 87
12.1 Esercizio (Sintesi PS-SP a costo minimo) . 87
12.2 Esercizio (sintesi a porte NOR) . 88Appunti sulle reti combinatorie (nuovo programma) – Giovanni Stea 
4
Version history 
20180912: prima versione Appunti sulle reti combinatorie (nuovo programma) – Giovanni Stea 
5
1 Le reti logiche come modello astratto di sistemi fisici 
Supponiamo di avere il seguente sistema fisico (cioè, realizzato in pratica): 
Le due scatolette comunicano tramite un filo elettrico. Nella scatoletta di sinistra ho la possibilità di 
impostare una tensione tra 0V e 5Vs, variandola nel tempo, e nella scatoletta di destra ho la pos￾sibilità di osservare una lampadina. In particolare, supponiamo che un osservatore sia in grado di 
decidere che: 
- se la tensione impostata sul filo è tra 0V e 0.8V, allora la lampada è spenta;
- se la tensione è tra 2V e 5V, allora la lampada è accesa; 
- se l

# Esercizio 16

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image16>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video16>.mp4")

image_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha due ingressi (LN, CK) e due uscite (LNA, LNB).
La FSM riveste il ruolo di controllore e va progettata.
* **La FSM ha:
    * variabili di stato X, Y, Z, W e K;
    * uscite LNA, LNB, E0, E1 e E2;
    * ingressi LN, P, Q0, Q1 e Q2.
Il datapath è costituito da:
* ** Primo flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * L'ingresso D è collegato al segnale di ingresso LN.
    * L'uscita (Q) entra in ingresso alla porta logica EXOR e alla FSM. L'uscita Q è chiamata Q0. 
    * L'ingresso E è collegato all'uscita E0 della FSM.
* ** Secondp flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * L'ingresso D è collegato al segnale di ingresso LN.
    * L'uscita (Q) entra in ingresso alla porta logica EXOR e alla FSM. L'uscita Q è chiamata Q1. 
    * L'ingresso E è collegato all'uscita E1 della FSM.
* ** Terzo flip-flop E-PET **
    * Riceve il segnale di clock (CK) comune a tutti i flip-flop.
    * L'ingresso D è collegato al segnale di ingresso LN.
    * L'uscita (Q) entra in ingresso alla FSM. L'uscita Q è chiamata Q2. 
    * L'ingresso E è collegato all'uscita E2 della FSM.
* ** Porta logica EXOR **
    * I suoi due ingressi sono collegati rispettivamente a:
        * Uscita (Q) del primo flip-flop E-PET chiamata Q0.
        * Uscita (Q) del secondo flip-flop E-PET chiamata Q1.
    * L'uscita della prima porta EXOR entra in ingresso alla FSM ed è chiamata P.
"""

prompt_text = f"""Si vuole progettare un sistema digitale sincrono che riceve e ritrasmette dati seriali. 

L'ingresso LN riceve dall'esterno una successione di bit, sincroni con il clock, raggruppati in pacchetti di cinque.
Ogni pacchetto inizia con il primo bit sempre al valore '1' (start bit), e termina con il quinto bit sempre a '0' (stop bit). 
Questi due bit non trasportano informazione, ma servono per identificare la posizione degli altri 3 bit (B0, B1, B2), che contengono i dati trasportati dalla sequenza. 
Ogni bit ha una durata fissa, chiamata tempo di bit. 
Nel nostro caso, il tempo di bit è pari al periodo del clock, e i fronti di salita di questo corrispondono al centro di tale intervallo. 
L'ingresso LN, a riposo, ha il valore '0'. Il dispositivo attende l'arrivo del bit di start, per poi ricevere ed elaborare tutti gli altri bit del pacchetto.

La figura {image} mostra l'architettura del sistema, dove la macchina a stati finiti FSM controlla tre flip-flop di tipo E-PET, che hanno il compito di immagazzinare temporaneamente i bit di informazione ricevuti dalla linea LN (B0 è memorizzato su Q0, B1 su Q1 e B2 su Q2). 
La macchina a stati pilota le due linee LNA e LNB, su cui saranno generate le sequenze seriali di uscita.
Il sistema controlla che il pacchetto ricevuto sia valido, cioè che termini correttamente con il bit di stop a '0'. Se questa condizione non è verificata, il sistema ignora i dati ricevuti, non attiva le linee di uscita, attende che la linea LN sia tornata a riposo e, infine, si mette in attesa di un nuovo pacchetto.

Se il pacchetto è valido, la macchina a stati utilizza il valore di Q2 per selezionare l'uscita si cui trasmettere (LNA se Q2 = '0', oppure LNB se Q2 = '1'). 
La sequenza trasmessa è: bit di start - Q0 - Q1 - P - bit di stop. 
Dopodichè è necessario attendere un ciclo di clock prima di poter ricevere un nuovo pacchetto. 
Il bit P rappresenta la parità calcolata su Q0 e Q1 dalla porta EXOR presente nel sistema.
Le variabili di stato sono X, Y, Z, W e K, le uscite sono LNA, LNB, E0, E1 e E2, e gli ingressi sono LN, P, Q0, Q1 e Q2.

Di seguito è presente la descrizione del sistema:
{image_description}

Progetta la FSM come macchina di Moore sincrona e realizza la tabella degli stati della FSM. 
"""

prompt_text_FW = f"""Si vuole progettare un sistema digitale sincrono che riceve e ritrasmette dati seriali. 

L'ingresso LN riceve dall'esterno una successione di bit, sincroni con il clock, raggruppati in pacchetti di cinque.
Ogni pacchetto inizia con il primo bit sempre al valore '1' (start bit), e termina con il quinto bit sempre a '0' (stop bit). 
Questi due bit non trasportano informazione, ma servono per identificare la posizione degli altri 3 bit (B0, B1, B2), che contengono i dati trasportati dalla sequenza. 
Ogni bit ha una durata fissa, chiamata tempo di bit. 
Nel nostro caso, il tempo di bit è pari al periodo del clock, e i fronti di salita di questo corrispondono al centro di tale intervallo. 
L'ingresso LN, a riposo, ha il valore '0'. Il dispositivo attende l'arrivo del bit di start, per poi ricevere ed elaborare tutti gli altri bit del pacchetto.

La figura {image} mostra l'architettura del sistema, dove la macchina a stati finiti FSM controlla tre flip-flop di tipo E-PET, che hanno il compito di immagazzinare temporaneamente i bit di informazione ricevuti dalla linea LN (B0 è memorizzato su Q0, B1 su Q1 e B2 su Q2). 
La macchina a stati pilota le due linee LNA e LNB, su cui saranno generate le sequenze seriali di uscita.
Il sistema controlla che il pacchetto ricevuto sia valido, cioè che termini correttamente con il bit di stop a '0'. Se questa condizione non è verificata, il sistema ignora i dati ricevuti, non attiva le linee di uscita, attende che la linea LN sia tornata a riposo e, infine, si mette in attesa di un nuovo pacchetto.

Se il pacchetto è valido, la macchina a stati utilizza il valore di Q2 per selezionare l'uscita si cui trasmettere (LNA se Q2 = '0', oppure LNB se Q2 = '1'). 
La sequenza trasmessa è: bit di start - Q0 - Q1 - P - bit di stop. 
Dopodichè è necessario attendere un ciclo di clock prima di poter ricevere un nuovo pacchetto. 
Il bit P rappresenta la parità calcolata su Q0 e Q1 dalla porta EXOR presente nel sistema.
Le variabili di stato sono X, Y, Z, W e K, le uscite sono LNA, LNB, E0, E1 e E2, e gli ingressi sono LN, P, Q0, Q1 e Q2.

Progetta la FSM come macchina di Moore sincrona e realizza la tabella degli stati della FSM. 
"""

## 1.1 Esercizio senza ulteriore contesto

In [10]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione FSM per Sistema di Ritrasmissione Dati Seriale

### Descrizione degli Stati

Il sistema può essere descritto attraverso i seguenti stati:

* **S0 (Attesa Start Bit):** Stato iniziale. Il sistema attende l'arrivo di un bit di start ('1') sulla linea LN.
* **S1 (Ricezione Bit 0):**  Il sistema ha ricevuto lo start bit e si prepara a memorizzare il primo bit di informazione (B0) nel flip-flop Q0.
* **S2 (Ricezione Bit 1):** Il sistema memorizza il secondo bit di informazione (B1) nel flip-flop Q1.
* **S3 (Ricezione Bit 2):** Il sistema memorizza il terzo bit di informazione (B2) nel flip-flop Q2.
* **S4 (Verifica Stop Bit):** Il sistema verifica che il bit ricevuto sia uno stop bit ('0'). 
    * Se il bit è '0', il pacchetto è valido e si passa allo stato S5. 
    * Se il bit è '1', il pacchetto non è valido e si torna allo stato S0.
* **S5 (Trasmissione Dati):** Il sistema trasmette la sequenza di output (start bit - Q0 - Q1 - P - stop bit) sulla

## 1.2 Esercizio con recupero diretto

In [11]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione FSM per sistema di ritrasmissione dati seriali

Ecco la progettazione della FSM come macchina di Moore sincrona e la sua tabella degli stati:

**Stati della FSM:**

* **S0: Attesa Inizio Pacchetto:** Attesa del bit di start ('1') sulla linea LN.
* **S1: Ricezione Bit 0:** Ricezione e memorizzazione del bit B0.
* **S2: Ricezione Bit 1:** Ricezione e memorizzazione del bit B1.
* **S3: Ricezione Bit 2:** Ricezione e memorizzazione del bit B2.
* **S4: Verifica Stop Bit:** Verifica che il bit di stop sia '0'.
* **S5: Trasmissione su LNA:** Trasmissione del pacchetto su LNA (se Q2 = '0').
* **S6: Trasmissione su LNB:** Trasmissione del pacchetto su LNB (se Q2 = '1').
* **S7: Attesa Ciclo:** Attesa di un ciclo di clock prima di tornare in S0.

**Tabella degli stati:**

| Stato Presente | LN | Stato Futuro | LNA | LNB | E0 | E1 | E2 |
|---|---|---|---|---|---|---|---|
| **S0** | 0 | S0 | 0 | 0 | 0 | 0 | 0 |
| **S0** | 1 | S1 | 0 | 0 | 1 | 0 | 0 |
| **S

## 1.3 Esercizio con recupero diretto e video soluzione

In [13]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
## Progettazione FSM per il sistema di ritrasmissione dati seriali

La FSM progettata è una macchina di Moore sincrona, il cui stato di uscita dipende unicamente dallo stato presente e non dagli ingressi. Di seguito è riportata la tabella degli stati completa, derivata dal diagramma ASM illustrato nel video:

| Stato | K | W | Z | Y | X | LN | P | Q0 | Q1 | Q2 | LNA | LNB | E0 | E1 | E2 | Stato successivo | K | W | Z | Y | X |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | a | 0 | 0 | 0 | 0 | 0 |
| a | 0 | 0 | 0 | 0 | 0 | 1 | - | - | - | - | 0 | 0 | 1 | 0 | 0 | b | 0 | 0 | 0 | 0 | 1 |
| b | 0 | 0 | 0 | 0 | 1 | - | - | - | - | - | 0 | 0 | 0 | 1 | 0 | c | 0 | 0 | 0 | 1 | 0 |
| c | 0 | 0 | 0 | 1 | 0 | - | - | - | - | - | 0 | 0 | 0 | 0 | 1 | d | 0 | 0 | 0 | 1 | 1 |
| d | 0 | 0 | 0 | 1 | 1 | - | - | - | - | - | 0 | 0 | 0 | 0 | 0 | f | 0 | 0 | 1 | 0 | 0 |
| f 

## 1.4 Esercizio con video soluzione

In [14]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
La soluzione all'esercizio è presente nel video allegato. 

Di seguito è riportata la tabella degli stati della FSM:

| Stato | K | W | Z | Y | X | LN | P | Q0 | Q1 | Q2 | LNA | LNB | E0 | E1 | E2 | Stato successivo | K | W | Z | Y | X |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | a | 0 | 0 | 0 | 0 | 0 |
| a | 0 | 0 | 0 | 0 | 0 | 1 | - | - | - | - | 0 | 0 | 1 | 0 | 0 | b | 0 | 0 | 0 | 0 | 1 |
| b | 0 | 0 | 0 | 0 | 1 | - | - | - | - | - | 0 | 0 | 0 | 1 | 0 | c | 0 | 0 | 0 | 1 | 0 |
| c | 0 | 0 | 0 | 1 | 0 | - | - | - | - | - | 0 | 0 | 0 | 0 | 1 | d | 0 | 0 | 0 | 1 | 1 |
| d | 0 | 0 | 0 | 1 | 1 | - | - | - | - | - | 0 | 0 | 0 | 0 | 0 | f | 0 | 0 | 1 | 0 | 0 |
| f | 0 | 0 | 1 | 0 | 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e | 0 | 0 | 1 | 0 | 1 |
| f | 0 | 0 | 1 | 0 | 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e | 0 | 0 | 1 | 0 | 1 |
| e | 0 | 0 | 1 |

## 1.5 Esercizio con libro di testo nel prompt

In [15]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Progettazione della FSM per il sistema di ritrasmissione dati seriali

La FSM sarà progettata come una macchina di Moore sincrona, il che significa che le uscite dipendono solo dallo stato corrente e non dagli ingressi.

**Stati della FSM:**

* **S0 (Attesa Start Bit):** Stato iniziale. La FSM attende l'arrivo del bit di start ('1') sulla linea LN.
* **S1 (Ricezione B0):** La FSM riceve il bit B0 e lo memorizza nel flip-flop Q0.
* **S2 (Ricezione B1):** La FSM riceve il bit B1 e lo memorizza nel flip-flop Q1.
* **S3 (Ricezione B2):** La FSM riceve il bit B2 e lo memorizza nel flip-flop Q2.
* **S4 (Verifica Stop Bit):** La FSM verifica che il bit di stop sia '0'. Se il bit è '0', il pacchetto è valido e la FSM procede allo stato S5. Se il bit non è '0', il pacchetto è invalido e la FSM torna allo stato S0.
* **S5 (Selezione Uscita):** La FSM seleziona l'uscita su cui trasmettere in base al valore di Q2 (LNA se Q2 = '0', LNB se Q2 = '1').
* **S6 (Trasmissione 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [16]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Tabella degli stati della FSM

| Stato | K W Z Y X | LN | P | Q2 | Q1 | Q0 | LNA | LNB | E0 | E1 | E2 | Stato successivo |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 0 0 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | a |
| a | 0 0 0 0 0 | 1 | - | - | - | - | 0 | 0 | 1 | 0 | 0 | b |
| b | 0 0 0 0 1 | 0 | - | - | - | - | 0 | 0 | 0 | 1 | 0 | c |
| b | 0 0 0 0 1 | 1 | - | - | - | - | 0 | 0 | 0 | 1 | 0 | c |
| c | 0 0 0 1 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 1 | d |
| c | 0 0 0 1 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 1 | d |
| d | 0 0 0 1 1 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | f |
| d | 0 0 0 1 1 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e |
| e | 0 0 1 0 0 | 0 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e |
| e | 0 0 1 0 0 | 1 | - | - | - | - | 0 | 0 | 0 | 0 | 0 | e |
| f | 0 0 1 0 1 | 0 | P | 0 | Q1 | Q0 | 0 | 0 | 0 | 0 | 0 | g |
| f | 0 0 1 0 1 | 1 | P | 0 | Q1 | Q0 | 0 | 0 | 0 | 0 | 0 | g |
| g | 0 0 1 1 0 | - | - | - | - | - | 1 | 0 | 0

## 1.7 Esercizio con recupero mediante tool

In [19]:
prompt_text = f"""Si vuole progettare un sistema digitale sincrono che riceve e ritrasmette dati seriali.

L'ingresso LN riceve dall'esterno una successione di bit, sincroni con il clock, raggruppati in pacchetti di cinque.
Ogni pacchetto inizia con il primo bit sempre al valore '1' (start bit), e termina con il quinto bit sempre a '0' (stop bit). 
Questi due bit non trasportano informazione, ma servono per identificare la posizione degli altri 3 bit (B0, B1, B2), che contengono i dati trasportati dalla sequenza. 
Ogni bit ha una durata fissa, chiamata tempo di bit. 
Nel nostro caso, il tempo di bit è pari al periodo del clock, e i fronti di salita di questo corrispondono al centro di tale intervallo. 
L'ingresso LN, a riposo, ha il valore '0'. Il dispositivo attende l'arrivo del bit di start, per poi ricevere ed elaborare tutti gli altri bit del pacchetto.

Di seguito è presente la descrizione del sistema: 
{image_description} 

Nell'architettura del sistema descritta sopra la macchina a stati finiti FSM controlla tre flip-flop di tipo E-PET, che hanno il compito di immagazzinare temporaneamente i bit di informazione ricevuti dalla linea LN (B0 è memorizzato su Q0, B1 su Q1 e B2 su Q2). 
La macchina a stati pilota le due linee LNA e LNB, su cui saranno generate le sequenze seriali di uscita.
Il sistema controlla che il pacchetto ricevuto sia valido, cioè che termini correttamente con il bit di stop a '0'. Se questa condizione non è verificata, il sistema ignora i dati ricevuti, non attiva le linee di uscita, attende che la linea LN sia tornata a riposo e, infine, si mette in attesa di un nuovo pacchetto.
Se il pacchetto è valido, la macchina a stati utilizza il valore di Q2 per selezionare l'uscita si cui trasmettere (LNA se Q2 = '0', oppure LNB se Q2 = '1'). 
La sequenza trasmessa è: bit di start - Q0 - Q1 - P - bit di stop. 
Dopodichè è necessario attendere un ciclo di clock prima di poter ricevere un nuovo pacchetto. 
Il bit P rappresenta la parità calcolata su Q0 e Q1 dalla porta EXOR presente nel sistema.

Le variabili di stato sono X, Y, Z, W e K, le uscite sono LNA, LNB, E0, E1 e E2, e gli ingressi sono LN, P, Q0, Q1 e Q2.

Realizza la tabella degli stati della FSM. """

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

| Stato Corrente | LN | P | Q0 | Q1 | Q2 | Stato Futuro | LNA | LNB | E0 | E1 | E2 |
|---|---|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 | - | - | - | - | S0 | 0 | 0 | 0 | 0 | 0 |
| S0 | 1 | - | - | - | - | S1 | 0 | 0 | 1 | 0 | 0 |
| S1 | - | - | - | - | - | S2 | 0 | 0 | 0 | 1 | 0 |
| S2 | - | - | - | - | - | S3 | 0 | 0 | 0 | 0 | 1 |
| S3 | - | - | - | - | - | S4 | 0 | 0 | 0 | 0 | 0 |
| S4 | - | - | Q0 | Q1 | 0 | S5 | 1 | 0 | 0 | 0 | 0 |
| S4 | - | - | Q0 | Q1 | 1 | S5 | 0 | 1 | 0 | 0 | 0 |
| S5 | - | - | - | - | - | S6 | Q0 | Q0 | 0 | 0 | 0 |
| S6 | - | - | - | - | - | S7 | Q1 | Q1 | 0 | 0 | 0 |
| S7 | - | - | - | - | - | S8 | P | P | 0 | 0 | 0 |
| S8 | - | - | - | - | - | S0 | 0 | 0 | 0 | 0 | 0 |
| S4 | - | - | - | - | - | S0 | 0 | 0 | 0 | 0 | 0 | 

Note:
* Il simbolo "-" indica un valore indifferente.
* Lo stato S0 è lo stato iniziale, in cui la FSM attende l'arrivo di un nuovo pacchetto.
* Lo stato S4 verifica se il pacchetto ricevuto è valido. Se il bit di stop è a '0', pas

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "fsm seriale bit"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "fsm seriale bit"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [17]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
+-------+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+-----+-----------------+---+---+---+---+---+
| Stato | K | W | Z | Y | X |LN| P | Q0| Q1| Q2| LNA| LNB| E0 | E1 | E2 | Stato Successivo | iK| iW| iZ| iY| iX|
+-------+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+-----+-----------------+---+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 | 0 | 0 | - | - | - | - |  0  |  0  |  0  |  0  |  0  |        a        | 0 | 0 | 0 | 0 | 0 | 
|   a   | 0 | 0 | 0 | 0 | 0 | 1 | - | - | - | - |  0  |  0  |  0  |  0  |  0  |        b        | 0 | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 0 | 1 | - | - | - | - | - |  0  |  0  |  1  |  0  |  0  |        c        | 0 | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 0 | 1 | 0 | - | - | - | - | - |  0  |  0  |  0  |  1  |  0  |        d        | 0 | 0 | 1 | 0 | 0 |
|   d   | 0 | 0 | 1 | 0 | 0 | - | - | - | - | - |  0  |  0  |  0  |  0  |  1  |        e        | 0 | 1 | 0 | 0 | 0 |
|   e   | 0 | 1 | 0 | 0 | 0 | - | - | - | -

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [18]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
| Stato | K | W | Z | Y | X |  LN | P | Q0| Q1| Q2|LNA|LNB| E0| E1| E2| Stato Successivo | iK| iW| iZ| iY| iX|
+-------+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 | 0 |  0  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        a        | 0 | 0 | 0 | 0 | 0 |
|   a   | 0 | 0 | 0 | 0 | 0 |  1  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        b        | 0 | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 0 | 1 |  -  | - | - | - | - | 0 | 0 | 1 | 0 | 0 |        c        | 0 | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 0 | 1 | 0 |  -  | - | - | - | - | 0 | 0 | 0 | 1 | 0 |        d        | 0 | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 0 | 1 | 1 |  -  | - | - | - | - | 0 | 0 | 0 | 0 | 1 |        e        | 0 | 0 | 1 | 0 | 0 |
|   e   | 0 | 0 | 1 | 0 | 0 |  -  | - | - | - | - | 0 | 0 | 0 | 0 | 0 |        f        | 0 | 1 | 

# Esercizio 17

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image17>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video17>.mp4")

image_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha tre ingressi (IN, CK, -Reset) e due uscite (SER, BSY).
Le due FSM (FSM-A, FSM-B) rivestono il ruolo di controllore e vanno progettate.

* **La FSM-A ha:
    * variabili di stato X, Y e Z;
    * uscite TRG, CL e BSY;
    * ingressi EN ed ETX.
    * segnale di clock CK.
    * segnale di inizializzazione -Reset.
    
* **La FSM-B ha:
    * variabili di stato X, Y, Z e W;
    * uscite SER ed ETX;
    * ingressi TRG, Q0, Q1 e Q2.
    * segnale di clock CK.
    * segnale di inizializzazione -Reset. 
    
Il datapath è costituito da:

* ** Flip-flop D-NET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * L'ingresso D è collegato al segnale di ingresso IN.
    * L'uscita (Q) entra in ingresso alla FSM-A e al contatore CNT col il nome EN. 
    
* ** Contatore a tre bit CNT **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Riceve in ingresso EN come segnale di abilitazione.
    * Riceve in ingresso CL come segnale di -Clear. 
    * Le uscite Q0, Q1 e Q2 entrano in ingresso alla FSM-B.
"""

prompt_text = f"""Si vuole progettare il sistema digitale la cui architettura è rappresentata nella figura {image}.
 Il sistema riceve in ingresso un segnale IN, normalmente a zero, sul quale si presentano impulsi a livello uno di durata variabile da uno a sette periodi del segnale di clock CK. 

Il flip-flop D-net, visibile in alto a sinistra, re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

Il sistema misura la durata degli impulsi, in termini di periodi di clock, e trasmette il valore della misura sulla linea seriale SER. Per la misura della durata degli impulsi il sistema utilizza il contatore CNT. CNT è un contatore binario a tre bit con abilitazione. Nel sistema, CNT è sincronizzato dallo stesso clock CK della macchina a stati finiti FSM-A e da questa riceve il clear asincrono CL. La FSM-A segnala alla FSM-B, attivando per un periodo di CK il segnale TRG, che la misura della durata dell'impulso è terminata e il valore del conteggio è disponibile sulle uscite Q2, Q1 e Q0 del contatore stesso. Quando necessario, la FSM-A azzera il contatore CNT tramite il segnale CL; inoltre, per tutto il tempo in cui il sistema è impegnato nella ricezione dell’impulso e nella trasmissione su SER, la FSM-A attiva BSY.

La FSM-B inizia a generare l’uscita SER quando riceve l’impulso TRG. Il dato seriale in uscita è formato da un bit di start a '1', i tre bit Q0, Q1, Q2 (LSB in testa) con la durata dell'impulso e un bit di stop a '0'. 
Inoltre, la FSM-B genera l'uscita ETX per informare la FSM-A della fine della trasmissione su SER.

Di seguito è presente la descrizione del sistema:
{image_description}

Progetta la FSM-A e la FSM-B come macchine di Moore sincrone.
Realizza per ciascuna la tabella degli stati. 
"""

prompt_text_FW = f"""Si vuole progettare il sistema digitale la cui architettura è rappresentata nella figura {image}.
 Il sistema riceve in ingresso un segnale IN, normalmente a zero, sul quale si presentano impulsi a livello uno di durata variabile da uno a sette periodi del segnale di clock CK. 

Il flip-flop D-net, visibile in alto a sinistra, re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

Il sistema misura la durata degli impulsi, in termini di periodi di clock, e trasmette il valore della misura sulla linea seriale SER. Per la misura della durata degli impulsi il sistema utilizza il contatore CNT. CNT è un contatore binario a tre bit con abilitazione. Nel sistema, CNT è sincronizzato dallo stesso clock CK della macchina a stati finiti FSM-A e da questa riceve il clear asincrono CL. La FSM-A segnala alla FSM-B, attivando per un periodo di CK il segnale TRG, che la misura della durata dell'impulso è terminata e il valore del conteggio è disponibile sulle uscite Q2, Q1 e Q0 del contatore stesso. Quando necessario, la FSM-A azzera il contatore CNT tramite il segnale CL; inoltre, per tutto il tempo in cui il sistema è impegnato nella ricezione dell’impulso e nella trasmissione su SER, la FSM-A attiva BSY.

La FSM-B inizia a generare l’uscita SER quando riceve l’impulso TRG. Il dato seriale in uscita è formato da un bit di start a '1', i tre bit Q0, Q1, Q2 (LSB in testa) con la durata dell'impulso e un bit di stop a '0'. 
Inoltre, la FSM-B genera l'uscita ETX per informare la FSM-A della fine della trasmissione su SER.

Progetta la FSM-A e la FSM-B come macchine di Moore sincrone.
Realizza per ciascuna la tabella degli stati. 
"""

## 1.1 Esercizio senza ulteriore contesto

In [21]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione FSM-A

**Stati:**

* **S0 (IDLE):** Stato iniziale, in attesa di un impulso su IN.
* **S1 (COUNT):** Rilevato impulso su IN, conteggio abilitato.
* **S2 (WAIT_ETX):** Impulso terminato, attende fine trasmissione da FSM-B.

**Tabella degli stati FSM-A:**

| Stato Presente | EN | ETX | Stato Futuro | TRG | CL  | BSY |
|----------------|----|-----|-------------|------|-----|------|
| S0             | 0  | X   | S0          | 0    | 1   | 0    |
| S0             | 1  | X   | S1          | 0    | 0   | 1    |
| S1             | X  | X   | S1          | 0    | 0   | 1    |
| S1             | 0  | X   | S2          | 1    | 0   | 1    |
| S2             | X  | 0   | S2          | 0    | 0   | 1    |
| S2             | X  | 1   | S0          | 0    | 1   | 0    |

**Descrizione:**

* **S0 (IDLE):** Il sistema attende un impulso su EN (proveniente da IN). Se EN=1, passa allo stato S1 e inizia il conteggio, altrimenti rimane in S0. CL è attivo per resett

## 1.2 Esercizio con recupero diretto

In [22]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione FSM-A

**Stati:**

* **S0 (IDLE):** Stato iniziale, in attesa di un impulso in ingresso.
* **S1 (COUNT):** Ricevuto l'impulso, conteggio della durata.
* **S2 (WAIT_TX):** Conteggio terminato, attesa fine trasmissione FSM-B.

**Ingressi:**

* **EN:** Abilitazione, alto quando l'impulso è presente in ingresso.
* **ETX:** Fine trasmissione da FSM-B.

**Uscite:**

* **TRG:** Trigger per FSM-B, alto per un ciclo di clock quando la misura è pronta.
* **CL:** Clear per il contatore, attivo basso.
* **BSY:** Busy, alto quando il sistema è occupato.

**Tabella degli stati:**

| Stato Presente | EN | ETX | Stato Futuro | TRG | CL | BSY |
|---|---|---|---|---|---|---|
| S0 | 0 | X | S0 | 0 | 1 | 0 |
| S0 | 1 | X | S1 | 0 | 0 | 1 |
| S1 | X | 0 | S1 | 0 | 1 | 1 |
| S1 | X | 1 | S2 | 1 | 1 | 1 |
| S2 | X | X | S0 | 0 | 1 | 0 |


## Progettazione FSM-B

**Stati:**

* **S0 (IDLE):** Stato iniziale, in attesa del trigger da FSM-A.
* **S1 (START):** Invio del b

## 1.3 Esercizio con recupero diretto e video soluzione

In [24]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
## Progettazione FSM-A

**Diagramma degli stati:**

```
     +-----+     EN=1      +-----+     EN=0      +-----+
     |     |--------------->|     |--------------->|     |
     |  A  +---------------+  B  +---------------+  C  |
     |     |<--------------|     |<--------------|     |
     +-----+     EN=0      +-----+     EN=1      +-----+
        ^                      |                      |
        |                      |                      |
        |                      |                      |
        |                      |    CL=0, TRG=1, BSY=1|
        |                      |                      |
        |                      V                      |
        |                  +-----+     ETX=0     +-----+
        |                  |     |--------------->|     |
        |                  |  D  |--------------->|  E  |
        |                  |     |<--------------|     |
        |                  +-----+     ETX=1     +-----+
        |

## 1.4 Esercizio con video soluzione

In [25]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video is about the design of two finite state machines, FSM-A and FSM-B, as synchronous Moore machines, and the creation of a state table for each.

**FSM-A**

The video begins by explaining the role of FSM-A, which is to measure the duration of pulses received on the IN input signal. FSM-A has three inputs: EN, ETX, and -Reset, and three outputs: TRG, CL, and BSY.

The video then proceeds to design the state diagram for FSM-A. The initial state is A, also known as the reset state. In this state, the output CL is set to 1 to clear the counter CNT. FSM-A then waits for the EN input to become 1, indicating the arrival of a pulse. When EN becomes 1, FSM-A transitions to state B, where it sets the BSY output to 1 to indicate that the system is busy. FSM-A remains in state B until EN returns to 0, indicating the end of the pulse. At this point, FSM-A transitions to state C, where it sets the TRG output to 1 to signal to FSM-B that the pulse duration measurement 

## 1.5 Esercizio con libro di testo nel prompt

In [26]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Progetto FSM-A e FSM-B

### FSM-A

**Tabella degli stati:**

| Stato | X Y Z | EN | ETX | Stato Successivo | TRG | CL | BSY |
|---|---|---|---|---|---|---|---|
| S0 | 0 0 0 | 0 | - | S0 | 0 | 0 | 0 |
| S0 | 0 0 0 | 1 | - | S1 | 0 | 1 | 1 |
| S1 | 0 0 1 | 1 | - | S2 | 0 | 0 | 1 |
| S2 | 0 1 0 | 1 | - | S3 | 0 | 0 | 1 |
| S3 | 0 1 1 | 1 | - | S4 | 0 | 0 | 1 |
| S4 | 1 0 0 | 1 | - | S5 | 0 | 0 | 1 |
| S5 | 1 0 1 | 1 | - | S6 | 0 | 0 | 1 |
| S6 | 1 1 0 | 1 | - | S7 | 0 | 0 | 1 |
| S7 | 1 1 1 | 1 | - | S7 | 0 | 0 | 1 |
| S1 | 0 0 1 | 0 | - | S8 | 1 | 0 | 1 |
| S2 | 0 1 0 | 0 | - | S8 | 1 | 0 | 1 |
| S3 | 0 1 1 | 0 | - | S8 | 1 | 0 | 1 |
| S4 | 1 0 0 | 0 | - | S8 | 1 | 0 | 1 |
| S5 | 1 0 1 | 0 | - | S8 | 1 | 0 | 1 |
| S6 | 1 1 0 | 0 | - | S8 | 1 | 0 | 1 |
| S7 | 1 1 1 | 0 | - | S8 | 1 | 0 | 1 |
| S8 | 0 0 0 | - | 0 | S8 | 0 | 0 | 1 |
| S8 | 0 0 0 | - | 1 | S0 | 0 | 0 | 0 |

**Descrizione:**

* **S0:** Stato di attesa di un impulso su EN. Azzera il contatore (CL=1)

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [27]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Progetto FSM-A

**Tabella degli stati FSM-A**

| Stato | ZYX | EN | ETX | TRG | CL | BSY | Stato Successivo |
|---|---|---|---|---|---|---|---|
| A | 000 | 0 | - | 0 | 1 | 0 | A |
| A | 000 | 1 | - | 0 | 1 | 0 | B |
| B | 001 | - | - | 0 | 0 | 1 | C |
| C | 011 | 1 | - | 0 | 0 | 1 | C |
| C | 011 | 0 | - | 1 | 0 | 1 | D |
| D | 010 | - | 0 | 0 | 0 | 1 | D |
| D | 010 | - | 1 | 0 | 1 | 0 | A |


**Diagramma degli stati FSM-A**

```
     EN=1
     ┌────┐
     ▼     │
  ┌───┐    │
  │ B ├────┘
  └───┐    │
      ▼    │ EN=0
  ┌───┐    │
  │ C ├────┘
  └───┐    │
   EN=0  ▼    │
  ┌───┐    │
  │ D ├────┘
  └───┐    │
      ▼    │ ETX=1
     ┌────┘
     ▼
  ┌───┐
  │ A │
  └───┐
      ▼
     -Reset
```

**Descrizione del funzionamento FSM-A**

* **Stato A (000):** Stato di inizializzazione. Il contatore CNT viene azzerato (CL=1) e le uscite TRG e BSY sono disattivate. Se EN=1, significa che è stato rilevato un impulso in ingresso e la macchina passa allo stato B.

## 1.7 Esercizio con recupero mediante tool

In [30]:
prompt_text = f"""Si vuole progettare il sistema digitale la cui architettura è descritta di seguito:
{image_description}

Il sistema riceve in ingresso un segnale IN, normalmente a zero, sul quale si presentano impulsi a livello uno di durata variabile da uno a sette periodi del segnale di clock CK. 

Il flip-flop D-net, visibile in alto a sinistra, re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

Il sistema misura la durata degli impulsi, in termini di periodi di clock, e trasmette il valore della misura sulla linea seriale SER. Per la misura della durata degli impulsi il sistema utilizza il contatore CNT. CNT è un contatore binario a tre bit con abilitazione. Nel sistema, CNT è sincronizzato dallo stesso clock CK della macchina a stati finiti FSM-A e da questa riceve il clear asincrono CL. La FSM-A segnala alla FSM-B, attivando per un periodo di CK il segnale TRG, che la misura della durata dell'impulso è terminata e il valore del conteggio è disponibile sulle uscite Q2, Q1 e Q0 del contatore stesso. Quando necessario, la FSM-A azzera il contatore CNT tramite il segnale CL; inoltre, per tutto il tempo in cui il sistema è impegnato nella ricezione dell’impulso e nella trasmissione su SER, la FSM-A attiva BSY.

La FSM-B inizia a generare l’uscita SER quando riceve l’impulso TRG. Il dato seriale in uscita è formato da un bit di start a '1', i tre bit Q0, Q1, Q2 (LSB in testa) con la durata dell'impulso e un bit di stop a '0'. 
Inoltre, la FSM-B genera l'uscita ETX per informare la FSM-A della fine della trasmissione su SER.

Progetta la FSM-A e la FSM-B e realizza per ciascuna la tabella degli stati. """

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## Progettazione FSM-A

**Stati:**

* **IDLE:** Stato iniziale, in attesa di un impulso su EN.
* **COUNT:** Conteggio della durata dell'impulso su EN.
* **SEND:** Invio del segnale TRG e attesa della fine della trasmissione (ETX).

**Tabella degli stati:**

| Stato corrente | EN | ETX | Stato futuro | TRG | CL  | BSY |
|--------------|----|-----|-------------|------|-----|-----|
| IDLE        | 0  | X   | IDLE        | 0    | 0   | 0   |
| IDLE        | 1  | X   | COUNT       | 0    | 1   | 1   |
| COUNT       | X  | X   | COUNT       | 0    | 0   | 1   |
| COUNT       | 0  | X   | SEND        | 1    | 0   | 1   |
| SEND        | X  | 0   | SEND        | 0    | 0   | 1   |
| SEND        | X  | 1   | IDLE        | 0    | 0   | 0   |

## Progettazione FSM-B

**Stati:**

* **IDLE:** Stato iniziale, in attesa del segnale TRG.
* **START:** Invio del bit di start ('1').
* **Q0:** Invio del bit Q0.
* **Q1:** Invio del bit Q1.
* **Q2:** Invio del bit Q2.
* **STOP:** Invio del bit di stop ('0')

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "fsm a progettazione sistema digitale"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "fsm a progettazione sistema digitale"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [28]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
## Progettazione FSM-A

**Tabella degli stati:**

| Stato | X | Y | Z | EN | ETX | TRG | CL | BSY | Stato Successivo | iX | iY | iZ |
|---|---|---|---|---|---|---|---|---|---|---|---|
| idle | 0 | 0 | 0 | 0 | - | 0 | 1 | 0 | idle | 0 | 0 | 0 |
| idle | 0 | 0 | 0 | 1 | - | 0 | 1 | 0 | count | 0 | 0 | 1 |
| count | 0 | 0 | 1 | 0 | - | 0 | 0 | 1 | wait | 0 | 1 | 0 |
| count | 0 | 0 | 1 | 1 | - | 0 | 0 | 1 | count | 0 | 0 | 1 |
| wait | 0 | 1 | 0 | 0 | - | 1 | 0 | 1 | send | 1 | 0 | 0 |
| wait | 0 | 1 | 0 | 1 | - | 1 | 0 | 1 | send | 1 | 0 | 0 |
| send | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | send | 1 | 0 | 0 |
| send | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | send | 1 | 0 | 0 |
| send | 1 | 0 | 0 | - | 1 | 0 | 1 | 0 | idle | 0 | 0 | 0 |

**Descrizione del funzionamento:**

* **idle:** Stato iniziale. La FSM attende l'arrivo di un impulso sul segnale EN.
* **count:** La FSM abilita il contatore CNT e attende che l'impulso su EN termini.
* **wait:** La FSM attende un ciclo di clock

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [29]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
## Progetto FSM-A

+-------+---+---+---+---+----+----+----+-----------------+---+---+---+
| Stato | Z | Y | X | EN | ETX | TRG | CL | Stato Successivo | iZ | iY | iX |
+-------+---+---+---+---+----+----+----+-----------------+---+---+---+
|   a   | 0 | 0 | 0 |  0 |  -  |  0  |  1 |        a        |  0 |  0 |  0 |
|   a   | 0 | 0 | 0 |  1 |  -  |  0  |  1 |        b        |  0 |  0 |  1 |
|   b   | 0 | 0 | 1 |  0 |  -  |  0  |  0 |        a        |  0 |  0 |  0 |
|   b   | 0 | 0 | 1 |  1 |  -  |  0  |  0 |        c        |  0 |  1 |  0 |
|   c   | 0 | 1 | 0 |  - |  -  |  0  |  0 |        c        |  0 |  1 |  0 |
|   c   | 0 | 1 | 0 |  - |  0  |  1  |  0 |        d        |  0 |  1 |  1 |
|   c   | 0 | 1 | 0 |  - |  1  |  -  |  0 |        a        |  0 |  0 |  0 |
|   d   | 0 | 1 | 1 |  - |  -  |  0  |  0 |        d        |  0 |  1 |  1 |
|   d   | 0 | 1 | 1 |  - |  0  |  -  |  0 |        e        |  1 |  0 |  0 |
|   e   | 1 | 0 | 0 |  - |  -  |  0  |  0 |

# Esercizio 18

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image18>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video18>.mp4")

image_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha tre ingressi (LN, CK, -Reset) e due uscite (PS, SYN).
Le due FSM (RX FSM, PUSLE FSM) rivestono il ruolo di controllore e vanno progettate.

* **La RX FSM ha:
    * variabili di stato X, Y e Z;
    * uscite GO, J0, K0, J1, K1, J2 e K2;
    * ingresso LS.
    * segnale di clock CK.
    * segnale di inizializzazione -Reset.
    
* **La PULSE FSM ha:
    * variabili di stato X, Y e Z;
    * uscite PS e SYN;
    * ingressi GO, P0, P1 e P2.
    * segnale di clock CK.
    * segnale di inizializzazione -Reset. 
    
Il datapath è costituito da:

* ** Flip-flop D-NET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * L'ingresso D è collegato al segnale di ingresso LN.
    * L'uscita (Q) entra in ingresso alla RX FSM col il nome LS. 
    
* ** Primo flip-flop JK-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * L'ingresso J è collegato al segnale J0 che esce dalla RX FSM. 
    * L'ingresso K è collegato al segnalo K0 che esce dalla RX FSM.
    * L'uscita (Q) è collegata all'ingresso D del primo filp-flop D-PET. L'uscita Q è chiamata Q0. 

* ** Secondo flip-flop JK-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * L'ingresso J è collegato al segnale J1 che esce dalla RX FSM. 
    * L'ingresso K è collegato al segnalo K1 che esce dalla RX FSM.
    * L'uscita (Q) è collegata all'ingresso D del secondo filp-flop D-PET. L'uscita Q è chiamata Q1. 

* ** Terzo flip-flop JK-PET **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * L'ingresso J è collegato al segnale J2 che esce dalla RX FSM. 
    * L'ingresso K è collegato al segnalo K2 che esce dalla RX FSM.
    * L'uscita (Q) è collegata all'ingresso D del terzo filp-flop D-PET. L'uscita Q è chiamata Q2. 

* ** Primo flip-flop D-PET **
    * L'ingresso del segnale di clock (CK) è il segnale GO che esce dalla RX FSM.
    * L'ingresso D è collegato all'uscita del primo flip-flop JK-PET.
    * L'uscita (Q) entra in ingresso alla PULSE FSM col il nome P0. 

* ** Secondo flip-flop D-PET **
    * L'ingresso del segnale di clock (CK) è il segnale GO che esce dalla RX FSM.
    * L'ingresso D è collegato all'uscita del secondo flip-flop JK-PET.
    * L'uscita (Q) entra in ingresso alla PULSE FSM col il nome P1. 

* ** Terzo flip-flop D-PET **
    * L'ingresso del segnale di clock (CK) è il segnale GO che esce dalla RX FSM.
    * L'ingresso D è collegato all'uscita del terzo flip-flop JK-PET.
    * L'uscita (Q) entra in ingresso alla PULSE FSM col il nome P2. 
"""

prompt_text = f"""Si vuole progettare un sistema digitale sincrono in grado di ricevere comandi attraverso una linea seriale e, conseguentemente, generare impulsi la cui durata dipende dalle informazioni contenute nei comandi stessi. La figura {image} mostra l'architettura del sistema, composto da due macchine a stati finiti e alcuni flip-flop. 

I comandi sono costituiti da pacchetti seriali, ricevuti tramite l'ingresso LN. Il pacchetto prevede un bit di start, tre bit di dato D0, D1 e D2, ed infine due bit di stop. Il tempo di bit è pari ad un ciclo di clock, e il cambiamento del bit è sincrono con i fronti di salita del clock CK.

Il flip-flop D-net re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

La macchina a stati finiti RX FSM ha il compito di ricevere e decodificare le sequenze re-sincronizzate dal flip-flop. Durante la ricezione di un nuovo pacchetto di dati, la RX FSM memorizza uno dopo l'altro i tre bit di informazione nei tre flip-flop JK-pet (D0 è memorizzato in Q0, D1 in Q1, e D2 in Q2). Se la sequenza ricevuta è valida (ossia i due bit di stop sono ricevuti correttamente a zero), la RX FSM attiva, per la durata di un ciclo di clock, il segnale GO.
L'attivazione del segnale GO copia inoltre sui flip-flop P0, P1 e P2 l'informazione ricevuta. Lo scopo di questa operazione è di permettere al ricevitore RX FSM di ricevere, subito dopo, una nuova sequenza di bit.

La macchina a stati finiti PULSE FSM attende l'attivazione del segnale GO. Quando questo è portato a '1', la PULSE FSM inizia la generazione di un impulso sull'uscita PS, la cui durata è funzione delle uscite dei flip-flop P0 e P1. La durata è definita come T= (2 + P1P0), dove P1P0 è un numero binario (P1 = MSB, P0 = LSB).
Se l'uscita del flip-flop P2 è a '1', la PULSE FSM attiva l'uscita SYN durante l' ultimo ciclo di clock dell'impulso PS, per avvisare che la generazione dell'impulso sta terminando.

Il reset asincrono !Reset forza l'intero sistema in uno stato di riposo, in cui attende l'arrivo di un bit di start sulla linea LN.

Di seguito è presente la descrizione del sistema:
{image_description}

Progetta la RX FSM come macchina di Mealy sincrona e la PULSE FSM come macchina di Moore sincrona.
Realizza per ciascuna la tabella degli stati. 
"""

prompt_text_FW = f"""Si vuole progettare un sistema digitale sincrono in grado di ricevere comandi attraverso una linea seriale e, conseguentemente, generare impulsi la cui durata dipende dalle informazioni contenute nei comandi stessi. La figura {image} mostra l'architettura del sistema, composto da due macchine a stati finiti e alcuni flip-flop. 

I comandi sono costituiti da pacchetti seriali, ricevuti tramite l'ingresso LN. Il pacchetto prevede un bit di start, tre bit di dato D0, D1 e D2, ed infine due bit di stop. Il tempo di bit è pari ad un ciclo di clock, e il cambiamento del bit è sincrono con i fronti di salita del clock CK.

Il flip-flop D-net re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

La macchina a stati finiti RX FSM ha il compito di ricevere e decodificare le sequenze re-sincronizzate dal flip-flop. Durante la ricezione di un nuovo pacchetto di dati, la RX FSM memorizza uno dopo l'altro i tre bit di informazione nei tre flip-flop JK-pet (D0 è memorizzato in Q0, D1 in Q1, e D2 in Q2). Se la sequenza ricevuta è valida (ossia i due bit di stop sono ricevuti correttamente a zero), la RX FSM attiva, per la durata di un ciclo di clock, il segnale GO.
L'attivazione del segnale GO copia inoltre sui flip-flop P0, P1 e P2 l'informazione ricevuta. Lo scopo di questa operazione è di permettere al ricevitore RX FSM di ricevere, subito dopo, una nuova sequenza di bit.

La macchina a stati finiti PULSE FSM attende l'attivazione del segnale GO. Quando questo è portato a '1', la PULSE FSM inizia la generazione di un impulso sull'uscita PS, la cui durata è funzione delle uscite dei flip-flop P0 e P1. La durata è definita come T= (2 + P1P0), dove P1P0 è un numero binario (P1 = MSB, P0 = LSB).
Se l'uscita del flip-flop P2 è a '1', la PULSE FSM attiva l'uscita SYN durante l' ultimo ciclo di clock dell'impulso PS, per avvisare che la generazione dell'impulso sta terminando.

Il reset asincrono !Reset forza l'intero sistema in uno stato di riposo, in cui attende l'arrivo di un bit di start sulla linea LN.

Progetta la RX FSM come macchina di Mealy sincrona e la PULSE FSM come macchina di Moore sincrona.
Realizza per ciascuna la tabella degli stati. 
"""

## 1.1 Esercizio senza ulteriore contesto

In [32]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione della RX FSM (Mealy)

**Stati:**

* **S0 (IDLE):** Stato iniziale, in attesa del bit di start.
* **S1 (START):** Ricevuto il bit di start, in attesa del primo bit di dato (D0).
* **S2 (DATA1):** Ricevuto D0, in attesa del secondo bit di dato (D1).
* **S3 (DATA2):** Ricevuto D1, in attesa del terzo bit di dato (D2).
* **S4 (STOP1):** Ricevuto D2, in attesa del primo bit di stop.
* **S5 (STOP2):** Ricevuto il primo bit di stop, in attesa del secondo bit di stop.
* **S6 (VALID):** Ricevuto il secondo bit di stop, pacchetto valido.

**Tabella degli stati:**

| Stato corrente | LS | Stato futuro | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S0 | 1 | S1 | 0 | 0 | X | 0 | X | 0 | X |
| S1 | 0 | S2 | 0 | LS | ~LS | 0 | X | 0 | X |
| S1 | 1 | S2 | 0 | LS | ~LS | 0 | X | 0 | X |
| S2 | 0 | S3 | 0 | 0 | X | LS | ~LS | 0 | X |
| S2 | 1 | S3 | 0 | 0 | X | LS | ~LS | 0 | X |
| S3 

## 1.2 Esercizio con recupero diretto

In [33]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione RX FSM (Mealy)

**Stati:**

* S0: Attesa bit di start
* S1: Ricezione D0
* S2: Ricezione D1
* S3: Ricezione D2
* S4: Ricezione primo bit di stop
* S5: Ricezione secondo bit di stop
* S6: Pacchetto valido

**Tabella degli stati:**

| Stato | LS | Stato successivo | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S0 | 1 | S1 | 0 | 0 | X | 0 | X | 0 | X |
| S1 | X | S2 | 0 | LS | X | 0 | X | 0 | X |
| S2 | X | S3 | 0 | 0 | X | LS | X | 0 | X |
| S3 | X | S4 | 0 | 0 | X | 0 | X | LS | X |
| S4 | 0 | S5 | 0 | 0 | X | 0 | X | 0 | X |
| S4 | 1 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S5 | 0 | S6 | 1 | 0 | X | 0 | X | 0 | X |
| S5 | 1 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| S6 | X | S0 | 0 | 0 | X | 0 | X | 0 | X |

**Descrizione:**

* La FSM attende in S0 l'arrivo di un bit di start (LS = 1).
* Quando arriva il bit di start, la FSM passa agli stati S1, S2, e S3 per ricevere i bit di d

## 1.3 Esercizio con recupero diretto e video soluzione

In [34]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
## Progettazione RX FSM

**Diagramma degli stati:**

```
     LS=0      LS=1
  ┌─────┐   ┌─────┐
  │  A  │───▶│  B  │
  └───┬─┘   └─┬───┘
      │       │
      │LS=1  │D0
      ▼       ▼
     LS=0      LS=1
  ┌─────┐   ┌─────┐
  │  C  │───▶│  D  │
  └───┬─┘   └─┬───┘
      │       │
      │D1     │D2
      ▼       ▼
     LS=0      LS=1
  ┌─────┐   ┌─────┐
  │  E  │───▶│  F  │
  └───┬─┘   └─┬───┘
      │       │
      │STOP0  │STOP1
      ▼       ▼
     LS=0      LS=1
  ┌─────┐   ┌─────┐
  │  G  │───▶│  H  │
  └─────┘   └───┬─┘
                │
                │LS=0/GO=1
                ▼
                A
```

**Tabella degli stati:**

| Stato | Z | Y | X | LS | GO | J0 | K0 | J1 | K1 | J2 | K2 | Stato Successivo |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | A |
| A | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | B |
| B | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | C |
| B | 0 | 0 | 1 | 1 | 0 | 1 | 0 |

## 1.4 Esercizio con video soluzione

In [41]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The video is a solution to the problem of designing two finite state machines (FSMs) for a synchronous digital system. The system receives commands through a serial line and generates pulses based on the command information.

**RX FSM (Receiver FSM)**

**State Diagram:**

The RX FSM has seven states: A, B, C, D, E, F, and H.

* **State A (Reset):**  The initial state. The FSM waits for a start bit ('1') on the input line (LS).
* **State B (Start Bit Received):** The FSM has received the start bit. It waits for the first data bit (D0).
* **State C (D0 Received):** The FSM has received D0. It waits for the second data bit (D1).
* **State D (D1 Received):** The FSM has received D1. It waits for the third data bit (D2).
* **State E (D2 Received):** The FSM has received D2. It waits for the first stop bit.
* **State F (First Stop Bit Received):** The FSM has received the first stop bit. It waits for the second stop bit.
* **State H (Second Stop Bit Received):** The 

## 1.5 Esercizio con libro di testo nel prompt

In [36]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Progettazione RX FSM

La RX FSM è una macchina di Mealy sincrona con le seguenti caratteristiche:

* **Ingressi:**
    * LS: linea seriale re-sincronizzata
* **Uscite:**
    * GO: segnale di attivazione per la PULSE FSM
    * J0, K0, J1, K1, J2, K2: ingressi dei flip-flop JK-PET
* **Variabili di stato:**
    * X, Y, Z
* **Segnale di clock:** CK
* **Segnale di inizializzazione:** !Reset

**Tabella degli stati RX FSM:**

| Stato | X Y Z | LS | GO | J0 K0 | J1 K1 | J2 K2 | Stato successivo |
|---|---|---|---|---|---|---|---|
| Idle | 0 0 0 | 0 | 0 | 0 X | 0 X | 0 X | Idle |
| Idle | 0 0 0 | 1 | 0 | 0 X | 0 X | 0 X | Ricezione_Start |
| Ricezione_Start | 0 0 1 | X | 0 | LS X | 0 X | 0 X | Ricezione_D0 |
| Ricezione_D0 | 0 1 0 | X | 0 | 0 X | LS X | 0 X | Ricezione_D1 |
| Ricezione_D1 | 0 1 1 | X | 0 | 0 X | 0 X | LS X | Ricezione_D2 |
| Ricezione_D2 | 1 0 0 | X | 0 | 0 X | 0 X | 0 X | Ricezione_Stop1 |
| Ricezione_Stop1 | 1 0 1 | 0 | 0 | 0 X | 0 X | 0 X | Ricezi

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [37]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Progettazione RX FSM (Mealy Sincrona)

**Tabella degli stati:**

| Stato | ZYX | LS | GO | J0 K0 | J1 K1 | J2 K2 | Stato Successivo |
|---|---|---|---|---|---|---|---|
| a | 000 | 0 | 0 | 00 | 00 | 00 | a |
| a | 000 | 1 | 0 | 00 | 00 | 00 | b |
| b | 001 | 0 | 0 | 11 | 00 | 00 | c |
| b | 001 | 1 | 0 | 00 | 00 | 00 | g |
| c | 010 | 0 | 0 | 00 | 11 | 00 | d |
| c | 010 | 1 | 0 | 00 | 00 | 00 | g |
| d | 011 | 0 | 0 | 00 | 00 | 11 | e |
| d | 011 | 1 | 0 | 00 | 00 | 00 | g |
| e | 100 | 0 | 0 | 00 | 00 | 00 | f |
| e | 100 | 1 | 0 | 00 | 00 | 00 | g |
| f | 101 | 0 | 1 | 00 | 00 | 00 | a |
| f | 101 | 1 | 0 | 00 | 00 | 00 | g |
| g | 110 | - | 0 | 00 | 00 | 00 | a |

**Variabili di stato:** X, Y, Z

**Ingresso:** LS

**Uscite:** GO, J0, K0, J1, K1, J2, K2

**Descrizione:**

* **Stato a:** Stato di reset, attende l'arrivo del bit di start (LS = 1).
* **Stato b:** Ricevuto il bit di start, attende il primo bit di dato (D0).
* **Stato c:** Ricevuto D0, attende 

## 1.7 Esercizio con recupero mediante tool

In [38]:
prompt_text = f"""Si vuole progettare un sistema digitale sincrono in grado di ricevere comandi attraverso una linea seriale e, conseguentemente, generare impulsi la cui durata dipende dalle informazioni contenute nei comandi stessi. L'architettura del sistema, composto da due macchine a stati finiti e alcuni flip-flop, è descritta di seguito:
{image_description}

I comandi sono costituiti da pacchetti seriali, ricevuti tramite l'ingresso LN. Il pacchetto prevede un bit di start, tre bit di dato D0, D1 e D2, ed infine due bit di stop. Il tempo di bit è pari ad un ciclo di clock, e il cambiamento del bit è sincrono con i fronti di salita del clock CK.

Il flip-flop D-net re-sincronizza sul fronte di discesa del clock le sequenze di bit ricevute.

La macchina a stati finiti RX FSM ha il compito di ricevere e decodificare le sequenze re-sincronizzate dal flip-flop. Durante la ricezione di un nuovo pacchetto di dati, la RX FSM memorizza uno dopo l'altro i tre bit di informazione nei tre flip-flop JK-pet (D0 è memorizzato in Q0, D1 in Q1, e D2 in Q2). Se la sequenza ricevuta è valida (ossia i due bit di stop sono ricevuti correttamente a zero), la RX FSM attiva, per la durata di un ciclo di clock, il segnale GO.
L'attivazione del segnale GO copia inoltre sui flip-flop P0, P1 e P2 l'informazione ricevuta. Lo scopo di questa operazione è di permettere al ricevitore RX FSM di ricevere, subito dopo, una nuova sequenza di bit.

La macchina a stati finiti PULSE FSM attende l'attivazione del segnale GO. Quando questo è portato a '1', la PULSE FSM inizia la generazione di un impulso sull'uscita PS, la cui durata è funzione delle uscite dei flip-flop P0 e P1. La durata è definita come T= (2 + P1P0), dove P1P0 è un numero binario (P1 = MSB, P0 = LSB).
Se l'uscita del flip-flop P2 è a '1', la PULSE FSM attiva l'uscita SYN durante l' ultimo ciclo di clock dell'impulso PS, per avvisare che la generazione dell'impulso sta terminando.

Il reset asincrono !Reset forza l'intero sistema in uno stato di riposo, in cui attende l'arrivo di un bit di start sulla linea LN.

Progetta la RX FSM e la PULSE FSM e realizza per ciascuna la tabella degli stati. """

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

## RX FSM

**Stati:**

* **IDLE:** Stato iniziale, in attesa del bit di start.
* **START:** Bit di start ricevuto, pronto a ricevere D0.
* **D0:** Ricevuto D0, pronto a ricevere D1.
* **D1:** Ricevuto D1, pronto a ricevere D2.
* **D2:** Ricevuto D2, pronto a ricevere il primo bit di stop.
* **STOP1:** Ricevuto il primo bit di stop, pronto a ricevere il secondo bit di stop.
* **STOP2:** Ricevuto il secondo bit di stop, pacchetto valido.
* **ERRORE:** Ricevuta una sequenza non valida.

**Tabella degli stati:**

| Stato Corrente | LS | Prossimo Stato | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| IDLE | 0 | IDLE | 0 | 0 | X | 0 | X | 0 | X |
| IDLE | 1 | START | 0 | 0 | X | 0 | X | 0 | X |
| START | X | D0 | 0 | LS | X | 0 | X | 0 | X |
| D0 | X | D1 | 0 | 0 | X | LS | X | 0 | X |
| D1 | X | D2 | 0 | 0 | X | 0 | X | LS | X |
| D2 | X | STOP1 | 0 | 0 | X | 0 | X | 0 | X |
| STOP1 | 0 | STOP2 | 0 | 0 | X | 0 | X | 0 | X |
| STOP1 | 1 | ERRORE | 0 | 0 | X | 

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {}
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {}
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [39]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
## Progettazione delle FSM

### RX FSM (Mealy)

La RX FSM riceve in ingresso il segnale LS, re-sincronizzato dal flip-flop D-NET, e ha il compito di riconoscere la sequenza di un pacchetto valido, memorizzando i bit di dato D0, D1 e D2 nei flip-flop JK-PET. 

**Tabella degli stati:**

| Stato | LS | Stato Successivo | GO | J0 | K0 | J1 | K1 | J2 | K2 |
|---|---|---|---|---|---|---|---|---|---|
| **S0** (Attesa Start) | 0 | S0 | 0 | 0 | X | 0 | X | 0 | X |
| **S0** (Attesa Start) | 1 | S1 | 0 | 0 | X | 0 | X | 0 | X |
| **S1** (Ricezione D0) | 0 | S2 | 0 | 1 | 0 | 0 | X | 0 | X |
| **S1** (Ricezione D0) | 1 | S2 | 0 | 0 | 1 | 0 | X | 0 | X |
| **S2** (Ricezione D1) | 0 | S3 | 0 | X | X | 1 | 0 | 0 | X |
| **S2** (Ricezione D1) | 1 | S3 | 0 | X | X | 0 | 1 | 0 | X |
| **S3** (Ricezione D2) | 0 | S4 | 0 | X | X | X | X | 1 | 0 |
| **S3** (Ricezione D2) | 1 | S4 | 0 | X | X | X | X | 0 | 1 |
| **S4** (Ricezione Stop 1) | 0 | S5 | 0 | X | X | X | X | X | X |
| **S4*

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [40]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
## Progetto RX FSM

La RX FSM è una macchina di Mealy sincrona con:

* **Ingressi:** LS (Line Serial)
* **Uscite:** GO, J0, K0, J1, K1, J2, K2
* **Variabili di stato:** X, Y, Z

La tabella degli stati della RX FSM è la seguente:

| Stato | ZYX | LS | Uscite J2 J1 J0 K2 K1 K0 GO | Stato Successivo |
|---|---|---|---|---|
| a | 000 | 0 | 0 0 0 0 0 0 0 | a |
| a | 000 | 1 | 0 0 0 0 0 0 0 | b |
| b | 001 | 0 | 0 0 1 0 0 0 0 | c |
| b | 001 | 1 | 0 0 0 0 0 0 0 | err |
| c | 010 | 0 | 0 1 0 0 0 0 0 | d |
| c | 010 | 1 | 0 0 0 0 0 0 0 | err |
| d | 011 | 0 | 1 0 0 0 0 0 0 | e |
| d | 011 | 1 | 0 0 0 0 0 0 0 | err |
| e | 100 | 0 | 0 0 0 0 0 0 0 | f |
| e | 100 | 1 | 0 0 0 0 0 0 0 | err |
| f | 101 | 0 | 0 0 0 1 1 1 0 | g |
| f | 101 | 1 | 0 0 0 0 0 0 0 | err |
| g | 110 | 0 | 0 0 0 0 0 0 1 | a |
| g | 110 | 1 | 0 0 0 0 0 0 0 | err |
| err | 111 | - | 0 0 0 0 0 0 0 | a |

**Descrizione del funzionamento:**

* **Stato a (000):** Stato iniziale di reset. La macchina atte

# Esercizio 19

## 1.0 Definizione del prompt

In [ ]:
image = Part.from_uri(mime_type="image/png", uri="gs://<bucket-name-it>/<image19>.png")
video = Part.from_uri(mime_type="video/mp4", uri="gs://<bucket-name-it>/<video19>.mp4")

image_description = """Il sistema presenta una struttura controllore - datapath. 
Il sistema ha quattrordici ingressi (CK, -Reset, PA3...PA0, PB3...PB0, PC3...PC0) e tre uscite (UA, UB, UC).
La CONTROL FSM riveste il ruolo di controllore e va progettata.

* **La CONTROL FSM ha:
    * variabili di stato X, Y e Z;
    * uscite UA, UB, UC, LDA, LDB e LDC;
    * ingressi TCA, TCB e TCC.
    * segnale di clock CK.
    * segnale di inizializzazione -Reset.
    
Il datapath è costituito da:

* ** Contatore universale a 4 bit A **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione al conteggio (En) è costante a uno.
    * Il segnale di abilitazione di TC (Et) è costante a uno. 
    * Il segnale U/-D che indica la direzione del conteggio è costante a zero.
    * Riceve in ingresso -Reset come segnale di -Clear. 
    * Il segnale di caricamento (LD) è collegato al segnale LDA che esce dalla CONTROL FSM.
    * L'uscita TC entra in ingresso alla CONTROL FSM con il nome TCA.
    * Gli ingressi P3...P0 sono gli ingressi PA3...PA0 del sistema.
    * Le uscite Q3...Q0 non sono collegate a niente.

* ** Contatore universale a 4 bit B **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione al conteggio (En) è costante a uno.
    * Il segnale di abilitazione di TC (Et) è costante a uno. 
    * Il segnale U/-D che indica la direzione del conteggio è costante a zero.
    * Riceve in ingresso -Reset come segnale di -Clear. 
    * Il segnale di caricamento (LD) è collegato al segnale LDB che esce dalla CONTROL FSM.
    * L'uscita TC entra in ingresso alla CONTROL FSM con il nome TCB.
    * Gli ingressi P3...P0 sono gli ingressi PB3...PB0 del sistema.
    * Le uscite Q3...Q0 non sono collegate a niente.

* ** Contatore universale a 4 bit C **
    * Riceve il segnale di clock (CK) comune a tutti i componenti sincroni.
    * Il segnale di abilitazione al conteggio (En) è costante a uno.
    * Il segnale di abilitazione di TC (Et) è costante a uno. 
    * Il segnale U/-D che indica la direzione del conteggio è costante a zero.
    * Riceve in ingresso -Reset come segnale di -Clear. 
    * Il segnale di caricamento (LD) è collegato al segnale LDC che esce dalla CONTROL FSM.
    * L'uscita TC entra in ingresso alla CONTROL FSM con il nome TCC.
    * Gli ingressi P3...P0 sono gli ingressi PC3...PC0 del sistema.
    * Le uscite Q3...Q0 non sono collegate a niente.
"""

prompt_text = f"""Si vuole progettare un generatore di onde quadre multifase programmabile. 

Questo dispositivo genera tre segnali ciclici, UA, UB e UC, che si comportano come di seguito:
- inizialmente, UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA è alto per un numero di cicli di clock pari a (TA+1), UB e UC nel frattempo sono bassi.
- di seguito UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA torna basso, UB diventa alto per un numero di cicli di clock pari a (TB+1), UC riamane basso.
- segue che UA, UB e UC sono tutti bassi per un ciclo di clock;
- infine, UA rimane basso, UB torna basso, UC diventa alto per un numero di cicli di clock pari a (TC+1).

La durata del tempo a uno di ciascun segnale (TA, TB, TC) sono programmabili in modo indipendente. L'architettura del sistema è descritta nella figura {image}. 
Tutti i moduli sono sincronizzati con il clock principale CK. La macchina a stati finiti genera i segnali richiesti UA, UB e UC. I tre contatori universali a 4 bit (Cnt4) sono incaricati di impostare la durata temporale di ciascun segnale di uscita e sono controllati dalla FSM (Finite State Machine). 

Di seguito è presente la descrizione del sistema:
{image_description}

Progetta la CONTROL FSM come macchina di Moore sincrona e realizzane la tabella degli stati.
"""

prompt_text_FW = f"""Si vuole progettare un generatore di onde quadre multifase programmabile. 

Questo dispositivo genera tre segnali ciclici, UA, UB e UC, che si comportano come di seguito:
- inizialmente, UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA è alto per un numero di cicli di clock pari a (TA+1), UB e UC nel frattempo sono bassi.
- di seguito UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA torna basso, UB diventa alto per un numero di cicli di clock pari a (TB+1), UC riamane basso.
- segue che UA, UB e UC sono tutti bassi per un ciclo di clock;
- infine, UA rimane basso, UB torna basso, UC diventa alto per un numero di cicli di clock pari a (TC+1).

La durata del tempo a uno di ciascun segnale (TA, TB, TC) sono programmabili in modo indipendente. L'architettura del sistema è descritta nella figura {image}. 
Tutti i moduli sono sincronizzati con il clock principale CK. La macchina a stati finiti genera i segnali richiesti UA, UB e UC. I tre contatori universali a 4 bit (Cnt4) sono incaricati di impostare la durata temporale di ciascun segnale di uscita e sono controllati dalla FSM (Finite State Machine). 


Progetta la CONTROL FSM come macchina di Moore sincrona e realizzane la tabella degli stati.
"""

## 1.1 Esercizio senza ulteriore contesto

In [43]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione della CONTROL FSM

La CONTROL FSM è una macchina di Moore sincrona con:

* **Ingressi:** CK, -Reset, TCA, TCB, TCC
* **Uscite:** UA, UB, UC, LDA, LDB, LDC
* **Variabili di stato:** X, Y, Z (per un totale di 8 stati possibili)

Per progettare la FSM, analizziamo il comportamento desiderato del sistema e definiamo gli stati e le transizioni:

**Stati:**

* **S0 (XYZ = 000):** Stato iniziale, tutti i segnali bassi.
* **S1 (XYZ = 001):** Caricamento del valore TA+1 nel contatore A.
* **S2 (XYZ = 010):** Attesa che il contatore A raggiunga TC, UA alto.
* **S3 (XYZ = 011):** Caricamento del valore TB+1 nel contatore B.
* **S4 (XYZ = 100):** Attesa che il contatore B raggiunga TC, UB alto.
* **S5 (XYZ = 101):** Caricamento del valore TC+1 nel contatore C.
* **S6 (XYZ = 110):** Attesa che il contatore C raggiunga TC, UC alto.
* **S7 (XYZ = 111):** Non utilizzato, transizione diretta a S0.

**Tabella degli stati:**

| Stato | X | Y | Z | TCA | TCB | TCC

## 1.2 Esercizio con recupero diretto

In [44]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image)

--------Response--------
## Progettazione della CONTROL FSM

La CONTROL FSM è una macchina di Moore sincrona con:

* **Ingressi:** TCA, TCB, TCC, CK, -Reset
* **Uscite:** UA, UB, UC, LDA, LDB, LDC
* **Stati:** S0, S1, S2, S3, S4, S5

**Descrizione degli stati:**

* **S0 (Inizializzazione):** Tutti i segnali di uscita sono bassi. I contatori vengono caricati con i valori di TA, TB e TC.
* **S1 (Attesa ciclo iniziale):** Tutti i segnali di uscita rimangono bassi per un ciclo di clock.
* **S2 (Generazione UA):** Il segnale UA è alto, mentre UB e UC sono bassi. Il contatore A viene decrementato ad ogni ciclo di clock.
* **S3 (Attesa tra UA e UB):** Tutti i segnali di uscita sono bassi per un ciclo di clock.
* **S4 (Generazione UB):** Il segnale UB è alto, mentre UA e UC sono bassi. Il contatore B viene decrementato ad ogni ciclo di clock.
* **S5 (Attesa tra UB e UC):** Tutti i segnali di uscita sono bassi per un ciclo di clock.
* **S6 (Generazione UC):** Il segnale UC è alto, mentre UA e U

## 1.3 Esercizio con recupero diretto e video soluzione

In [46]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Di seguito è presente del contesto addizionale che dovrebbe essere utilizzato per produrre la risposta:
{text_retrieved}
Per rispondere in maniera corretta utilizza anche la soluzione fornita in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user wants to design a programmable multi-phase square wave generator. The device should generate three cyclic signals, UA, UB, and UC, with programmable durations TA, TB, and TC. The system architecture is described in the image provided.

The user wants to design the CONTROL FSM as a synchronous Moore machine and create its state table.

Here is the state table for the CONTROL FSM:

| State | W | Z | Y | X | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC | Stato Successivo | W' | Z' | Y' | X' |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | 0 | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | b | 0 | 0 | 0 | 1 |
| b | 0 | 0 | 0 | 1 | 0 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | c | 0 | 0 | 1 | 0 |
| b | 0 | 0 | 0 | 1 | 1 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | d | 0 | 0 | 1 | 1 |
| c | 0 | 0 | 1 | 0 | 0 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | c | 0 | 0 | 1 | 0 |
| c | 0 | 0 | 1 | 0 | 1 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | d | 0 | 0 | 1 | 1 |

## 1.4 Esercizio con video soluzione

In [53]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Per rispondere in maniera corretta utilizza la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, video)

--------Response--------
The user is writing a table of states based on a state machine diagram. The table has the following columns:
* W
* Z
* Y
* X
* TCA
* TCB
* TCC
* LDA
* LDB
* LDC
* UA
* UB
* UC
* Stato
* Stato Successivo

The user fills in the table based on the state machine diagram. The diagram shows the different states of the machine and the transitions between them. The transitions are based on the values of the inputs TCA, TCB, and TCC. The user writes the values of the inputs, outputs, and the next state for each state in the table.

The user also assigns binary values to the states. The states are labeled A, B, C, D, E, F, G, H, and I. The user assigns the following binary values to the states:
* A: 0000
* B: 0001
* C: 0010
* D: 0011
* E: 0100
* F: 0101
* G: 0110
* H: 0111
* I: 1000

The user completes the table of states by filling in the binary values for the states and the values of the inputs, outputs, and the next state for each state.

## 1.5 Esercizio con libro di testo nel prompt

In [48]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook)

--------Response--------
## Progettazione della CONTROL FSM

La CONTROL FSM è una macchina di Moore sincrona, quindi le uscite dipendono solo dallo stato corrente e non dagli ingressi. 

**Variabili di stato:**

* X, Y, Z (3 variabili di stato, quindi 8 possibili stati)

**Uscite:**

* UA, UB, UC: segnali di uscita del generatore di onde quadre
* LDA, LDB, LDC: segnali di caricamento per i contatori A, B e C

**Ingressi:**

* TCA, TCB, TCC: segnali di Terminal Count dai contatori A, B e C

**Segnale di clock:**

* CK

**Segnale di inizializzazione:**

* -Reset

**Tabella degli stati:**

| Stato | X | Y | Z | UA | UB | UC | LDA | LDB | LDC | Stato successivo |
|---|---|---|---|---|---|---|---|---|---|---|
| S0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 | S1 |
| S1 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | S2 se TCA=1, S1 altrimenti |
| S2 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | S3 |
| S3 | 0 | 1 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | S4 se TCB=1, S3 altrimenti |
| S4 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | S5 

## 1.6 Esercizio con libro di testo nel prompt e video soluzione

In [49]:
prompt = f"""<OBJECTIVE>
{prompt_text}
</OBJECTIVE>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, image, textbook, video)

--------Response--------
## Tabella degli stati della CONTROL FSM

| Stato | W | Z | Y | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC | Stato Successivo |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | 0 | 0 | 0 | - | - | - | 1 | 0 | 0 | 0 | 0 | 0 | b |
| b | 0 | 0 | 1 | 1 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | c |
| c | 0 | 0 | 1 | 0 | - | - | 0 | 0 | 0 | 1 | 0 | 0 | c |
| c | 0 | 0 | 1 | 1 | - | - | 0 | 1 | 0 | 0 | 0 | 0 | d |
| d | 0 | 1 | 0 | - | 1 | - | 0 | 0 | 0 | 0 | 1 | 0 | e |
| e | 0 | 1 | 0 | - | 0 | - | 0 | 0 | 0 | 0 | 1 | 0 | e |
| e | 0 | 1 | 0 | - | 1 | - | 0 | 0 | 1 | 0 | 0 | 0 | f |
| f | 1 | 0 | 0 | - | - | 1 | 0 | 0 | 0 | 0 | 0 | 1 | g |
| g | 1 | 0 | 0 | - | - | 0 | 0 | 0 | 0 | 0 | 0 | 1 | g |
| g | 1 | 0 | 0 | - | - | 1 | 1 | 0 | 0 | 0 | 0 | 0 | b |

**Descrizione degli stati:**

* **a (000):** Stato iniziale. Tutti i segnali di uscita (UA, UB, UC) sono bassi. Il contatore A viene caricato con il valore presente sugli ingressi PA3...PA0.
* **b (001)

## 1.7 Esercizio con recupero mediante tool

In [52]:
prompt_text = f"""Si vuole progettare un generatore di onde quadre multifase programmabile. L'architettura del sistema è descritta di seguito:
{image_description}

Questo dispositivo genera tre segnali ciclici, UA, UB e UC, che si comportano come di seguito:
- inizialmente, UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA è alto per un numero di cicli di clock pari a (TA+1), UB e UC nel frattempo sono bassi.
- di seguito UA, UB e UC sono tutti bassi per un ciclo di clock;
- dopodiché, UA torna basso, UB diventa alto per un numero di cicli di clock pari a (TB+1), UC riamane basso.
- segue che UA, UB e UC sono tutti bassi per un ciclo di clock;
- infine, UA rimane basso, UB torna basso, UC diventa alto per un numero di cicli di clock pari a (TC+1).

La durata del tempo a uno di ciascun segnale (TA, TB, TC) sono programmabili in modo indipendente. L'architettura del sistema è descritta nella figura {image}. 
Tutti i moduli sono sincronizzati con il clock principale CK. La macchina a stati finiti genera i segnali richiesti UA, UB e UC. I tre contatori universali a 4 bit (Cnt4) sono incaricati di impostare la durata temporale di ciascun segnale di uscita e sono controllati dalla FSM (Finite State Machine). 

Progetta la CONTROL FSM e realizzane la tabella degli stati. """

responses = rag_model.generate_content(
  prompt_text,
  generation_config=generation_config1,
  safety_settings=safety_settings,
  stream=True,
  )

for response in responses:
    print(response.text, end="")

La richiesta riguarda la progettazione di una FSM (Finite State Machine) per un generatore di onde quadre multifase programmabile. La FSM controlla tre contatori a 4 bit (A, B, C) che determinano la durata di tre segnali di uscita (UA, UB, UC). 

Ecco una possibile implementazione della tabella degli stati per la CONTROL FSM:

| Stato | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC | Stato Successivo |
|---|---|---|---|---|---|---|---|---|---|---| 
| Init | x | x | x | 1 | 1 | 1 | 0 | 0 | 0 | Attesa |
| Attesa | x | x | x | 0 | 0 | 0 | 0 | 0 | 0 | A_alto |
| A_alto | 1 | x | x | 0 | 0 | 0 | 1 | 0 | 0 | A_alto |
| A_alto | 0 | x | x | 0 | 0 | 0 | 1 | 0 | 0 | Attesa_B |
| Attesa_B | x | x | x | 0 | 0 | 0 | 0 | 0 | 0 | B_alto |
| B_alto | x | 1 | x | 0 | 0 | 0 | 0 | 1 | 0 | B_alto |
| B_alto | x | 0 | x | 0 | 0 | 0 | 0 | 1 | 0 | Attesa_C |
| Attesa_C | x | x | x | 0 | 0 | 0 | 0 | 0 | 0 | C_alto |
| C_alto | x | x | 1 | 0 | 0 | 0 | 0 | 0 | 1 | C_alto |
| C_alto | x | x | 0 | 0 | 0 | 0 |

ValueError: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "grounding_metadata": {
    "retrieval_queries": [
      "generatore di onde quadre multifase programmabile progettazione controllore fsm"
    ]
  }
}
Response:
{
  "candidates": [
    {
      "grounding_metadata": {
        "retrieval_queries": [
          "generatore di onde quadre multifase programmabile progettazione controllore fsm"
        ]
      }
    }
  ]
}

## 1.8 Esercizio con few-shot prompting

In [50]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image)

--------Response--------
+-------+---+---+---+---+----+----+----+----+----+----+----+----+-----------------+---+---+---+---+
| Stato | X | Y | Z | TCA| TCB| TCC| LDA| LDB| LDC| UA | UB | UC | Stato Successivo | iX| iY| iZ|
+-------+---+---+---+---+----+----+----+----+----+----+----+----+-----------------+---+---+---+---+
|   rs  | 0 | 0 | 0 |  - |  - |  - |  0 |  0 |  0 |  0 |  0 |  0 |        a         | 0 | 0 | 1 |
|   a   | 0 | 0 | 1 |  - |  - |  - |  1 |  0 |  0 |  1 |  0 |  0 |        b         | 0 | 1 | 0 |
|   b   | 0 | 1 | 0 |  0 |  - |  - |  0 |  0 |  0 |  1 |  0 |  0 |        b         | 0 | 1 | 0 |
|   b   | 0 | 1 | 0 |  1 |  - |  - |  0 |  0 |  0 |  0 |  0 |  0 |        c         | 0 | 1 | 1 |
|   c   | 0 | 1 | 1 |  - |  - |  - |  0 |  1 |  0 |  0 |  1 |  0 |        d         | 1 | 0 | 0 |
|   d   | 1 | 0 | 0 |  - |  0 |  - |  0 |  0 |  0 |  0 |  1 |  0 |        d         | 1 | 0 | 0 |
|   d   | 1 | 0 | 0 |  - |  1 |  - |  0 |  0 |  0 |  0 |  0 |  0 |        e         | 1 |

## 1.9 Esercizio con few-shot prompting, libro di testo nel prompt e video soluzione

In [51]:
input = f'{{"Sistema": "{image}", "Richiesta": "{prompt_text_FW}", "Descrizione del sistema": "{image_description}"}}'

prompt = f"""<OBJECTIVE>
{input}
</OBJECTIVE>

<FEW_SHOT_EXAMPLES>
1. Esempio #1
Input: {input1}
Output: {solution1}
2. Esempio #2
Input: {input2}
Output: {solution2}
3. Esempio #3
Input: {input3}
Output: {solution3}
</FEW_SHOT_EXAMPLES>

<CONTEXT> 
Utilizza il file pdf di seguito per fornire la risposta che contiene informazioni utili sulle macchine a stati finiti:
{textbook}
Per rispondere in maniera corretta utilizza anche la soluzione in formato video:
{video}
</CONTEXT>"""

askMultimodalGemini(prompt, generation_config1, example_image1, example_image2, example_image3, image, textbook, video)

--------Response--------
+-------+---+---+---+-----+-----+-----+-----+-----+-----+-----+-----+-----------------+---+---+---+---+
| Stato | W | Z | Y | X | TCA | TCB | TCC | LDA | LDB | LDC | UA | UB | UC | Stato Successivo | iW | iZ | iY | iX |
+-------+---+---+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------------+---+---+---+---+
|   a   | 0 | 0 | 0 | 0 |  -  |  -  |  -  |  1  |  1  |  1  |  0 |  0 |  0 |        b        | 0 | 0 | 0 | 1 |
|   b   | 0 | 0 | 0 | 1 |  -  |  -  |  -  |  0  |  0  |  0  |  1 |  0 |  0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  0  |  -  |  -  |  0  |  0  |  0  |  1 |  0 |  0 |        c        | 0 | 0 | 1 | 0 |
|   c   | 0 | 0 | 1 | 0 |  1  |  -  |  -  |  0  |  0  |  0  |  0 |  0 |  0 |        d        | 0 | 0 | 1 | 1 |
|   d   | 0 | 0 | 1 | 1 |  -  |  -  |  -  |  0  |  1  |  0  |  0 |  0 |  0 |        e        | 0 | 1 | 0 | 0 |
|   e   | 0 | 1 | 0 | 0 |  -  |  0  |  -  |  0  |  0  |  0  |  0 |  1 |  0 |        